In [1]:
import re

In [2]:
import zipfile, os
import shutil
import tarfile
import re
import xml.etree.ElementTree as etree
import pandas as pd
from bs4 import BeautifulSoup
from xml.etree import ElementTree
import os
import json
import pandas as pd
import re
from nltk.tokenize import sent_tokenize
from pdfrw import PdfReader
import numpy as np
from nltk import ngrams
import scispacy
import spacy

In [4]:
import pandas as pd
short_df1 = pd.read_excel(r'input_data/input_data/all_molecules_with_intervention2.xlsx')
short_df1.head()

XML  \
0  2da20cd7-0125-4728-a219-7cde1329c3e0.xml   
1  b979da00-7c42-457e-b0d4-900c4844459c.xml   
2  ecd677c9-8722-4314-aa15-9c956b38bbd3.xml   
3  53c91dea-5b3c-48fe-ba40-536c77e50012.xml   
4  d1a3a84e-d66f-439c-9874-3c3fdfaf6403.xml   

                                               Title  \
0  Review article: anti-adhesion therapies for in...   
1  Treatment options for patients with multiple s...   
2  Intravenous immunoglobulin for maintenance tre...   
3  Natalizumab improves ambulation in relapsing&#...   
4  Application of Quantitative Pharmacology Appro...   

                                      Full_body_text  \
0  \n \n Introduction \n Inflammatory bowel disea...   
1  \n \n Introduction \n Interferon beta (IFN β )...   
2  \n \n INTRODUCTION \n Multifocal motor neuropa...   
3  \n \n Introduction \n Mobility problems are re...   
4  \n \n Domagrozumab is a human IgG1 monoclonal ...   

                                       Abstract_full                DOI  \
0  {'Abstract': 'A high proportion of patients wi...  10.1111/apt.12639   
1  {'Abstract': 'Although the first‐line disease‐...  10.1111/ene.12299   
2  {'Abstract': 'Intravenous immunoglobulin (IVIg...  10.1111/jns.12268   
3  {'Abstract': 'Impaired ambulation is a promine...  10.1111/ene.12618   
4  {'Abstract': 'Domagrozumab, a monoclonal antib...  10.1002/jcph.1015   

                                 URL     Molecule Publisher_type  \
0  https://doi.org/10.1111/apt.12639  Natalizumab          Wiley   
1  https://doi.org/10.1111/ene.12299  Natalizumab          Wiley   
2  https://doi.org/10.1111/jns.12268  Natalizumab          Wiley   
3  https://doi.org/10.1111/ene.12618  Natalizumab          Wiley   
4  https://doi.org/10.1002/jcph.1015  Natalizumab          Wiley   

                                       Molecule_text  \
0  ['we performed a literature review in pubmed a...   
1  ['natalizumab and fingolimod are approved for ...   
2  ['the main exclusion criteria were receiving (...   
3  ['whether natalizumab, a monoclonal antibody s...   
4  ['estimated clearance for various monoclonal a...   

                                        Intervention  ...   Age2_value  \
0  showed that intravenous (iv) infusion of natal...  ...  Not present   
1  in contrast, fingolimod 0.5 mg is a smaller mo...  ...  Not present   
2  the main exclusion criteria were receiving (1)...  ...  Not present   
3  timer: prospective analysis \n \n participants...  ...  Not present   
4  estimated clearance for various monoclonal ant...  ...  Not present   

  Age2_number Age2_groups Unnamed: 0.3 Unnamed: 0.2 Unnamed: 0.1 Unnamed: 0  \
0         0.0         NaN          NaN          NaN          NaN        NaN   
1         0.0         NaN          NaN          NaN          NaN        NaN   
2         0.0         NaN          NaN          NaN          NaN        NaN   
3         0.0         NaN          NaN          NaN          NaN        NaN   
4         0.0         NaN          NaN          NaN          NaN        NaN   

  phase21                                      Intervention2            DOSAGE  
0     NaN  showed that intravenous (iv) infusion of natal...               NaN  
1     NaN  in contrast fingolimod 0.5 mg is a smaller mol...               NaN  
2     NaN  the main exclusion criteria were receiving (1)...               NaN  
3     NaN  timer: prospective analysis   participants and...               NaN  
4     NaN  estimated clearance for various monoclonal ant...  ['0.183 (0.05)']  

[5 rows x 46 columns]

In [5]:
short_df1.head()

XML  \
0  2da20cd7-0125-4728-a219-7cde1329c3e0.xml   
1  b979da00-7c42-457e-b0d4-900c4844459c.xml   
2  ecd677c9-8722-4314-aa15-9c956b38bbd3.xml   
3  53c91dea-5b3c-48fe-ba40-536c77e50012.xml   
4  d1a3a84e-d66f-439c-9874-3c3fdfaf6403.xml   

                                               Title  \
0  Review article: anti-adhesion therapies for in...   
1  Treatment options for patients with multiple s...   
2  Intravenous immunoglobulin for maintenance tre...   
3  Natalizumab improves ambulation in relapsing&#...   
4  Application of Quantitative Pharmacology Appro...   

                                      Full_body_text  \
0  \n \n Introduction \n Inflammatory bowel disea...   
1  \n \n Introduction \n Interferon beta (IFN β )...   
2  \n \n INTRODUCTION \n Multifocal motor neuropa...   
3  \n \n Introduction \n Mobility problems are re...   
4  \n \n Domagrozumab is a human IgG1 monoclonal ...   

                                       Abstract_full                DOI  \
0  {'Abstract': 'A high proportion of patients wi...  10.1111/apt.12639   
1  {'Abstract': 'Although the first‐line disease‐...  10.1111/ene.12299   
2  {'Abstract': 'Intravenous immunoglobulin (IVIg...  10.1111/jns.12268   
3  {'Abstract': 'Impaired ambulation is a promine...  10.1111/ene.12618   
4  {'Abstract': 'Domagrozumab, a monoclonal antib...  10.1002/jcph.1015   

                                 URL     Molecule Publisher_type  \
0  https://doi.org/10.1111/apt.12639  Natalizumab          Wiley   
1  https://doi.org/10.1111/ene.12299  Natalizumab          Wiley   
2  https://doi.org/10.1111/jns.12268  Natalizumab          Wiley   
3  https://doi.org/10.1111/ene.12618  Natalizumab          Wiley   
4  https://doi.org/10.1002/jcph.1015  Natalizumab          Wiley   

                                       Molecule_text  \
0  ['we performed a literature review in pubmed a...   
1  ['natalizumab and fingolimod are approved for ...   
2  ['the main exclusion criteria were receiving (...   
3  ['whether natalizumab, a monoclonal antibody s...   
4  ['estimated clearance for various monoclonal a...   

                                        Intervention  ...   Age2_value  \
0  showed that intravenous (iv) infusion of natal...  ...  Not present   
1  in contrast, fingolimod 0.5 mg is a smaller mo...  ...  Not present   
2  the main exclusion criteria were receiving (1)...  ...  Not present   
3  timer: prospective analysis \n \n participants...  ...  Not present   
4  estimated clearance for various monoclonal ant...  ...  Not present   

  Age2_number Age2_groups Unnamed: 0.3 Unnamed: 0.2 Unnamed: 0.1 Unnamed: 0  \
0         0.0         NaN          NaN          NaN          NaN        NaN   
1         0.0         NaN          NaN          NaN          NaN        NaN   
2         0.0         NaN          NaN          NaN          NaN        NaN   
3         0.0         NaN          NaN          NaN          NaN        NaN   
4         0.0         NaN          NaN          NaN          NaN        NaN   

  phase21                                      Intervention2            DOSAGE  
0     NaN  showed that intravenous (iv) infusion of natal...               NaN  
1     NaN  in contrast fingolimod 0.5 mg is a smaller mol...               NaN  
2     NaN  the main exclusion criteria were receiving (1)...               NaN  
3     NaN  timer: prospective analysis   participants and...               NaN  
4     NaN  estimated clearance for various monoclonal ant...  ['0.183 (0.05)']  

[5 rows x 46 columns]

In [6]:
short_df1.columns

Index(['XML', 'Title', 'Full_body_text', 'Abstract_full', 'DOI', 'URL',
       'Molecule', 'Publisher_type', 'Molecule_text', 'Intervention',
       'STRENGTH', 'DRUG', 'Molecule_in_Title', 'DISEASE_abstract',
       'DISEASE_title', 'DISEASE', 'Title_short', 'combined_text',
       '1st_line/2nd_line', 'Patients', 'ORR_abstract', 'ORR_fulltext',
       'ORR_short', 'OS_short', 'PFS_short', 'Comparator', 'Arms',
       'Mono/Combi', 'phase', 'PHASES', 'phase2', 'PHASES2', 'Age',
       'Age_Number', 'age_groups', 'Age2', 'Age2_value', 'Age2_number',
       'Age2_groups', 'Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1',
       'Unnamed: 0', 'phase21', 'Intervention2', 'DOSAGE'],
      dtype='object')

In [7]:
short_df1.Molecule.value_counts()

Rituximab        1926
Nivolumab        1829
Pembrolizumab    1755
Atezolizumab     1072
Ipilimumab        873
Tocilizumab       721
ipilimumab        543
Ustekinumab       435
Durvalumab        427
Mepolizumab       290
Canakinumab       289
Tremelimumab      275
Daratumumab       272
Avelumab          271
Dupilumab         227
Ofatumumab        222
Ixekizumab        206
Pertuzumab        198
Natalizumab       150
Daclizumab        149
Benralizumab      147
Brodalumab        145
Guselkumab        141
Ocrelizumab       108
Reslizumab         96
Sarilumab          96
Risankizumab       92
Tildrakizumab      86
Emicizumab         76
Cemiplimab         68
Sintilimab         68
Isatuximab         65
Tislelizumab       52
Spartalizumab      14
Trastuzumab         4
Quavonlimab         3
Secukinumab         1
Name: Molecule, dtype: int64

In [8]:
short_df1.shape

(13392, 46)

In [9]:
import pandas as pd
short_df2 = pd.read_excel(r'input_data/input_data/all_molecules_with_intervention3_updated_v2_cleaned (1).xlsx')
short_df2.head()

XML  \
0  0105196e-6a01-4c12-948d-2711408448a0.xml   
1  0156732d-513b-4887-9f28-c165941bd584.xml   
2  01cd980e-0c6c-44f4-b63d-1c22c0aa6c6b.xml   
3  01e55ab3-1713-41cc-89f5-0c262fd81bda.xml   
4  02654351-4837-4c7f-b767-79a22ed67500.xml   

                                               Title  \
0          Secukinumab on Refractory Lupus Nephritis   
1  Efficacy and safety of secukinumab in patients...   
2  MPO-ANCA-associated necrotizing glomerulonephr...   
3  SARS-COV-2 Infection, Vaccination, and Immune-...   
4  A Reappraisal on the Potential Ability of Huma...   

                                       Abstract_full  \
0  Lupus nephritis (LN) is the most frequent seve...   
1  The appropriate dosing regimens of secukinumab...   
2  Renal involvement in rheumatoid arthritis (RA)...   
3  The relationship between infections or vaccine...   
4  Neutrophils are known to perform a series of e...   

                                      Full_body_text  \
0  {'Introduction': 'Systemic lupus erythematosus...   
1  {'INTRODUCTION': 'Psoriatic arthritis (PsA) is...   
2  {'Implication for health policy/practice/resea...   
3  {'Introduction': 'Relationships between viral ...   
4  {'Introduction': 'The IL-17 family of cytokine...   

                          DOI                                           URL  \
0        10.7759/cureus.17198        http://dx.doi.org/10.7759/cureus.17198   
1  10.6061/clinics/2021/e2820  http://dx.doi.org/10.6061/clinics/2021/e2820   
2        10.15171/jnp.2017.10        http://dx.doi.org/10.15171/jnp.2017.10   
3   10.3389/fimmu.2022.859550   http://dx.doi.org/10.3389/fimmu.2022.859550   
4    10.3389/fimmu.2018.00795    http://dx.doi.org/10.3389/fimmu.2018.00795   

      Molecule Publisher_type  \
0  Secukinumab            NLM   
1  Secukinumab            NLM   
2  Secukinumab            NLM   
3  Secukinumab            NLM   
4  Secukinumab            NLM   

                                       Molecule_text  \
0  ['thus, it was decided to start treatment with...   
1  ['the appropriate dosing regimens of secukinum...   
2  ['a 55-year-old man with a 27-year history of ...   
3  ['prednisone was added to the biologic-disease...   
4  ['neutrophils were then suspended at 5\\u2009×...   

                                        Intervention  ... FEV1 SGRQ ACQ NEDA  \
0  it was decided to suspend belimumab in april 2...  ...  NaN  NaN NaN  NaN   
1  a systematic search was conducted using major ...  ...  NaN  NaN NaN  NaN   
2  after one year of treatment with an anti-inter...  ...  NaN  NaN NaN  NaN   
3  prednisone was added to the biologic-disease-m...  ...  NaN  NaN NaN  NaN   
4  neutrophils were then suspended at 5×106/ml in...  ...  NaN  NaN NaN  NaN   

  AQLQ AQoL                               Intervention_cleaned  \
0  NaN  NaN  it was decided to suspend belimumab in april 2...   
1  NaN  NaN  a systematic search was conducted using major ...   
2  NaN  NaN  after one year of treatment with an anti-inter...   
3  NaN  NaN  prednisone was added to the biologic-disease-m...   
4  NaN  NaN  neutrophils were then suspended at 5×106/ml in...   

                                        Dosage_clean PHASES21  \
0                               ['300 mg', '760 mg']      NaN   
1                               ['300 mg', '150 mg']      NaN   
2                                ['2.6 g', '3.1 mg']      NaN   
3                                          ['25 mg']      NaN   
4  ['2 μg', '1 μg', '50 μg', '20 μg', '20 ng', '5...      NaN   

                          DISEASE_title_filtered  
0                            {'lupus nephritis'}  
1                        {'psoriatic arthritis'}  
2  {'rheumatoid arthritis','glomerulonephritis'}  
3                                  {'infection'}  
4                                      {'IL-17'}  

[5 rows x 85 columns]

In [10]:
short_df2.Molecule.value_counts()

Secukinumab     185
Tralokinumab     17
Name: Molecule, dtype: int64

In [11]:
import pandas as pd
short_df3 = pd.read_excel(r'input_data/input_data/all_Quavonlimab_df.xlsx')
short_df3.head()

XML  \
0  18e921a8-a474-472b-87b8-3b485aba6523.xml   
1  1cbe3767-2ba9-4c0b-84ce-b317f0de97f5.xml   
2  2219f6c7-9818-48ff-a7f4-98738af78c93.xml   
3  31608097-86a2-445e-9857-4664864d34ed.xml   
4  361eee71-47c4-49c9-852d-a91a9fe89280.xml   

                                               Title  \
0  Clinical progress of immunotherapy for small c...   
1  Advanced Nanotechnology for Enhancing Immune C...   
2  Immunotherapy for Hepatocellular Carcinoma: Cu...   
3  Clinical Insights Into Novel Immune Checkpoint...   
4  Addressing the Elephant in the Immunotherapy R...   

                                       Abstract_full  \
0  Small cell lung cancer is a malignant tumor wi...   
1  Immune checkpoint receptor signaling pathways ...   
2  Hepatocellular carcinoma (HCC) is the most pre...   
3  The success of immune checkpoint inhibitors (I...   
4  Immunotherapy has transformed the treatment of...   

                                      Full_body_text  \
0  {'': 'Small cell lung cancer (SCLC) is a type ...   
1  {'1. Introduction': 'Immunotherapy is a modern...   
2  {'Introduction': 'Hepatocellular carcinoma (HC...   
3  {'Background': 'Cancer cells have characterist...   
4  {'1. Introduction': 'Cytotoxic T-Lymphocyte As...   

                                    DOI  \
0  10.3779/j.issn.1009-3419.2022.102.15   
1                  10.3390/nano11030661   
2             10.3389/fimmu.2021.765101   
3             10.3389/fphar.2021.681320   
4               10.3390/cancers14061580   

                                                 URL     Molecule  \
0  http://dx.doi.org/10.3779/j.issn.1009-3419.202...  Quavonlimab   
1             http://dx.doi.org/10.3390/nano11030661  Quavonlimab   
2        http://dx.doi.org/10.3389/fimmu.2021.765101  Quavonlimab   
3        http://dx.doi.org/10.3389/fphar.2021.681320  Quavonlimab   
4          http://dx.doi.org/10.3390/cancers14061580  Quavonlimab   

  Publisher_type                                      Molecule_text  \
0            NLM  ['in 2021, the preliminary results of a phase ...   
1            NLM  ["ctla-4': 'the first antibodies that progress...   
2            NLM                                                 []   
3            NLM                                                 []   
4            NLM                                                 []   

   Intervention  ...  ARR  exacerbation_rate  EDSS ABR FEV1 SGRQ ACQ NEDA  \
0           NaN  ...  NaN                NaN   NaN NaN  NaN  NaN NaN  NaN   
1           NaN  ...  NaN                NaN   NaN NaN  NaN  NaN NaN  NaN   
2           NaN  ...  NaN                NaN   NaN NaN  NaN  NaN NaN  NaN   
3           NaN  ...  NaN                NaN   NaN NaN  NaN  NaN NaN  NaN   
4           NaN  ...  NaN                NaN   NaN NaN  NaN  NaN NaN  NaN   

  AQLQ  AQoL  
0  NaN   NaN  
1  NaN   NaN  
2  NaN   NaN  
3  NaN   NaN  
4  NaN   NaN  

[5 rows x 81 columns]

In [12]:
import pandas as pd
short_df4 = pd.read_excel(r'input_data/input_data/new_molecules_tobecleaned_updated.xlsx')
short_df4.head()

XML  \
0  00f032a4-e43d-473a-8ab4-cb1bbd3e315a.xml   
1  019f6af1-50dd-4187-a48e-9a680c64c19d.xml   
2  0218a206-0392-42ec-9125-4b851b054356.xml   
3  029d2795-cd14-4ed9-9177-e7cac3063f60.xml   
4  02d78328-b868-473d-a2f2-a7ad9794b7f8.xml   

                                               Title  \
0                                  Current Abstracts   
1  Tiamulin inhibits TNF-α and alleviates psorias...   
2                             This Issue At A Glance   
3  Safety of etanercept in patients with psoriasi...   
4  Impact of weight and body mass index on high-s...   

                                       Abstract_full  \
0                                                NaN   
1  ['Highlights', '•', 'A high-throughput screeni...   
2                                                NaN   
3                                                NaN   
4                                                NaN   

                                      Full_body_text  \
0  ['\n', '\n', '\n', 'Treatment of Early Rheumat...   
1  ['1', 'Introduction', 'Psoriasis is a chronic ...   
2  ['Risk of bacterial keratitis in daytime conta...   
3  ['\n', '\n', '\n', 'To the Editor:', ' We repo...   
4  ['Background: To determine the impact of adali...   

                              DOI  \
0   10.1016/S0049-0172(11)00038-2   
1  10.1016/j.jdermsci.2022.05.006   
2    10.1016/j.ophtha.2012.06.022   
3      10.1016/j.jaad.2009.07.010   
4      10.1016/j.jaad.2011.11.230   

                                                URL    Molecule  \
0   http://dx.doi.org/10.1016/S0049-0172(11)00038-2  Adalimumab   
1  http://dx.doi.org/10.1016/j.jdermsci.2022.05.006  Adalimumab   
2    http://dx.doi.org/10.1016/j.ophtha.2012.06.022  Adalimumab   
3      http://dx.doi.org/10.1016/j.jaad.2009.07.010  Adalimumab   
4      http://dx.doi.org/10.1016/j.jaad.2011.11.230  Adalimumab   

  Publisher_type                                      Molecule_text  \
0       Elsevier  ['thirty were treated with infliximab 25 with ...   
1       Elsevier  ["'the development of monoclonal antibodies ta...   
2       Elsevier  ["'a multicenter prospective study of 131 pati...   
3       Elsevier  ["'' '' '' '' 'table i' '' '' '' '' '' 'publis...   
4       Elsevier  ["['background: to determine the impact of ada...   

  Molecule_in_Title  ...                                         Comparator  \
0               NaN  ...                              infliximab,etanercept   
1               NaN  ...                                                NaN   
2               NaN  ...                                                NaN   
3               NaN  ...  ribavirin,sulfasalazine,gibofsky,methotrexate,...   
4  Molecule Present  ...                                                NaN   

       Arms Mono/Combi                                        Title_short  \
0  2 ARM(S)      combi                                  Current Abstracts   
1      None       none  Tiamulin inhibits TNF-α and alleviates psorias...   
2  1 ARM(S)       mono                             This Issue At A Glance   
3  2 ARM(S)      combi  Safety of etanercept in patients with psoriasi...   
4  1 ARM(S)       mono  Impact of weight and body mass index on high-s...   

               Age Age_Number age_groups  \
0  age not present          0        NaN   
1  age not present          0        NaN   
2  age not present          0        NaN   
3  age not present          0        NaN   
4  age not present          0        NaN   

                                Intervention_cleaned  \
0  infliximab (5 mg/kg body weight) was given int...   
1  in both models adalimumab (10μg/ml) inhibited ...   
2  subjects 61 males 70 a female with a mean age ...   
3  '' '' '' '' 'table i' '' '' '' '' '' 'publishe...   
4  patients with moderate to severe hs (hs-physic...   

                                  Dosage_clean PHASES_Cleaned  
0                   ['5 mg', '25 mg', '40 mg']            NaN  
1                        

In [13]:
# short_df4.to_excel('/Users/savioel2/Desktop/HawkAISandoz/2022_release/Insights Module/interim_qlik/new_molecules_tobecleaned.xlsx')

In [13]:
short_df4.Molecule.value_counts()

Adalimumab    4235
Etanercept    3934
Name: Molecule, dtype: int64

In [14]:
short_df4.shape

(8169, 33)

In [15]:
short_df3.Molecule.value_counts()

Quavonlimab    24
Name: Molecule, dtype: int64

In [16]:
short_df3.shape

(24, 81)

In [18]:
# short_df4[['phase2', 'PHASES2', 'PHASES']].head(60)

In [17]:
short_df4.columns

Index(['XML', 'Title', 'Abstract_full', 'Full_body_text', 'DOI', 'URL',
       'Molecule', 'Publisher_type', 'Molecule_text', 'Molecule_in_Title',
       'STRENGTH', 'DRUG', 'DISEASE_abstract', 'DISEASE_title', 'DISEASE',
       'combined_text', '1st_line/2nd_line', 'Patients', 'ORR_abstract',
       'ORR_fulltext', 'ORR_short', 'OS_short', 'PFS_short', 'Comparator',
       'Arms', 'Mono/Combi', 'Title_short', 'Age', 'Age_Number', 'age_groups',
       'Intervention_cleaned', 'Dosage_clean', 'PHASES_Cleaned'],
      dtype='object')

In [20]:
# new_mol_df = short_df4[['Title','Title_short','PHASES', 'Patients',
#                                    '1st_line/2nd_line','Intervention_cleaned','Comparator',
#                       'Dosage_clean','URL','DISEASE_title_filtered','Mono/Combi',
#                       'Arms','Age','age_groups']]
# new_mol_df.shape

In [18]:
short_df3.columns

Index(['XML', 'Title', 'Abstract_full', 'Full_body_text', 'DOI', 'URL',
       'Molecule', 'Publisher_type', 'Molecule_text', 'Intervention',
       'Molecule_in_Title', 'STRENGTH', 'DRUG', 'DISEASE_abstract',
       'DISEASE_title', 'DISEASE', 'combined_text', '1st_line/2nd_line',
       'Patients', 'ORR_abstract', 'ORR_fulltext', 'ORR_short', 'OS_short',
       'PFS_short', 'Comparator', 'Arms', 'Mono/Combi', 'Title_short',
       'phase2', 'PHASES2', 'PHASES', 'Age', 'Age_Number', 'age_groups',
       'Number of evaluable subjects', 'ORR', 'OS_percentage', 'OS_month',
       'PFS_percentage', 'PFS_month', 'DOSAGE_VAL', 'exacerbation_abstract',
       'exacerbation_fulltext', 'exacerbation_short', 'NPS_abstract',
       'NPS_fulltext', 'NPS_short', 'ARR_abstract', 'ARR_fulltext',
       'ARR_short', 'EDSS_abstract', 'EDSS_fulltext', 'EDSS_short',
       'ABR_abstract', 'ABR_fulltext', 'ABR_short', 'SGRQ_abstract',
       'SGRQ_fulltext', 'SGRQ_short', 'ACQ_abstract', 'ACQ_fulltext',


In [19]:
short_df2.shape

(202, 85)

In [20]:
short_df1.shape

(13392, 46)

In [119]:
short_df = pd.concat([short_df1,short_df2,short_df3,short_df4], axis=0,sort=False)
short_df.shape

(21787, 98)

In [120]:
short_df.columns

Index(['XML', 'Title', 'Full_body_text', 'Abstract_full', 'DOI', 'URL',
       'Molecule', 'Publisher_type', 'Molecule_text', 'Intervention',
       'STRENGTH', 'DRUG', 'Molecule_in_Title', 'DISEASE_abstract',
       'DISEASE_title', 'DISEASE', 'Title_short', 'combined_text',
       '1st_line/2nd_line', 'Patients', 'ORR_abstract', 'ORR_fulltext',
       'ORR_short', 'OS_short', 'PFS_short', 'Comparator', 'Arms',
       'Mono/Combi', 'phase', 'PHASES', 'phase2', 'PHASES2', 'Age',
       'Age_Number', 'age_groups', 'Age2', 'Age2_value', 'Age2_number',
       'Age2_groups', 'Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1',
       'Unnamed: 0', 'phase21', 'Intervention2', 'DOSAGE',
       'Number of evaluable subjects', 'ORR', 'OS_percentage', 'OS_month',
       'PFS_percentage', 'PFS_month', 'DOSAGE_VAL', 'exacerbation_abstract',
       'exacerbation_fulltext', 'exacerbation_short', 'NPS_abstract',
       'NPS_fulltext', 'NPS_short', 'ARR_abstract', 'ARR_fulltext',
       'ARR_short', 'EDSS

In [25]:
short_df.columns

Index(['Unnamed: 0.4', 'XML', 'Title', 'Full_body_text', 'Abstract_full',
       'DOI', 'URL', 'Molecule', 'Publisher_type', 'Molecule_text',
       'Intervention', 'STRENGTH', 'DRUG', 'Molecule_in_Title',
       'DISEASE_abstract', 'DISEASE_title', 'DISEASE', 'Title_short',
       'combined_text', '1st_line/2nd_line', 'Patients', 'ORR_abstract',
       'ORR_fulltext', 'ORR_short', 'OS_short', 'PFS_short', 'Comparator',
       'Arms', 'Mono/Combi', 'phase', 'PHASES', 'phase2', 'PHASES2', 'Age',
       'Age_Number', 'age_groups', 'Age2', 'Age2_value', 'Age2_number',
       'Age2_groups', 'Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1',
       'Unnamed: 0', 'phase21', 'Intervention2', 'DOSAGE',
       'Number of evaluable subjects', 'ORR', 'OS_percentage', 'OS_month',
       'PFS_percentage', 'PFS_month', 'DOSAGE_VAL', 'exacerbation_abstract',
       'exacerbation_fulltext', 'exacerbation_short', 'NPS_abstract',
       'NPS_fulltext', 'NPS_short', 'ARR_abstract', 'ARR_fulltext',
       'A

In [30]:
short_df

XML  \
0     2da20cd7-0125-4728-a219-7cde1329c3e0.xml   
1     b979da00-7c42-457e-b0d4-900c4844459c.xml   
2     ecd677c9-8722-4314-aa15-9c956b38bbd3.xml   
3     53c91dea-5b3c-48fe-ba40-536c77e50012.xml   
4     d1a3a84e-d66f-439c-9874-3c3fdfaf6403.xml   
5     fe4f166b-c4ee-4356-8225-b67f3c0bb8ca.xml   
6     9afd06e7-31b1-4eb9-9e9c-07782b63400d.xml   
7     468cb4e3-6cec-4ff8-85fa-7e1eeb87f0fd.xml   
8     c11b4865-ceec-4566-837f-c62b7f65c993.xml   
9     edf10fd7-46d0-4c41-a0f3-56451b8e5124.xml   
10    cdab46fb-e282-421f-a3eb-cef7d3aab403.xml   
11    b58aa178-fe45-4fb0-8b27-604d69dd7b21.xml   
12    b0d4787a-7f52-4850-a120-d1d3567b3cd9.xml   
13    c90d9fcf-0021-47b7-af91-a91530f107ef.xml   
14    3cba0e7d-164e-47e1-9acc-c2f58906cf37.xml   
15    0749ac49-4bff-4210-b1cf-d6152db6fa23.xml   
16    ed33c249-b022-4474-a7af-8a5072ca7e96.xml   
17    634ba728-aa7a-44f4-be49-be1a18e4a354.xml   
18    530caae7-74a9-4c3b-9977-ed62f918a16c.xml   
19    39a8d70a-492e-4b6a-b2a9-4683fcd41f8f.xml   
20    ee27a2f0-953c-4376-8374-c483aee80664.xml   
21    9cc217c2-56df-4582-950e-e37ebd4b9326.xml   
22    f28c0497-3a2c-4da6-a9a8-f88628529aba.xml   
23    f35ef6ea-4a6e-461e-b6c1-972039914957.xml   
24    64379b97-4601-4821-bdb5-b310c6c6e67e.xml   
25    58d5546d-f559-43ac-bb94-2d4ad2d40d2a.xml   
26    b129cc5e-285f-4928-a560-a03a2d7c7a22.xml   
27    cb74abed-5ed4-4029-9117-078efbce9780.xml   
28    b06c6c50-069f-4e22-8701-3c657477de1d.xml   
29    f5b54f94-638b-48e8-94c2-33288d13e506.xml   
...                                        ...   
8139  fa1ce4bd-0435-4485-8b25-a8a1e20a825f.xml   
8140  fa5a8c19-d879-450b-a4cc-b1713899fe8b.xml   
8141  fa8059de-5ff1-49ac-aff7-9b52de669a9f.xml   
8142  faa15633-9789-4742-9c6e-e5feff7ae6fb.xml   
8143  fabd969e-d1a1-4e67-a009-e5624bb2534c.xml   
8144  fac56fbd-b582-4e88-a18b-3fe935dd52c2.xml   
8145  facde1b2-65bc-4dac-833e-0d7e578b38aa.xml   
8146  fad08045-7720-4297-9388-07a558201c85.xml   
8147  fb0c5b81-591a-4434-a3a3-0bad12be99f3.xml   
8148  fb2177d1-b7d5-4632-9013-5d83162dd111.xml   
8149  fb62cbb2-8867-44c8-8907-bf458f6e5b94.xml   
8150  fb8611c5-21b8-4ba1-9fe9-c60f75a5ceb9.xml   
8151  fb91d08a-22c4-4963-896d-e2fe64cc964a.xml   
8152  fb9b0a8a-0e1d-4ea0-98d1-fbafe67d36cf.xml   
8153  fba562cd-37da-4b74-b501-d6f22d04bddd.xml   
8154  fba72093-bb09-4de1-a691-971c0d263053.xml   
8155  fba74847-62a6-499e-8d50-7e42045a6132.xml   
8156  fbb324d7-1502-4b9e-9e79-d5df613165eb.xml   
8157  fbca531d-af7b-403f-b0d2-80ca65ca5f8a.xml   
8158  fbdc4654-9ed1-41a9-aeb2-4fa7df5eb9a1.xml   
8159  fbe5db4c-bff0-4e31-9a6f-7e62386765f6.xml   
8160  fbf16690-32e7-40ee-9b56-0bb416c0f0a9.xml   
8161  fbfc8576-6ded-4324-9c05-b5d9e63385a3.xml   
8162  fc03532a-7473-4553-94a6-57a996b9003e.xml   
8163  fc09dd4f-1489-4dc7-b9c3-08047aafcb1e.xml   
8164  fc23a41e-50fa-421c-b9f5-ceef8e86743a.xml   
8165  fc49ba3a-1b7d-4484-a45c-cae5cc900101.xml   
8166  fc82b4ea-6d8f-41ec-941d-c77d1db1fc31.xml   
8167  fc97341b-f488-4117-a093-bf3c1b1a70d2.xml   
8168  fc9aa26f-355c-4720-bbfa-effa75b61916.xml   

                                                  Title  \
0     Review article: anti-adhesion therapies for in...   
1     Treatment options for patients with multiple s...   
2     Intravenous immunoglobulin for maintenance tre...   
3     Natalizumab improves ambulation in relapsing&#...   
4     Application of Quantitative Pharmacology Appro...   
5     Current and emerging disease-modulatory therap...   
6     Immunoadsorption and plasma exchange&#8212;Eff...   
7     Modeling the Efficacy of Natalizumab in Multip...   
8     Clinical pharmacology, safety and effects of a...   
9     Paediatric multiple sclerosis: a new era in di...   
10    Effect of natalizumab on brain atrophy and dis...   
11    Therapeutic Application of Monoclonal Antibodi...   
12    A Randomized Trial Evaluating Various Administ...   
13    Established and novel disease-modifying treatm...   
14    Wielding the Double-Edged Sword of Inflammatio...  

In [121]:
short_df.Molecule.value_counts()

Adalimumab       4235
Etanercept       3934
Rituximab        1926
Nivolumab        1829
Pembrolizumab    1755
Atezolizumab     1072
Ipilimumab        873
Tocilizumab       721
ipilimumab        543
Ustekinumab       435
Durvalumab        427
Mepolizumab       290
Canakinumab       289
Tremelimumab      275
Daratumumab       272
Avelumab          271
Dupilumab         227
Ofatumumab        222
Ixekizumab        206
Pertuzumab        198
Secukinumab       186
Natalizumab       150
Daclizumab        149
Benralizumab      147
Brodalumab        145
Guselkumab        141
Ocrelizumab       108
Sarilumab          96
Reslizumab         96
Risankizumab       92
Tildrakizumab      86
Emicizumab         76
Sintilimab         68
Cemiplimab         68
Isatuximab         65
Tislelizumab       52
Quavonlimab        27
Tralokinumab       17
Spartalizumab      14
Trastuzumab         4
Name: Molecule, dtype: int64

In [122]:
import unicodedata
import re
def clean_textcodes(intervention):
    return re.sub(" +"," ",unicodedata.normalize("NFKD",getcleanfulltext(intervention)))
def getcleanfulltext(text):
    text = str(text)
    text = text.replace("\\n", "").replace("\n", "").replace("\t", "").replace("\\t", "").replace("\\'", "").replace(",","").replace("\\xa0","").replace("\\u2009","").replace("\\u2005","").replace("\\u2003","").replace("\\u2008","").replace("\\u2002","").replace("\\u200a","").replace("\\u202f","")
    return re.sub(" +"," ",text)

short_df['Abstract_full'] = short_df.Abstract_full.apply(clean_textcodes)
short_df['Full_body_text'] = short_df.Full_body_text.apply(clean_textcodes)
short_df.Abstract_full.to_list()

["{'Abstract': 'A high proportion of patients with inflammatory bowel disease (IBD) do not achieve clinical remission with the current therapies including mesalazine (mesalamine) immunossupresants (IMS) and antibodies against tumour necrosis factor (anti‐TNF). Moreover IMS and anti‐TNF involve a nonnegligible risk for infections and/or malignancies. The anti‐adhesion molecules are one of the most interesting new treatments because of their gut‐selectivity. To review the physiopathology of the adhesion molecules and the current drugs targeting this mechanism. We performed a literature review in PubMed and in clinicaltrials.gov using the terms ‘anti‐adhesion molecules’ ‘inflammatory bowel disease’ ‘natalizumab’ ‘vedolizumab’ ‘AMG181’ ‘Etrolizumab’ ‘PF‐00547659’ ‘AJM300’ ‘Alicaforsen’ and ‘CCX282‐B’ up to November 2013. A total of eight drugs were found including those targeting the α4β1 α4β7 or αEβ7 integrins as well as the ICAM‐1 and MAdCAM‐1 addressins and the chemokine receptor 9. The

In [123]:
def number_of_patients(text): 
    text = str(text)   
    text = text.strip()
    try:
        patient= re.findall(r'[-+]?\d*\.?\d+ *patients|\d+ *patient', text)
        if patient:
            return patient[0]
        else:
            return 'NA'
    except:
        return 'NA'


short_df['Number of evaluable subjects'] = short_df['Abstract_full'].apply(number_of_patients)
short_df[['Abstract_full','Number of evaluable subjects']]

Abstract_full  \
0     {'Abstract': 'A high proportion of patients wi...   
1     {'Abstract': 'Although the first‐line disease‐...   
2     {'Abstract': 'Intravenous immunoglobulin (IVIg...   
3     {'Abstract': 'Impaired ambulation is a promine...   
4     {'Abstract': 'Domagrozumab a monoclonal antibo...   
5     {'Abstract': "The treatment of multiple sclero...   
6     {'Abstract': 'Therapeutic plasma exchange (TPE...   
7     {'Abstract': 'Natalizumab is approved for mult...   
8     {'Abstract': 'AMG 181 pharmacokinetics/pharmac...   
9     {'Abstract': "Multiple sclerosis is a chronic ...   
10    {'Abstract': 'The long‐term benefit of nataliz...   
11    {'Abstract': 'Multiple sclerosis (MS) is a het...   
12    {'Abstract': "The study's primary objective wa...   
13    {'Abstract': 'Multiple sclerosis (MS) is a pre...   
14    {'Abstract': 'Stroke is a leading cause of dis...   
15    {'Abstract': 'Stroke is a leading cause of dis...   
16    {'Abstract': 'Multiple sclerosis (MS) is a chr...   
17    {'Abstract': "The treatment of multiple sclero...   
18    {'Abstract': 'Natalizumab is approved for mult...   
19    {'Abstract': 'Therapeutic plasma exchange (TPE...   
20    {'Abstract': 'To report the relationship betwe...   
21    {'Abstract': 'Twenty per cent of patients with...   
22    {'Abstract': 'Recent advances regarding the in...   
23    {'Abstract': 'To investigate the efficacy and ...   
24    {'Abstract': 'Using the treatment goal of “no ...   
25    {'Abstract': 'Requiring a walking aid is a fun...   
26    {'Abstract': 'Reaching Expanded Disability Sta...   
27    {'Abstract': 'Ocrelizumab is a humanized monoc...   
28    {'Abstract': 'Using the treatment goal of “no ...   
29    {'Abstract': 'To evaluate the safety and effic...   
...                                                 ...   
8139  {'Abstract': 'Phototherapy is a well‐recognize...   
8140  {'Abstract': 'Patients with rheumatoid arthrit...   
8141  {'Abstract': 'To assess whether the intensity ...   
8142                                                 {}   
8143  {'Abstract': 'Graft‐versus‐host disease (GVHD)...   
8144  {'Abstract': 'The efficacy of biologic therapi...   
8145  {'Abstract': 'This article presents updated re...   
8146  {'Abstract': 'To evaluate the use of biologica...   
8147  {'Abstract': 'To evaluate and compare the rete...   
8148                                                 {}   
8149                                                 {}   
8150  {'Abstract': 'Linked Article: Bisschop et al. ...   
8151  {'Abstract': 'Gram‐negative infections can res...   
8152  {'Abstract': 'Tildrakizumab is a fairly new ty...   
8153  {'Abstract': "Following a review of the safety...   
8154                                                 {}   
8155  {'Abstract': 'Tumour necrosis factor‐α is a pl...   
8156  {'Abstract': 'The increase in vascular resista...   
8157                                                 {}   
8158  {'Abstract': 'Nanogele haben sich aufgrund ihr...   
8159                                                 {}   
8160  {'Abstract': 'To explore sustainability of ach...   
8161                                                 {}   
8162                                                 {}   
8163  {'Abstract': 'The aim of this study was to inv...   
8164  {'Abstract': 'Background Combination treatment...   
8165  {'Abstract': 'Tumor necrosis factor receptor–a...   
8166  {'Abstract': ' Little is known about thymus fu...   
8167  {'Abstract': 'Abstract: Objectives:An excessiv...   
8168  {'Abstract': 'Abstract: Warts are common and a...   

     Number of evaluable subjects  
0                              NA  
1                              NA  
2                     13 patients  
3                              NA  
4                              NA  
5                              NA  
6                     32 patients  
7                              NA  
8                              NA  
9                      

In [124]:
print("len is - " + str(len(set(short_df['Number of evaluable subjects'].tolist()))))

len is - 1000


In [125]:
def lencheck(abs,fulltxt):
    if abs!= None and fulltxt!=None:
        return abs
    elif abs==None and fulltxt!=None:
        return fulltxt
    elif abs!=None and fulltxt==None:
        return abs
    else:
        return 'NA'

In [126]:
def get_orr_pattern1(cleaned_relevant_lst):
    dynamic_pattern_list = """objective response.*\d*[.,]%.*\d*%.*CI|
    ORR.*\d*[.,]%.*\d*%.*CI|
    response rate.*\d*[.,]%.*\d*%.*CI|
    objective response rate.*\d*[.,]%.*\d*%.*CI|
    overall response rates.*\d*[.,]%.*\d*%.*CI|
    Overall response rate.*\d*[.,]%.*\d*%.*CI|
    orr.*\d*[.,]%.*\d*%.*CI|
    response  rate.*\d*[.,]%.*\d*%.*CI|
    Objective Response Rate.*\d*[.,]%.*\d*%.*CI|
    Overall response rates.*\d*[.,]%.*\d*%.*CI|
    Overall response.*\d*[.,]%.*\d*%.*CI"""
    return [sentence.strip() for sentence in cleaned_relevant_lst if 
            re.findall(dynamic_pattern_list,sentence)]

def get_orr_pattern2(cleaned_relevant_lst):
    dynamic_pattern_list = """objective response.*\d*[.,]%.*\d*%|
    response rate.*\d*[.,]%.*\d*%|
    objective response rate.*\d*[.,]%.*\d*%|
    overall response rates.*\d*[.,]%.*\d*%|
    Overall response rate.*\d*[.,]%.*\d*%|
    orr.*\d*[.,]%.*\d*%|
    response  rate.*\d*[.,]%.*\d*%|
    Objective Response Rate.*\d*[.,]%.*\d*%|
    Overall response rates.*\d*[.,]%.*\d*%|
    Overall response.*\d*[.,]%.*\d*%|
    ORR.*\d*[.,]%.*\d*%|
    least-squares.*\d*%|"""
    return [sentence.strip() for sentence in cleaned_relevant_lst if 
            re.findall(dynamic_pattern_list,sentence)]

def get_orr_pattern3(cleaned_relevant_lst):
    dynamic_pattern_list = "TPS.*\d*%"
    return [sentence.strip() for sentence in cleaned_relevant_lst if 
            re.findall(dynamic_pattern_list,sentence)]
            
min_sent_list = []
orrdatatable = pd.DataFrame([])
orrlst = []
full_length = 0

orr_short = []
def parse_pdfs_getorr_short(text):
        try:           
            short_sentence_list=sent_tokenize(text)
            short_cleaned_relevant_lst = [x.replace('\n','') for x in short_sentence_list]           
            orr_sentence = get_orr_pattern2(short_cleaned_relevant_lst)
            if len(orr_sentence)>0:
                orr_sentence2 = get_orr_pattern2(short_cleaned_relevant_lst)
                if len(orr_sentence2)>0:
                    worddistance_list = []
                    combinations_list = ['objective response.*?%.*?%','response rate.*?%.*?%','ORR.*?%.*?%','response  rate.*?%.*?%','overall response rates.*?%.*?%','objective response rate.*?%.*?%']                     
                    for sent in orr_sentence2:
                        for param in combinations_list:
                            word_length = [len(i.split())-2 for i in re.findall(param,sent)] 
                            if word_length:
                                worddistance_list.append(word_length[0])
                    if len(worddistance_list)>0:
                        min_value = min([i for i in worddistance_list if i > 0])
                        if min_value:
                            min_sent_list = []
                            for sent in orr_sentence2:
                                for param in combinations_list:
#                                     print(re.findall(param,sent))
                                    word_length = [len(i.split())-2 for i in re.findall(param,sent)]                                   
                                    if word_length and word_length[0] == min_value:
                                        min_sent_list.append(sent)
                                        orr_short = re.findall(param,sent)
#                                         print(re.findall(param,sent))
                        if min_sent_list:
                            return orr_short
                        else:
                            return 'NA'
                    else:                        
                        patterns = ['objective response','response rate','objective response rate','response  rate','overall response rate','orr','overall response rates','objective response rate']
                        # sentence_list
                        relevant_list = []
                        for sentence in short_cleaned_relevant_lst:
                            for param in patterns:
                                if param in sentence:
                                    relevant_list.append(sentence)
                        return 'NA'                                                           
                else:
                    return 'NA'
                    

        except:
            return "Not able to parse"
#             orrlst.append([file,"Not able to parse"])
#             print(f'Not able to parse {file}')
    
def lencheck(abs,fulltxt):
    if abs!='NA' and fulltxt!='NA':
        return abs
    elif abs=='NA' and fulltxt!='NA':
        return fulltxt
    elif abs!='NA' and fulltxt=='NA':
        return abs
    else:
        return "NA"


short_df['ORR_abstract'] = short_df.Abstract_full.apply(parse_pdfs_getorr_short)
short_df['ORR_fulltext'] = short_df.Full_body_text.apply(parse_pdfs_getorr_short)
short_df['ORR_short'] = short_df.apply(lambda x: lencheck(x.ORR_abstract, x.ORR_fulltext), axis=1)
short_df['ORR_short']

0                                                      NA
1                                                      NA
2                                                      NA
3                                                      NA
4                                                      NA
5                                                      NA
6                                                      NA
7                                                      NA
8                                                      NA
9                                                      NA
10                                                     NA
11                                                     NA
12                                                     NA
13                                                     NA
14                                                     NA
15                                                     NA
16                                                     NA
17            

In [1]:
def get_orr_pattern1(cleaned_relevant_lst):
    dynamic_pattern_list = """objective response.*\d*[.,]%.*\d*%.*CI|
    ORR.*\d*[.,]%.*\d*%.*CI|
    response rate.*\d*[.,]%.*\d*%.*CI|
    objective response rate.*\d*[.,]%.*\d*%.*CI|
    overall response rates.*\d*[.,]%.*\d*%.*CI|
    Overall response rate.*\d*[.,]%.*\d*%.*CI|
    orr.*\d*[.,]%.*\d*%.*CI|
    response  rate.*\d*[.,]%.*\d*%.*CI|
    Objective Response Rate.*\d*[.,]%.*\d*%.*CI|
    Overall response rates.*\d*[.,]%.*\d*%.*CI|
    Overall response.*\d*[.,]%.*\d*%.*CI"""
    return [sentence.strip() for sentence in cleaned_relevant_lst if 
            re.findall(dynamic_pattern_list,sentence)]

In [3]:
import re
get_orr_pattern1('response rate assuming a response rate of 40%')

[]

In [127]:
def ossentence(text): 
    oslen = []
    worddistance_list = []
    text = str(text)
    try:
        short_sentence_list=sent_tokenize(text)
        short_cleaned_relevant_lst = [x.replace('\n','') for x in short_sentence_list]           
        combinations_list = ['overall survival.*\d*%','OS.*\d*%','Overall survival.*\d*%','estimated rate of overall survival.*\d*%',
                                                 'overall survival.*\d*%','overall survival rate.*\d*%','6-month survival.*\d*%','overall  survival.*\d*%']
        for sent in short_cleaned_relevant_lst:        
            for param in combinations_list:
                word_length = [len(i.split())-2 for i in re.findall(param,sent)] 
                if word_length:
                    worddistance_list.append(word_length[0])
            if len(worddistance_list)>0:
                min_value = min([i for i in worddistance_list if i > 0])
                if min_value:
                    min_sent_list = []
                    for sent in short_cleaned_relevant_lst:
                        for param in combinations_list:
                            word_length = [len(i.split())-2 for i in re.findall(param,sent)]                                   
                            if word_length and word_length[0] == min_value:
                                oslen.append(sent)
                                os_short = re.findall(param,sent)
                            else:
                                'NA'
                else:
                    return 'NA'
                if oslen:
                    return os_short
                else:
                    'NA'
            else:
                'NA'
    except:
            return 'NA'

short_df['OS_short'] = short_df.Full_body_text.apply(ossentence)
short_df['OS_short']

0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
        ... 
8139    None
8140    None
8141    None
8142    None
8143    None
8144    None
8145    None
8146    None
8147    None
8148    None
8149    None
8150    None
8151    None
8152    None
8153    None
8154    None
8155    None
8156    None
8157    None
8158    None
8159    None
8160    None
8161    None
8162    None
8163    None
8164    None
8165    None
8166    None
8167    None
8168    None
Name: OS_short, Length: 21787, dtype: object

In [128]:
def pfssentence(text): 
    oslen = []
    worddistance_list = []
    text = str(text)
    try:
        short_sentence_list=sent_tokenize(text)
        short_cleaned_relevant_lst = [x.replace('\n','') for x in short_sentence_list]           
        combinations_list = ['Median progression-free survival.*?%',
                                         'median PFS.*?%','PFS.*?%','pfs.*?%',
                                         'progression-free survival.*?%']
        for sent in short_cleaned_relevant_lst:        
            for param in combinations_list:
                word_length = [len(i.split())-2 for i in re.findall(param,sent)] 
                if word_length:
                    worddistance_list.append(word_length[0])
            if len(worddistance_list)>0:
                min_value = min([i for i in worddistance_list if i > 0])
                if min_value:
                    min_sent_list = []
                    for sent in short_cleaned_relevant_lst:
                        for param in combinations_list:
                            word_length = [len(i.split())-2 for i in re.findall(param,sent)]                                   
                            if word_length and word_length[0] == min_value:
                                oslen.append(sent)
                                os_short = re.findall(param,sent)
                            else:
                                'NA'
                else:
                    return 'NA'
                if oslen:
                    return os_short
                else:
                    'NA'
            else:
                'NA'
    except:
            return 'NA'

short_df['PFS_short'] = short_df.Full_body_text.apply(pfssentence)
short_df['PFS_short']

0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
        ... 
8139    None
8140    None
8141    None
8142    None
8143    None
8144    None
8145    None
8146    None
8147    None
8148    None
8149    None
8150    None
8151    None
8152    None
8153    None
8154    None
8155    None
8156    None
8157    None
8158    None
8159    None
8160    None
8161    None
8162    None
8163    None
8164    None
8165    None
8166    None
8167    None
8168    None
Name: PFS_short, Length: 21787, dtype: object

In [129]:
def efssentence(text): 
    oslen = []
    worddistance_list = []
    text = str(text)
    try:
        short_sentence_list=sent_tokenize(text)
        short_cleaned_relevant_lst = [x.replace('\n','') for x in short_sentence_list]           
        combinations_list = ['EFS.*?%','event free survival.*?%','Event Free Survival.*?%']
        for sent in short_cleaned_relevant_lst:        
            for param in combinations_list:
                word_length = [len(i.split())-2 for i in re.findall(param,sent)] 
                if word_length:
                    worddistance_list.append(word_length[0])
            if len(worddistance_list)>0:
                min_value = min([i for i in worddistance_list if i > 0])
                if min_value:
                    min_sent_list = []
                    for sent in short_cleaned_relevant_lst:
                        for param in combinations_list:
                            word_length = [len(i.split())-2 for i in re.findall(param,sent)]                                   
                            if word_length and word_length[0] == min_value:
                                oslen.append(sent)
                                os_short = re.findall(param,sent)
                            else:
                                'NA'
                else:
                    return 'NA'
                if oslen:
                    return os_short
                else:
                    'NA'
            else:
                'NA'
    except:
            return 'NA'

short_df['EFS_short'] = short_df.Full_body_text.apply(efssentence)
short_df['EFS_short']

0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
        ... 
8139    None
8140    None
8141    None
8142    None
8143    None
8144    None
8145    None
8146    None
8147    None
8148    None
8149    None
8150    None
8151    None
8152    None
8153    None
8154    None
8155    None
8156    None
8157    None
8158    None
8159    None
8160    None
8161    None
8162    None
8163    None
8164    None
8165    None
8166    None
8167    None
8168    None
Name: EFS_short, Length: 21787, dtype: object

In [130]:
def exacerbation_sentence(text): 
    oslen = []
    worddistance_list = []
    text = str(text).strip()
    try:
        short_sentence_list=sent_tokenize(text)
        short_cleaned_relevant_lst = [x.replace('\n','').replace('\\xa','') for x in short_sentence_list]           
        combinations_list = ['AER.*\d+\.\d{2,}','Asthma Exacerbation Rate.*\d+\.\d{2,}''exacerbation.*\d+\.\d{2,}','exacerbations.*\d+\.\d{2,}','exacerbation rate.*\d', 'exacerbation.*\d', 'exacerbation.*\d per year']
        

        for sent in short_cleaned_relevant_lst:        
            for param in combinations_list:
                word_length = [len(i.split())-2 for i in re.findall(param,sent)] 
                if word_length:
                    worddistance_list.append(word_length[0])
            if len(worddistance_list)>0:
                min_value = min([i for i in worddistance_list if i > 0])
                if min_value:
                    min_sent_list = []
                    for sent in short_cleaned_relevant_lst:
                        for param in combinations_list:
                            word_length = [len(i.split())-2 for i in re.findall(param,sent)]                                   
                            if word_length and word_length[0] == min_value:
                                oslen.append(sent)
                                os_short = re.findall(param,sent)
                            else:
                                'NA'
                else:
                    return 'NA'
                if oslen:
                    return os_short
                else:
                    'NA'
            else:
                'NA'
    except:
            return 'NA'
        
short_df['exacerbation_abstract'] = short_df.Abstract_full.apply(exacerbation_sentence)
short_df['exacerbation_fulltext'] = short_df.Full_body_text.apply(exacerbation_sentence)


short_df['exacerbation_short'] = short_df.apply(lambda x: lencheck(x.exacerbation_abstract, x.exacerbation_fulltext), axis=1)

In [131]:
def ARR_sentence(text): 
    oslen = []
    worddistance_list = []
    text = str(text).strip()
    try:
        short_sentence_list=sent_tokenize(text)
        short_cleaned_relevant_lst = [x.replace('\n','').replace('\\xa','') for x in short_sentence_list]           
        combinations_list = ['ARR.*\d+\.\d{2,}','ARR.*\d', 'ARR.*\d+\.\d{2,}','annualized relapse rate.*\d+\.\d{2,}']
        

        for sent in short_cleaned_relevant_lst:        
            for param in combinations_list:
                word_length = [len(i.split())-2 for i in re.findall(param,sent)] 
                if word_length:
                    worddistance_list.append(word_length[0])
            if len(worddistance_list)>0:
                min_value = min([i for i in worddistance_list if i > 0])
                if min_value:
                    min_sent_list = []
                    for sent in short_cleaned_relevant_lst:
                        for param in combinations_list:
                            word_length = [len(i.split())-2 for i in re.findall(param,sent)]                                   
                            if word_length and word_length[0] == min_value:
                                oslen.append(sent)
                                os_short = re.findall(param,sent)
                            else:
                                'NA'
                else:
                    return 'NA'
                if oslen:
                    return os_short
                else:
                    'NA'
            else:
                'NA' 
    except:
            return 'NA'
        
short_df['ARR_abstract'] = short_df.Abstract_full.apply(ARR_sentence)
short_df['ARR_fulltext'] = short_df.Full_body_text.apply(ARR_sentence)


short_df['ARR_short'] = short_df.apply(lambda x: lencheck(x.ARR_abstract, x.ARR_fulltext), axis=1)



In [133]:
def EDSS_sentence(text): 
    oslen = []
    worddistance_list = []
    text = str(text).strip()
    try:
        short_sentence_list=sent_tokenize(text)
        short_cleaned_relevant_lst = [x.replace('\n','').replace('\\xa','') for x in short_sentence_list]           
        combinations_list = ['EDSS score.*\d+\.\d{2,}','EDSS.*\d+\.\d{2,}']
        

        for sent in short_cleaned_relevant_lst:        
            for param in combinations_list:
                word_length = [len(i.split())-2 for i in re.findall(param,sent)] 
                if word_length:
                    worddistance_list.append(word_length[0])
            if len(worddistance_list)>0:
                min_value = min([i for i in worddistance_list if i > 0])
                if min_value:
                    min_sent_list = []
                    for sent in short_cleaned_relevant_lst:
                        for param in combinations_list:
                            word_length = [len(i.split())-2 for i in re.findall(param,sent)]                                   
                            if word_length and word_length[0] == min_value:
                                oslen.append(sent)
                                os_short = re.findall(param,sent)
                            else:
                                'NA'
                else:
                    return 'NA'
                if oslen:
                    return os_short
                else:
                    'NA'
            else:
                'NA' 
    except:
            return 'NA'
        
short_df['EDSS_abstract'] = short_df.Abstract_full.apply(EDSS_sentence)
short_df['EDSS_fulltext'] = short_df.Full_body_text.apply(EDSS_sentence)

short_df['EDSS_short'] = short_df.apply(lambda x: lencheck(x.EDSS_abstract, x.EDSS_fulltext), axis=1)

In [134]:
short_df

XML  \
0     2da20cd7-0125-4728-a219-7cde1329c3e0.xml   
1     b979da00-7c42-457e-b0d4-900c4844459c.xml   
2     ecd677c9-8722-4314-aa15-9c956b38bbd3.xml   
3     53c91dea-5b3c-48fe-ba40-536c77e50012.xml   
4     d1a3a84e-d66f-439c-9874-3c3fdfaf6403.xml   
5     fe4f166b-c4ee-4356-8225-b67f3c0bb8ca.xml   
6     9afd06e7-31b1-4eb9-9e9c-07782b63400d.xml   
7     468cb4e3-6cec-4ff8-85fa-7e1eeb87f0fd.xml   
8     c11b4865-ceec-4566-837f-c62b7f65c993.xml   
9     edf10fd7-46d0-4c41-a0f3-56451b8e5124.xml   
10    cdab46fb-e282-421f-a3eb-cef7d3aab403.xml   
11    b58aa178-fe45-4fb0-8b27-604d69dd7b21.xml   
12    b0d4787a-7f52-4850-a120-d1d3567b3cd9.xml   
13    c90d9fcf-0021-47b7-af91-a91530f107ef.xml   
14    3cba0e7d-164e-47e1-9acc-c2f58906cf37.xml   
15    0749ac49-4bff-4210-b1cf-d6152db6fa23.xml   
16    ed33c249-b022-4474-a7af-8a5072ca7e96.xml   
17    634ba728-aa7a-44f4-be49-be1a18e4a354.xml   
18    530caae7-74a9-4c3b-9977-ed62f918a16c.xml   
19    39a8d70a-492e-4b6a-b2a9-4683fcd41f8f.xml   
20    ee27a2f0-953c-4376-8374-c483aee80664.xml   
21    9cc217c2-56df-4582-950e-e37ebd4b9326.xml   
22    f28c0497-3a2c-4da6-a9a8-f88628529aba.xml   
23    f35ef6ea-4a6e-461e-b6c1-972039914957.xml   
24    64379b97-4601-4821-bdb5-b310c6c6e67e.xml   
25    58d5546d-f559-43ac-bb94-2d4ad2d40d2a.xml   
26    b129cc5e-285f-4928-a560-a03a2d7c7a22.xml   
27    cb74abed-5ed4-4029-9117-078efbce9780.xml   
28    b06c6c50-069f-4e22-8701-3c657477de1d.xml   
29    f5b54f94-638b-48e8-94c2-33288d13e506.xml   
...                                        ...   
8139  fa1ce4bd-0435-4485-8b25-a8a1e20a825f.xml   
8140  fa5a8c19-d879-450b-a4cc-b1713899fe8b.xml   
8141  fa8059de-5ff1-49ac-aff7-9b52de669a9f.xml   
8142  faa15633-9789-4742-9c6e-e5feff7ae6fb.xml   
8143  fabd969e-d1a1-4e67-a009-e5624bb2534c.xml   
8144  fac56fbd-b582-4e88-a18b-3fe935dd52c2.xml   
8145  facde1b2-65bc-4dac-833e-0d7e578b38aa.xml   
8146  fad08045-7720-4297-9388-07a558201c85.xml   
8147  fb0c5b81-591a-4434-a3a3-0bad12be99f3.xml   
8148  fb2177d1-b7d5-4632-9013-5d83162dd111.xml   
8149  fb62cbb2-8867-44c8-8907-bf458f6e5b94.xml   
8150  fb8611c5-21b8-4ba1-9fe9-c60f75a5ceb9.xml   
8151  fb91d08a-22c4-4963-896d-e2fe64cc964a.xml   
8152  fb9b0a8a-0e1d-4ea0-98d1-fbafe67d36cf.xml   
8153  fba562cd-37da-4b74-b501-d6f22d04bddd.xml   
8154  fba72093-bb09-4de1-a691-971c0d263053.xml   
8155  fba74847-62a6-499e-8d50-7e42045a6132.xml   
8156  fbb324d7-1502-4b9e-9e79-d5df613165eb.xml   
8157  fbca531d-af7b-403f-b0d2-80ca65ca5f8a.xml   
8158  fbdc4654-9ed1-41a9-aeb2-4fa7df5eb9a1.xml   
8159  fbe5db4c-bff0-4e31-9a6f-7e62386765f6.xml   
8160  fbf16690-32e7-40ee-9b56-0bb416c0f0a9.xml   
8161  fbfc8576-6ded-4324-9c05-b5d9e63385a3.xml   
8162  fc03532a-7473-4553-94a6-57a996b9003e.xml   
8163  fc09dd4f-1489-4dc7-b9c3-08047aafcb1e.xml   
8164  fc23a41e-50fa-421c-b9f5-ceef8e86743a.xml   
8165  fc49ba3a-1b7d-4484-a45c-cae5cc900101.xml   
8166  fc82b4ea-6d8f-41ec-941d-c77d1db1fc31.xml   
8167  fc97341b-f488-4117-a093-bf3c1b1a70d2.xml   
8168  fc9aa26f-355c-4720-bbfa-effa75b61916.xml   

                                                  Title  \
0     Review article: anti-adhesion therapies for in...   
1     Treatment options for patients with multiple s...   
2     Intravenous immunoglobulin for maintenance tre...   
3     Natalizumab improves ambulation in relapsing&#...   
4     Application of Quantitative Pharmacology Appro...   
5     Current and emerging disease-modulatory therap...   
6     Immunoadsorption and plasma exchange&#8212;Eff...   
7     Modeling the Efficacy of Natalizumab in Multip...   
8     Clinical pharmacology, safety and effects of a...   
9     Paediatric multiple sclerosis: a new era in di...   
10    Effect of natalizumab on brain atrophy and dis...   
11    Therapeutic Application of Monoclonal Antibodi...   
12    A Randomized Trial Evaluating Various Administ...   
13    Established and novel disease-modifying treatm...   
14    Wielding the Double-Edged Sword of Inflammatio...  

In [135]:
def ABR_sentence(text): 
    oslen = []
    worddistance_list = []
    text = str(text).strip()
    try:
        short_sentence_list=sent_tokenize(text)
        short_cleaned_relevant_lst = [x.replace('\n','').replace('\\xa','') for x in short_sentence_list]           
        combinations_list = ['ABR.*\d+\.\d{2,}','ABR.*\d', 'ABR.*\d+\.\d{2,} events','annualized bleeding rate.*\d+\.\d{2,}']
        

        for sent in short_cleaned_relevant_lst:        
            for param in combinations_list:
                word_length = [len(i.split())-2 for i in re.findall(param,sent)] 
                if word_length:
                    worddistance_list.append(word_length[0])
            if len(worddistance_list)>0:
                min_value = min([i for i in worddistance_list if i > 0])
                if min_value:
                    min_sent_list = []
                    for sent in short_cleaned_relevant_lst:
                        for param in combinations_list:
                            word_length = [len(i.split())-2 for i in re.findall(param,sent)]                                   
                            if word_length and word_length[0] == min_value:
                                oslen.append(sent)
                                os_short = re.findall(param,sent)
                            else:
                                'NA'
                else:
                    return 'NA'
                if oslen:
                    return os_short
                else:
                    'NA'
            else:
                'NA' 
    except:
            return 'NA'
        
short_df['ABR_abstract'] = short_df.Abstract_full.apply(ABR_sentence)
short_df['ABR_fulltext'] = short_df.Full_body_text.apply(ABR_sentence)

short_df['ABR_short'] = short_df.apply(lambda x: lencheck(x.ABR_abstract, x.ABR_fulltext), axis=1)

In [137]:
short_df

XML  \
0     2da20cd7-0125-4728-a219-7cde1329c3e0.xml   
1     b979da00-7c42-457e-b0d4-900c4844459c.xml   
2     ecd677c9-8722-4314-aa15-9c956b38bbd3.xml   
3     53c91dea-5b3c-48fe-ba40-536c77e50012.xml   
4     d1a3a84e-d66f-439c-9874-3c3fdfaf6403.xml   
5     fe4f166b-c4ee-4356-8225-b67f3c0bb8ca.xml   
6     9afd06e7-31b1-4eb9-9e9c-07782b63400d.xml   
7     468cb4e3-6cec-4ff8-85fa-7e1eeb87f0fd.xml   
8     c11b4865-ceec-4566-837f-c62b7f65c993.xml   
9     edf10fd7-46d0-4c41-a0f3-56451b8e5124.xml   
10    cdab46fb-e282-421f-a3eb-cef7d3aab403.xml   
11    b58aa178-fe45-4fb0-8b27-604d69dd7b21.xml   
12    b0d4787a-7f52-4850-a120-d1d3567b3cd9.xml   
13    c90d9fcf-0021-47b7-af91-a91530f107ef.xml   
14    3cba0e7d-164e-47e1-9acc-c2f58906cf37.xml   
15    0749ac49-4bff-4210-b1cf-d6152db6fa23.xml   
16    ed33c249-b022-4474-a7af-8a5072ca7e96.xml   
17    634ba728-aa7a-44f4-be49-be1a18e4a354.xml   
18    530caae7-74a9-4c3b-9977-ed62f918a16c.xml   
19    39a8d70a-492e-4b6a-b2a9-4683fcd41f8f.xml   
20    ee27a2f0-953c-4376-8374-c483aee80664.xml   
21    9cc217c2-56df-4582-950e-e37ebd4b9326.xml   
22    f28c0497-3a2c-4da6-a9a8-f88628529aba.xml   
23    f35ef6ea-4a6e-461e-b6c1-972039914957.xml   
24    64379b97-4601-4821-bdb5-b310c6c6e67e.xml   
25    58d5546d-f559-43ac-bb94-2d4ad2d40d2a.xml   
26    b129cc5e-285f-4928-a560-a03a2d7c7a22.xml   
27    cb74abed-5ed4-4029-9117-078efbce9780.xml   
28    b06c6c50-069f-4e22-8701-3c657477de1d.xml   
29    f5b54f94-638b-48e8-94c2-33288d13e506.xml   
...                                        ...   
8139  fa1ce4bd-0435-4485-8b25-a8a1e20a825f.xml   
8140  fa5a8c19-d879-450b-a4cc-b1713899fe8b.xml   
8141  fa8059de-5ff1-49ac-aff7-9b52de669a9f.xml   
8142  faa15633-9789-4742-9c6e-e5feff7ae6fb.xml   
8143  fabd969e-d1a1-4e67-a009-e5624bb2534c.xml   
8144  fac56fbd-b582-4e88-a18b-3fe935dd52c2.xml   
8145  facde1b2-65bc-4dac-833e-0d7e578b38aa.xml   
8146  fad08045-7720-4297-9388-07a558201c85.xml   
8147  fb0c5b81-591a-4434-a3a3-0bad12be99f3.xml   
8148  fb2177d1-b7d5-4632-9013-5d83162dd111.xml   
8149  fb62cbb2-8867-44c8-8907-bf458f6e5b94.xml   
8150  fb8611c5-21b8-4ba1-9fe9-c60f75a5ceb9.xml   
8151  fb91d08a-22c4-4963-896d-e2fe64cc964a.xml   
8152  fb9b0a8a-0e1d-4ea0-98d1-fbafe67d36cf.xml   
8153  fba562cd-37da-4b74-b501-d6f22d04bddd.xml   
8154  fba72093-bb09-4de1-a691-971c0d263053.xml   
8155  fba74847-62a6-499e-8d50-7e42045a6132.xml   
8156  fbb324d7-1502-4b9e-9e79-d5df613165eb.xml   
8157  fbca531d-af7b-403f-b0d2-80ca65ca5f8a.xml   
8158  fbdc4654-9ed1-41a9-aeb2-4fa7df5eb9a1.xml   
8159  fbe5db4c-bff0-4e31-9a6f-7e62386765f6.xml   
8160  fbf16690-32e7-40ee-9b56-0bb416c0f0a9.xml   
8161  fbfc8576-6ded-4324-9c05-b5d9e63385a3.xml   
8162  fc03532a-7473-4553-94a6-57a996b9003e.xml   
8163  fc09dd4f-1489-4dc7-b9c3-08047aafcb1e.xml   
8164  fc23a41e-50fa-421c-b9f5-ceef8e86743a.xml   
8165  fc49ba3a-1b7d-4484-a45c-cae5cc900101.xml   
8166  fc82b4ea-6d8f-41ec-941d-c77d1db1fc31.xml   
8167  fc97341b-f488-4117-a093-bf3c1b1a70d2.xml   
8168  fc9aa26f-355c-4720-bbfa-effa75b61916.xml   

                                                  Title  \
0     Review article: anti-adhesion therapies for in...   
1     Treatment options for patients with multiple s...   
2     Intravenous immunoglobulin for maintenance tre...   
3     Natalizumab improves ambulation in relapsing&#...   
4     Application of Quantitative Pharmacology Appro...   
5     Current and emerging disease-modulatory therap...   
6     Immunoadsorption and plasma exchange&#8212;Eff...   
7     Modeling the Efficacy of Natalizumab in Multip...   
8     Clinical pharmacology, safety and effects of a...   
9     Paediatric multiple sclerosis: a new era in di...   
10    Effect of natalizumab on brain atrophy and dis...   
11    Therapeutic Application of Monoclonal Antibodi...   
12    A Randomized Trial Evaluating Various Administ...   
13    Established and novel disease-modifying treatm...   
14    Wielding the Double-Edged Sword of Inflammatio...  

In [138]:
def SGRQ_sentence(text): 
    oslen = []
    worddistance_list = []
    text = str(text).strip()
    try:
        short_sentence_list=sent_tokenize(text)
        short_cleaned_relevant_lst = [x.replace('\n','').replace('\\xa','') for x in short_sentence_list]           
        combinations_list = ['SGRQ.*\d+\.\d{2,}','SGRQ.*\d%', 'SGRQ.*\d+\.\d{2,}','SGRQ.*\d%','St George.*\d+\.\d{2,}']
        

        for sent in short_cleaned_relevant_lst:        
            for param in combinations_list:
                word_length = [len(i.split())-2 for i in re.findall(param,sent)] 
                if word_length:
                    worddistance_list.append(word_length[0])
            if len(worddistance_list)>0:
                min_value = min([i for i in worddistance_list if i > 0])
                if min_value:
                    min_sent_list = []
                    for sent in short_cleaned_relevant_lst:
                        for param in combinations_list:
                            word_length = [len(i.split())-2 for i in re.findall(param,sent)]                                   
                            if word_length and word_length[0] == min_value:
                                oslen.append(sent)
                                os_short = re.findall(param,sent)
                            else:
                                'NA'
                else:
                    return 'NA'
                if oslen:
                    return os_short
                else:
                    'NA'
            else:
                'NA' 
    except:
            return 'NA'
        
short_df['SGRQ_abstract'] = short_df.Abstract_full.apply(SGRQ_sentence)
short_df['SGRQ_fulltext'] = short_df.Full_body_text.apply(SGRQ_sentence)

short_df['SGRQ_short'] = short_df.apply(lambda x: lencheck(x.SGRQ_abstract, x.SGRQ_fulltext), axis=1)

In [139]:
def ACQ_sentence(text): 
    oslen = []
    worddistance_list = []
    text = str(text).strip()
    try:
        short_sentence_list=sent_tokenize(text)
        short_cleaned_relevant_lst = [x.replace('\n','').replace('\\xa','') for x in short_sentence_list]           
        combinations_list = ['ACQ.*\d+\.\d{2,}','ACQ.*\d%', 'ACQ5.*\d+\.\d{2,}','ACQ5.*\d%','ACQ6.*\d+\.\d{2,}','ACQ7.*\d+\.\d{2,}']
        

        for sent in short_cleaned_relevant_lst:        
            for param in combinations_list:
                word_length = [len(i.split())-2 for i in re.findall(param,sent)] 
                if word_length:
                    worddistance_list.append(word_length[0])
            if len(worddistance_list)>0:
                min_value = min([i for i in worddistance_list if i > 0])
                if min_value:
                    min_sent_list = []
                    for sent in short_cleaned_relevant_lst:
                        for param in combinations_list:
                            word_length = [len(i.split())-2 for i in re.findall(param,sent)]                                   
                            if word_length and word_length[0] == min_value:
                                oslen.append(sent)
                                os_short = re.findall(param,sent)
                            else:
                                'NA'
                else:
                    return 'NA'
                if oslen:
                    return os_short
                else:
                    'NA'
            else:
                'NA' 
    except:
            return 'NA'
        
short_df['ACQ_abstract'] = short_df.Abstract_full.apply(ACQ_sentence)
short_df['ACQ_fulltext'] = short_df.Full_body_text.apply(ACQ_sentence)

short_df['ACQ_short'] = short_df.apply(lambda x: lencheck(x.ACQ_abstract, x.ACQ_fulltext), axis=1)

In [140]:
def FEV_sentence(text): 
    oslen = []
    worddistance_list = []
    text = str(text).strip()
    try:
        short_sentence_list=sent_tokenize(text)
        short_cleaned_relevant_lst = [x.replace('\n','').replace('\\xa','') for x in short_sentence_list]           
        combinations_list = ['FEV 1.*\d+\.\d{2,}','FEV1.*\d+\.\d{2,}', 'FEV1.*\d+\.\d{2,}','FEV1.*\d%']
        

        for sent in short_cleaned_relevant_lst:        
            for param in combinations_list:
                word_length = [len(i.split())-2 for i in re.findall(param,sent)] 
                if word_length:
                    worddistance_list.append(word_length[0])
            if len(worddistance_list)>0:
                min_value = min([i for i in worddistance_list if i > 0])
                if min_value:
                    min_sent_list = []
                    for sent in short_cleaned_relevant_lst:
                        for param in combinations_list:
                            word_length = [len(i.split())-2 for i in re.findall(param,sent)]                                   
                            if word_length and word_length[0] == min_value:
                                oslen.append(sent)
                                os_short = re.findall(param,sent)
                            else:
                                'NA'
                else:
                    return 'NA'
                if oslen:
                    return os_short
                else:
                    'NA'
            else:
                'NA' 
    except:
            return 'NA'
        
short_df['FEV_abstract'] = short_df.Abstract_full.apply(FEV_sentence)
short_df['FEV_fulltext'] = short_df.Full_body_text.apply(FEV_sentence)

short_df['FEV_short'] = short_df.apply(lambda x: lencheck(x.FEV_abstract, x.FEV_fulltext), axis=1)

In [141]:
def ACR_sentence(text): 
    oslen = []
    worddistance_list = []
    text = str(text).strip()
    try:
        short_sentence_list=sent_tokenize(text)
        short_cleaned_relevant_lst = [x.replace('\n','').replace('\\xa','') for x in short_sentence_list]           
        combinations_list = ['ACR.*\d+\.\d{2,}','ACR.*\d+\.\d{2,}%']
        

        for sent in short_cleaned_relevant_lst:        
            for param in combinations_list:
                word_length = [len(i.split())-2 for i in re.findall(param,sent)] 
                if word_length:
                    worddistance_list.append(word_length[0])
            if len(worddistance_list)>0:
                min_value = min([i for i in worddistance_list if i > 0])
                if min_value:
                    min_sent_list = []
                    for sent in short_cleaned_relevant_lst:
                        for param in combinations_list:
                            word_length = [len(i.split())-2 for i in re.findall(param,sent)]                                   
                            if word_length and word_length[0] == min_value:
                                oslen.append(sent)
                                os_short = re.findall(param,sent)
                            else:
                                'NA'
                else:
                    return 'NA'
                if oslen:
                    return os_short
                else:
                    'NA'
            else:
                'NA' 
    except:
            return 'NA'
        
short_df['ACR_abstract'] = short_df.Abstract_full.apply(ACR_sentence)
short_df['ACR_fulltext'] = short_df.Full_body_text.apply(ACR_sentence)

short_df['ACR_short'] = short_df.apply(lambda x: lencheck(x.ACR_abstract, x.ACR_fulltext), axis=1)

In [142]:
short_df

XML  \
0     2da20cd7-0125-4728-a219-7cde1329c3e0.xml   
1     b979da00-7c42-457e-b0d4-900c4844459c.xml   
2     ecd677c9-8722-4314-aa15-9c956b38bbd3.xml   
3     53c91dea-5b3c-48fe-ba40-536c77e50012.xml   
4     d1a3a84e-d66f-439c-9874-3c3fdfaf6403.xml   
5     fe4f166b-c4ee-4356-8225-b67f3c0bb8ca.xml   
6     9afd06e7-31b1-4eb9-9e9c-07782b63400d.xml   
7     468cb4e3-6cec-4ff8-85fa-7e1eeb87f0fd.xml   
8     c11b4865-ceec-4566-837f-c62b7f65c993.xml   
9     edf10fd7-46d0-4c41-a0f3-56451b8e5124.xml   
10    cdab46fb-e282-421f-a3eb-cef7d3aab403.xml   
11    b58aa178-fe45-4fb0-8b27-604d69dd7b21.xml   
12    b0d4787a-7f52-4850-a120-d1d3567b3cd9.xml   
13    c90d9fcf-0021-47b7-af91-a91530f107ef.xml   
14    3cba0e7d-164e-47e1-9acc-c2f58906cf37.xml   
15    0749ac49-4bff-4210-b1cf-d6152db6fa23.xml   
16    ed33c249-b022-4474-a7af-8a5072ca7e96.xml   
17    634ba728-aa7a-44f4-be49-be1a18e4a354.xml   
18    530caae7-74a9-4c3b-9977-ed62f918a16c.xml   
19    39a8d70a-492e-4b6a-b2a9-4683fcd41f8f.xml   
20    ee27a2f0-953c-4376-8374-c483aee80664.xml   
21    9cc217c2-56df-4582-950e-e37ebd4b9326.xml   
22    f28c0497-3a2c-4da6-a9a8-f88628529aba.xml   
23    f35ef6ea-4a6e-461e-b6c1-972039914957.xml   
24    64379b97-4601-4821-bdb5-b310c6c6e67e.xml   
25    58d5546d-f559-43ac-bb94-2d4ad2d40d2a.xml   
26    b129cc5e-285f-4928-a560-a03a2d7c7a22.xml   
27    cb74abed-5ed4-4029-9117-078efbce9780.xml   
28    b06c6c50-069f-4e22-8701-3c657477de1d.xml   
29    f5b54f94-638b-48e8-94c2-33288d13e506.xml   
...                                        ...   
8139  fa1ce4bd-0435-4485-8b25-a8a1e20a825f.xml   
8140  fa5a8c19-d879-450b-a4cc-b1713899fe8b.xml   
8141  fa8059de-5ff1-49ac-aff7-9b52de669a9f.xml   
8142  faa15633-9789-4742-9c6e-e5feff7ae6fb.xml   
8143  fabd969e-d1a1-4e67-a009-e5624bb2534c.xml   
8144  fac56fbd-b582-4e88-a18b-3fe935dd52c2.xml   
8145  facde1b2-65bc-4dac-833e-0d7e578b38aa.xml   
8146  fad08045-7720-4297-9388-07a558201c85.xml   
8147  fb0c5b81-591a-4434-a3a3-0bad12be99f3.xml   
8148  fb2177d1-b7d5-4632-9013-5d83162dd111.xml   
8149  fb62cbb2-8867-44c8-8907-bf458f6e5b94.xml   
8150  fb8611c5-21b8-4ba1-9fe9-c60f75a5ceb9.xml   
8151  fb91d08a-22c4-4963-896d-e2fe64cc964a.xml   
8152  fb9b0a8a-0e1d-4ea0-98d1-fbafe67d36cf.xml   
8153  fba562cd-37da-4b74-b501-d6f22d04bddd.xml   
8154  fba72093-bb09-4de1-a691-971c0d263053.xml   
8155  fba74847-62a6-499e-8d50-7e42045a6132.xml   
8156  fbb324d7-1502-4b9e-9e79-d5df613165eb.xml   
8157  fbca531d-af7b-403f-b0d2-80ca65ca5f8a.xml   
8158  fbdc4654-9ed1-41a9-aeb2-4fa7df5eb9a1.xml   
8159  fbe5db4c-bff0-4e31-9a6f-7e62386765f6.xml   
8160  fbf16690-32e7-40ee-9b56-0bb416c0f0a9.xml   
8161  fbfc8576-6ded-4324-9c05-b5d9e63385a3.xml   
8162  fc03532a-7473-4553-94a6-57a996b9003e.xml   
8163  fc09dd4f-1489-4dc7-b9c3-08047aafcb1e.xml   
8164  fc23a41e-50fa-421c-b9f5-ceef8e86743a.xml   
8165  fc49ba3a-1b7d-4484-a45c-cae5cc900101.xml   
8166  fc82b4ea-6d8f-41ec-941d-c77d1db1fc31.xml   
8167  fc97341b-f488-4117-a093-bf3c1b1a70d2.xml   
8168  fc9aa26f-355c-4720-bbfa-effa75b61916.xml   

                                                  Title  \
0     Review article: anti-adhesion therapies for in...   
1     Treatment options for patients with multiple s...   
2     Intravenous immunoglobulin for maintenance tre...   
3     Natalizumab improves ambulation in relapsing&#...   
4     Application of Quantitative Pharmacology Appro...   
5     Current and emerging disease-modulatory therap...   
6     Immunoadsorption and plasma exchange&#8212;Eff...   
7     Modeling the Efficacy of Natalizumab in Multip...   
8     Clinical pharmacology, safety and effects of a...   
9     Paediatric multiple sclerosis: a new era in di...   
10    Effect of natalizumab on brain atrophy and dis...   
11    Therapeutic Application of Monoclonal Antibodi...   
12    A Randomized Trial Evaluating Various Administ...   
13    Established and novel disease-modifying treatm...   
14    Wielding the Double-Edged Sword of Inflammatio...  

In [43]:
short_df[['ACR_short']][short_df['ACR_short']!='NA'].tail(60)

ACR_short
8109      None
8110      None
8111      None
8112      None
8113      None
8114      None
8115      None
8116      None
8117      None
8118      None
8119      None
8120      None
8121      None
8122      None
8123      None
8124      None
8125      None
8126      None
8127      None
8128      None
8129      None
8130      None
8131      None
8132      None
8133      None
8134      None
8135      None
8136      None
8137      None
8138      None
8139      None
8140      None
8141      None
8142      None
8143      None
8144      None
8145      None
8146      None
8147      None
8148      None
8149      None
8150      None
8151      None
8152      None
8153      None
8154      None
8155      None
8156      None
8157      None
8158      None
8159      None
8160      None
8161      None
8162      None
8163      None
8164      None
8165      None
8166      None
8167      None
8168      None

In [143]:
def sPGA_sentence(text): 
    oslen = []
    worddistance_list = []
    text = str(text).strip()
    try:
        short_sentence_list=sent_tokenize(text)
        short_cleaned_relevant_lst = [x.replace('\n','').replace('\\xa','') for x in short_sentence_list]           
        combinations_list = ['sPGA.*\d+\.\d{2,}','sPGA.*\d+\.\d{2,}%']
        

        for sent in short_cleaned_relevant_lst:        
            for param in combinations_list:
                word_length = [len(i.split())-2 for i in re.findall(param,sent)] 
                if word_length:
                    worddistance_list.append(word_length[0])
            if len(worddistance_list)>0:
                min_value = min([i for i in worddistance_list if i > 0])
                if min_value:
                    min_sent_list = []
                    for sent in short_cleaned_relevant_lst:
                        for param in combinations_list:
                            word_length = [len(i.split())-2 for i in re.findall(param,sent)]                                   
                            if word_length and word_length[0] == min_value:
                                oslen.append(sent)
                                os_short = re.findall(param,sent)
                            else:
                                'NA'
                else:
                    return 'NA'
                if oslen:
                    return os_short
                else:
                    'NA'
            else:
                'NA' 
    except:
            return 'NA'
        
short_df['sPGA_abstract'] = short_df.Abstract_full.apply(sPGA_sentence)
short_df['sPGA_fulltext'] = short_df.Full_body_text.apply(sPGA_sentence)

short_df['sPGA_short'] = short_df.apply(lambda x: lencheck(x.sPGA_abstract, x.sPGA_fulltext), axis=1)

In [45]:
short_df[['sPGA_short']][short_df['sPGA_short']!='NA'].tail(60)

sPGA_short
8109       None
8110       None
8111       None
8112       None
8113       None
8114       None
8115       None
8116       None
8117       None
8118       None
8119       None
8120       None
8121       None
8122       None
8123       None
8124       None
8125       None
8126       None
8127       None
8128       None
8129       None
8130       None
8131       None
8132       None
8133       None
8134       None
8135       None
8136       None
8137       None
8138       None
8139       None
8140       None
8141       None
8142       None
8143       None
8144       None
8145       None
8146       None
8147       None
8148       None
8149       None
8150       None
8151       None
8152       None
8153       None
8154       None
8155       None
8156       None
8157       None
8158       None
8159       None
8160       None
8161       None
8162       None
8163       None
8164       None
8165       None
8166       None
8167       None
8168       None

In [144]:
def CRP_sentence(text): 
    oslen = []
    worddistance_list = []
    text = str(text).strip()
    try:
        short_sentence_list=sent_tokenize(text)
        short_cleaned_relevant_lst = [x.replace('\n','').replace('\\xa','') for x in short_sentence_list]           
        combinations_list = ['CRP.*\d+\.\d{2,}','CRP.*\d+\.\d{2,}mg/']
        

        for sent in short_cleaned_relevant_lst:        
            for param in combinations_list:
                word_length = [len(i.split())-2 for i in re.findall(param,sent)] 
                if word_length:
                    worddistance_list.append(word_length[0])
            if len(worddistance_list)>0:
                min_value = min([i for i in worddistance_list if i > 0])
                if min_value:
                    min_sent_list = []
                    for sent in short_cleaned_relevant_lst:
                        for param in combinations_list:
                            word_length = [len(i.split())-2 for i in re.findall(param,sent)]                                   
                            if word_length and word_length[0] == min_value:
                                oslen.append(sent)
                                os_short = re.findall(param,sent)
                            else:
                                'NA'
                else:
                    return 'NA'
                if oslen:
                    return os_short
                else:
                    'NA'
            else:
                'NA' 
    except:
            return 'NA'
        
short_df['CRP_abstract'] = short_df.Abstract_full.apply(CRP_sentence)
short_df['CRP_fulltext'] = short_df.Full_body_text.apply(CRP_sentence)

short_df['CRP_short'] = short_df.apply(lambda x: lencheck(x.CRP_abstract, x.CRP_fulltext), axis=1)



In [47]:
short_df[['CRP_short']][short_df['CRP_short']!='NA'].head(60)

CRP_short
0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
30      None
31      None
32      None
33      None
34      None
35      None
36      None
37      None
38      None
39      None
40      None
41      None
42      None
43      None
44      None
45      None
46      None
47      None
48      None
49      None
50      None
51      None
52      None
53      None
54      None
55      None
56      None
57      None
58      None
59      None

In [145]:
def DAS_sentence(text): 
    oslen = []
    worddistance_list = []
    text = str(text).strip()
    try:
        short_sentence_list=sent_tokenize(text)
        short_cleaned_relevant_lst = [x.replace('\n','').replace('\\xa','') for x in short_sentence_list]           
        combinations_list = ['DAS .*\d+\.\d{2,}','DAS .*\d+\.\d{2,}%']
        

        for sent in short_cleaned_relevant_lst:        
            for param in combinations_list:
                word_length = [len(i.split())-2 for i in re.findall(param,sent)] 
                if word_length:
                    worddistance_list.append(word_length[0])
            if len(worddistance_list)>0:
                min_value = min([i for i in worddistance_list if i > 0])
                if min_value:
                    min_sent_list = []
                    for sent in short_cleaned_relevant_lst:
                        for param in combinations_list:
                            word_length = [len(i.split())-2 for i in re.findall(param,sent)]                                   
                            if word_length and word_length[0] == min_value:
                                oslen.append(sent)
                                os_short = re.findall(param,sent)
                            else:
                                'NA'
                else:
                    return 'NA'
                if oslen:
                    return os_short
                else:
                    'NA'
            else:
                'NA' 
    except:
            return 'NA'
        
short_df['DAS_abstract'] = short_df.Abstract_full.apply(DAS_sentence)
short_df['DAS_fulltext'] = short_df.Full_body_text.apply(DAS_sentence)

short_df['DAS_short'] = short_df.apply(lambda x: lencheck(x.DAS_abstract, x.DAS_fulltext), axis=1)

In [146]:
def ASAS_sentence(text): 
    oslen = []
    worddistance_list = []
    text = str(text).strip()
    try:
        short_sentence_list=sent_tokenize(text)
        short_cleaned_relevant_lst = [x.replace('\n','').replace('\\xa','') for x in short_sentence_list]           
        combinations_list = ['ASAS .*\d+\.\d{2,}','ASAS .*\d+\.\d{2,}%']
        

        for sent in short_cleaned_relevant_lst:        
            for param in combinations_list:
                word_length = [len(i.split())-2 for i in re.findall(param,sent)] 
                if word_length:
                    worddistance_list.append(word_length[0])
            if len(worddistance_list)>0:
                min_value = min([i for i in worddistance_list if i > 0])
                if min_value:
                    min_sent_list = []
                    for sent in short_cleaned_relevant_lst:
                        for param in combinations_list:
                            word_length = [len(i.split())-2 for i in re.findall(param,sent)]                                   
                            if word_length and word_length[0] == min_value:
                                oslen.append(sent)
                                os_short = re.findall(param,sent)
                            else:
                                'NA'
                else:
                    return 'NA'
                if oslen:
                    return os_short
                else:
                    'NA'
            else:
                'NA' 
    except:
            return 'NA'
        
short_df['ASAS_abstract'] = short_df.Abstract_full.apply(ASAS_sentence)
short_df['ASAS_fulltext'] = short_df.Full_body_text.apply(ASAS_sentence)

short_df['ASAS_short'] = short_df.apply(lambda x: lencheck(x.ASAS_abstract, x.ASAS_fulltext), axis=1)

In [50]:
short_df[['ASAS_short']][short_df['ASAS_short']!='NA'].head(60)

ASAS_short
0        None
1        None
2        None
3        None
4        None
5        None
6        None
7        None
8        None
9        None
10       None
11       None
12       None
13       None
14       None
15       None
16       None
17       None
18       None
19       None
20       None
21       None
22       None
23       None
24       None
25       None
26       None
27       None
28       None
29       None
30       None
31       None
32       None
33       None
34       None
35       None
36       None
37       None
38       None
39       None
40       None
41       None
42       None
43       None
44       None
45       None
46       None
47       None
48       None
49       None
50       None
51       None
52       None
53       None
54       None
55       None
56       None
57       None
58       None
59       None

In [147]:
def ES_sentence(text): 
    oslen = []
    worddistance_list = []
    text = str(text).strip()
    try:
        short_sentence_list=sent_tokenize(text)
        short_cleaned_relevant_lst = [x.replace('\n','').replace('\\xa','') for x in short_sentence_list]           
        combinations_list = ['ES .*\d+\.\d{2,}','ES .*\d+\.\d{2,}%','erosion score.*\d+\.\d{2,}','Erosion Score.*\d+\.\d{2,}','Erosion score.*\d+\.\d{2,}']
        

        for sent in short_cleaned_relevant_lst:        
            for param in combinations_list:
                word_length = [len(i.split())-2 for i in re.findall(param,sent)] 
                if word_length:
                    worddistance_list.append(word_length[0])
            if len(worddistance_list)>0:
                min_value = min([i for i in worddistance_list if i > 0])
                if min_value:
                    min_sent_list = []
                    for sent in short_cleaned_relevant_lst:
                        for param in combinations_list:
                            word_length = [len(i.split())-2 for i in re.findall(param,sent)]                                   
                            if word_length and word_length[0] == min_value:
                                oslen.append(sent)
                                os_short = re.findall(param,sent)
                            else:
                                'NA'
                else:
                    return 'NA'
                if oslen:
                    return os_short
                else:
                    'NA'
            else:
                'NA' 
    except:
            return 'NA'
        
short_df['ES_abstract'] = short_df.Abstract_full.apply(ES_sentence)
short_df['ES_fulltext'] = short_df.Full_body_text.apply(ES_sentence)

short_df['ES_short'] = short_df.apply(lambda x: lencheck(x.ES_abstract, x.ES_fulltext), axis=1)

In [52]:
short_df[['ES_short']][short_df['ES_short']!='NA'].tail(60)

ES_short
8109     None
8110     None
8111     None
8112     None
8113     None
8114     None
8115     None
8116     None
8117     None
8118     None
8119     None
8120     None
8121     None
8122     None
8123     None
8124     None
8125     None
8126     None
8127     None
8128     None
8129     None
8130     None
8131     None
8132     None
8133     None
8134     None
8135     None
8136     None
8137     None
8138     None
8139     None
8140     None
8141     None
8142     None
8143     None
8144     None
8145     None
8146     None
8147     None
8148     None
8149     None
8150     None
8151     None
8152     None
8153     None
8154     None
8155     None
8156     None
8157     None
8158     None
8159     None
8160     None
8161     None
8162     None
8163     None
8164     None
8165     None
8166     None
8167     None
8168     None

In [148]:
def TSS_sentence(text): 
    oslen = []
    worddistance_list = []
    text = str(text).strip()
    try:
        short_sentence_list=sent_tokenize(text)
        short_cleaned_relevant_lst = [x.replace('\n','').replace('\\xa','') for x in short_sentence_list]           
        combinations_list = ['TSS .*\d+\.\d{2,}','TSS .*\d+\.\d{2,}%','Total sharp score.*\d+\.\d{2,}']
        

        for sent in short_cleaned_relevant_lst:        
            for param in combinations_list:
                word_length = [len(i.split())-2 for i in re.findall(param,sent)] 
                if word_length:
                    worddistance_list.append(word_length[0])
            if len(worddistance_list)>0:
                min_value = min([i for i in worddistance_list if i > 0])
                if min_value:
                    min_sent_list = []
                    for sent in short_cleaned_relevant_lst:
                        for param in combinations_list:
                            word_length = [len(i.split())-2 for i in re.findall(param,sent)]                                   
                            if word_length and word_length[0] == min_value:
                                oslen.append(sent)
                                os_short = re.findall(param,sent)
                            else:
                                'NA'
                else:
                    return 'NA'
                if oslen:
                    return os_short
                else:
                    'NA'
            else:
                'NA' 
    except:
            return 'NA'
        
short_df['TSS_abstract'] = short_df.Abstract_full.apply(TSS_sentence)
short_df['TSS_fulltext'] = short_df.Full_body_text.apply(TSS_sentence)

short_df['TSS_short'] = short_df.apply(lambda x: lencheck(x.TSS_abstract, x.TSS_fulltext), axis=1)

In [54]:
short_df[['TSS_short']][short_df['TSS_short']!='NA'].head(60)

TSS_short
0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
30      None
31      None
32      None
33      None
34      None
35      None
36      None
37      None
38      None
39      None
40      None
41      None
42      None
43      None
44      None
45      None
46      None
47      None
48      None
49      None
50      None
51      None
52      None
53      None
54      None
55      None
56      None
57      None
58      None
59      None

In [149]:
def FEV_sentence(text): 
    oslen = []
    worddistance_list = []
    text = str(text).strip()
    try:
        short_sentence_list=sent_tokenize(text)
        short_cleaned_relevant_lst = [x.replace('\n','').replace('\\xa','') for x in short_sentence_list]           
        combinations_list = ['FEV 1.*\d+\.\d{2,}','FEV1.*\d+\.\d{2,}', 'FEV1.*\d+\.\d{2,}','FEV1.*\d%']
        

        for sent in short_cleaned_relevant_lst:        
            for param in combinations_list:
                word_length = [len(i.split())-2 for i in re.findall(param,sent)] 
                if word_length:
                    worddistance_list.append(word_length[0])
            if len(worddistance_list)>0:
                min_value = min([i for i in worddistance_list if i > 0])
                if min_value:
                    min_sent_list = []
                    for sent in short_cleaned_relevant_lst:
                        for param in combinations_list:
                            word_length = [len(i.split())-2 for i in re.findall(param,sent)]                                   
                            if word_length and word_length[0] == min_value:
                                oslen.append(sent)
                                os_short = re.findall(param,sent)
                            else:
                                'NA'
                else:
                    return 'NA'
                if oslen:
                    return os_short
                else:
                    'NA'
            else:
                'NA' 
    except:
            return 'NA'
        
short_df['FEV_abstract'] = short_df.Abstract_full.apply(FEV_sentence)
short_df['FEV_fulltext'] = short_df.Full_body_text.apply(FEV_sentence)

short_df['FEV_short'] = short_df.apply(lambda x: lencheck(x.FEV_abstract, x.FEV_fulltext), axis=1)

In [150]:
def MRI_sentence(text): 
    oslen = []
    worddistance_list = []
    text = str(text).strip()
    try:
        short_sentence_list=sent_tokenize(text)
        short_cleaned_relevant_lst = [x.replace('\n','').replace('\\xa','') for x in short_sentence_list]           
        combinations_list = ['MRI.*\d+\.\d{2,}','MRI.*\d%']
        

        for sent in short_cleaned_relevant_lst:        
            for param in combinations_list:
                word_length = [len(i.split())-2 for i in re.findall(param,sent)] 
                if word_length:
                    worddistance_list.append(word_length[0])
            if len(worddistance_list)>0:
                min_value = min([i for i in worddistance_list if i > 0])
                if min_value:
                    min_sent_list = []
                    for sent in short_cleaned_relevant_lst:
                        for param in combinations_list:
                            word_length = [len(i.split())-2 for i in re.findall(param,sent)]                                   
                            if word_length and word_length[0] == min_value:
                                oslen.append(sent)
                                os_short = re.findall(param,sent)
                            else:
                                'NA'
                else:
                    return 'NA'
                if oslen:
                    return os_short
                else:
                    'NA'
            else:
                'NA' 
    except:
            return 'NA'
        
short_df['MRI_abstract'] = short_df.Abstract_full.apply(MRI_sentence)
short_df['MRI_fulltext'] = short_df.Full_body_text.apply(MRI_sentence)

short_df['MRI_short'] = short_df.apply(lambda x: lencheck(x.MRI_abstract, x.MRI_fulltext), axis=1)

In [57]:
short_df[['MRI_short']][short_df['MRI_short']!='NA'].head(60)

MRI_short
0                                                None
1                                                None
2                                                None
3                                                None
4                                                None
5                                                None
6                                                None
7                                                None
8                                                None
9                                                None
10  [MRI) status, the number of natalizumab infusi...
11                                               None
12                                               None
13                                               None
14                                               None
15                                               None
16                                               None
17                                               None
18                                               None
19                                               None
20                                               None
21                                               None
22                                               None
23                                               None
24  [MRI activity alone (80.6% [95% CI 68.6–89.6],...
25                                               None
26                                               None
27                                               None
28  [MRI activity alone (80.6% [95% CI 68.6–89.6],...
29                                               None
30                                               None
31                                               None
32                                               None
33                                               None
34                                               None
35                                               None
36                                               None
37                                               None
38                                               None
39                                               None
40                                               None
41                                               None
42                                               None
43                                               None
44                                               None
45                                               None
46                                               None
47                                               None
48                                               None
49                                               None
50                                               None
51                                               None
52                                               None
53                                               None
54                                               None
55                                               None
56                                               None
57                                               None
58                                               None
59                                               None

In [151]:
def PASI_sentence(text): 
    oslen = []
    worddistance_list = []
    text = str(text).strip()
    try:
        short_sentence_list=sent_tokenize(text)
        short_cleaned_relevant_lst = [x.replace('\n','').replace('\\xa','') for x in short_sentence_list]           
        combinations_list = ['PASI.*\d+\.\d{2,}','PASI.*\d%']
        

        for sent in short_cleaned_relevant_lst:        
            for param in combinations_list:
                word_length = [len(i.split())-2 for i in re.findall(param,sent)] 
                if word_length:
                    worddistance_list.append(word_length[0])
            if len(worddistance_list)>0:
                min_value = min([i for i in worddistance_list if i > 0])
                if min_value:
                    min_sent_list = []
                    for sent in short_cleaned_relevant_lst:
                        for param in combinations_list:
                            word_length = [len(i.split())-2 for i in re.findall(param,sent)]                                   
                            if word_length and word_length[0] == min_value:
                                oslen.append(sent)
                                os_short = re.findall(param,sent)
                            else:
                                'NA'
                else:
                    return 'NA'
                if oslen:
                    return os_short
                else:
                    'NA'
            else:
                'NA' 
    except:
            return 'NA'
        
short_df['PASI_abstract'] = short_df.Abstract_full.apply(PASI_sentence)
short_df['PASI_fulltext'] = short_df.Full_body_text.apply(PASI_sentence)

short_df['PASI_short'] = short_df.apply(lambda x: lencheck(x.PASI_abstract, x.PASI_fulltext), axis=1)

In [152]:
def NEDA_sentence(text): 
    oslen = []
    worddistance_list = []
    text = str(text).strip()
    try:
        short_sentence_list=sent_tokenize(text)
        short_cleaned_relevant_lst = [x.replace('\n','').replace('\\xa','') for x in short_sentence_list]           
        combinations_list = ['NEDA.*\d+\.\d{2,}','NEDA.*\d%','NEDA-3.*\d%','NEDA-3.*\d+\.\d{2,}']
        

        for sent in short_cleaned_relevant_lst:        
            for param in combinations_list:
                word_length = [len(i.split())-2 for i in re.findall(param,sent)] 
                if word_length:
                    worddistance_list.append(word_length[0])
            if len(worddistance_list)>0:
                min_value = min([i for i in worddistance_list if i > 0])
                if min_value:
                    min_sent_list = []
                    for sent in short_cleaned_relevant_lst:
                        for param in combinations_list:
                            word_length = [len(i.split())-2 for i in re.findall(param,sent)]                                   
                            if word_length and word_length[0] == min_value:
                                oslen.append(sent)
                                os_short = re.findall(param,sent)
                            else:
                                'NA'
                else:
                    return 'NA'
                if oslen:
                    return os_short
                else:
                    'NA'
            else:
                'NA' 
    except:
            return 'NA'
        
short_df['NEDA_abstract'] = short_df.Abstract_full.apply(NEDA_sentence)
short_df['NEDA_fulltext'] = short_df.Full_body_text.apply(NEDA_sentence)

short_df['NEDA_short'] = short_df.apply(lambda x: lencheck(x.NEDA_abstract, x.NEDA_fulltext), axis=1)

In [153]:
def SAA_sentence(text): 
    oslen = []
    worddistance_list = []
    text = str(text).strip()
    try:
        short_sentence_list=sent_tokenize(text)
        short_cleaned_relevant_lst = [x.replace('\n','').replace('\\xa','') for x in short_sentence_list]           
        combinations_list = ['SAA.*\d+\.\d{2,}','SAA.*\d+\.\d{2,}mg/']
        

        for sent in short_cleaned_relevant_lst:        
            for param in combinations_list:
                word_length = [len(i.split())-2 for i in re.findall(param,sent)] 
                if word_length:
                    worddistance_list.append(word_length[0])
            if len(worddistance_list)>0:
                min_value = min([i for i in worddistance_list if i > 0])
                if min_value:
                    min_sent_list = []
                    for sent in short_cleaned_relevant_lst:
                        for param in combinations_list:
                            word_length = [len(i.split())-2 for i in re.findall(param,sent)]                                   
                            if word_length and word_length[0] == min_value:
                                oslen.append(sent)
                                os_short = re.findall(param,sent)
                            else:
                                'NA'
                else:
                    return 'NA'
                if oslen:
                    return os_short
                else:
                    'NA'
            else:
                'NA' 
    except:
            return 'NA'
        
short_df['SAA_abstract'] = short_df.Abstract_full.apply(SAA_sentence)
short_df['SAA_fulltext'] = short_df.Full_body_text.apply(SAA_sentence)

short_df['SAA_short'] = short_df.apply(lambda x: lencheck(x.SAA_abstract, x.SAA_fulltext), axis=1)

In [154]:
def NRS_sentence(text): 
    oslen = []
    worddistance_list = []
    text = str(text).strip()
    try:
        short_sentence_list=sent_tokenize(text)
        short_cleaned_relevant_lst = [x.replace('\n','').replace('\\xa','') for x in short_sentence_list]           
        combinations_list = ['NRS.*\d+\.\d{2,}','NRS.*\d%']
        

        for sent in short_cleaned_relevant_lst:        
            for param in combinations_list:
                word_length = [len(i.split())-2 for i in re.findall(param,sent)] 
                if word_length:
                    worddistance_list.append(word_length[0])
            if len(worddistance_list)>0:
                min_value = min([i for i in worddistance_list if i > 0])
                if min_value:
                    min_sent_list = []
                    for sent in short_cleaned_relevant_lst:
                        for param in combinations_list:
                            word_length = [len(i.split())-2 for i in re.findall(param,sent)]                                   
                            if word_length and word_length[0] == min_value:
                                oslen.append(sent)
                                os_short = re.findall(param,sent)
                            else:
                                'NA'
                else:
                    return 'NA'
                if oslen:
                    return os_short
                else:
                    'NA'
            else:
                'NA' 
    except:
            return 'NA'
        
short_df['NRS_abstract'] = short_df.Abstract_full.apply(NRS_sentence)
short_df['NRS_fulltext'] = short_df.Full_body_text.apply(NRS_sentence)

short_df['NRS_short'] = short_df.apply(lambda x: lencheck(x.NRS_abstract, x.NRS_fulltext), axis=1)

In [155]:
def BASDAI_sentence(text): 
    oslen = []
    worddistance_list = []
    text = str(text).strip()
    try:
        short_sentence_list=sent_tokenize(text)
        short_cleaned_relevant_lst = [x.replace('\n','').replace('\\xa','') for x in short_sentence_list]           
        combinations_list = ['BASDAI.*\d+\.\d{2,}','BASDAI.*\d%']
        

        for sent in short_cleaned_relevant_lst:        
            for param in combinations_list:
                word_length = [len(i.split())-2 for i in re.findall(param,sent)] 
                if word_length:
                    worddistance_list.append(word_length[0])
            if len(worddistance_list)>0:
                min_value = min([i for i in worddistance_list if i > 0])
                if min_value:
                    min_sent_list = []
                    for sent in short_cleaned_relevant_lst:
                        for param in combinations_list:
                            word_length = [len(i.split())-2 for i in re.findall(param,sent)]                                   
                            if word_length and word_length[0] == min_value:
                                oslen.append(sent)
                                os_short = re.findall(param,sent)
                            else:
                                'NA'
                else:
                    return 'NA'
                if oslen:
                    return os_short
                else:
                    'NA'
            else:
                'NA' 
    except:
            return 'NA'
        
short_df['BASDAI_abstract'] = short_df.Abstract_full.apply(BASDAI_sentence)
short_df['BASDAI_fulltext'] = short_df.Full_body_text.apply(BASDAI_sentence)

short_df['BASDAI_short'] = short_df.apply(lambda x: lencheck(x.BASDAI_abstract, x.BASDAI_fulltext), axis=1)

In [63]:
short_df[['BASDAI_short']][short_df['BASDAI_short']!='NA'].head(60)

BASDAI_short
0          None
1          None
2          None
3          None
4          None
5          None
6          None
7          None
8          None
9          None
10         None
11         None
12         None
13         None
14         None
15         None
16         None
17         None
18         None
19         None
20         None
21         None
22         None
23         None
24         None
25         None
26         None
27         None
28         None
29         None
30         None
31         None
32         None
33         None
34         None
35         None
36         None
37         None
38         None
39         None
40         None
41         None
42         None
43         None
44         None
45         None
46         None
47         None
48         None
49         None
50         None
51         None
52         None
53         None
54         None
55         None
56         None
57         None
58         None
59         None

In [156]:
def AQLQ_sentence(text): 
    oslen = []
    worddistance_list = []
    text = str(text).strip()
    try:
        short_sentence_list=sent_tokenize(text)
        short_cleaned_relevant_lst = [x.replace('\n','').replace('\\xa','') for x in short_sentence_list]           
        combinations_list = ['AQLQ.*\d+\.\d{2,}','AQLQ.*\d%','AQoL.*\d+\.\d{2,}','AQoL.*\d%','AQOL.*\d+\.\d{2,}','AQOL.*\d%']
        

        for sent in short_cleaned_relevant_lst:        
            for param in combinations_list:
                word_length = [len(i.split())-2 for i in re.findall(param,sent)] 
                if word_length:
                    worddistance_list.append(word_length[0])
            if len(worddistance_list)>0:
                min_value = min([i for i in worddistance_list if i > 0])
                if min_value:
                    min_sent_list = []
                    for sent in short_cleaned_relevant_lst:
                        for param in combinations_list:
                            word_length = [len(i.split())-2 for i in re.findall(param,sent)]                                   
                            if word_length and word_length[0] == min_value:
                                oslen.append(sent)
                                os_short = re.findall(param,sent)
                            else:
                                'NA'
                else:
                    return 'NA'
                if oslen:
                    return os_short
                else:
                    'NA'
            else:
                'NA' 
    except:
            return 'NA'
        
short_df['AQLQ_abstract'] = short_df.Abstract_full.apply(AQLQ_sentence)
short_df['AQLQ_fulltext'] = short_df.Full_body_text.apply(AQLQ_sentence)

short_df['AQLQ_short'] = short_df.apply(lambda x: lencheck(x.AQLQ_abstract, x.AQLQ_fulltext), axis=1)

In [157]:
def mrdsentence(text): 
    oslen = []
    worddistance_list = []
    text = str(text)
    try:
        short_sentence_list=sent_tokenize(text)
        short_cleaned_relevant_lst = [x.replace('\n','') for x in short_sentence_list]           
        combinations_list = ['minimal residual disease.*?%.*?%','residual disease.*?%.*?%','MRD.*?%.*?%',]                     

        for sent in short_cleaned_relevant_lst:        
            for param in combinations_list:
                word_length = [len(i.split())-2 for i in re.findall(param,sent)] 
                if word_length:
                    worddistance_list.append(word_length[0])
            if len(worddistance_list)>0:
                min_value = min([i for i in worddistance_list if i > 0])
                if min_value:
                    min_sent_list = []
                    for sent in short_cleaned_relevant_lst:
                        for param in combinations_list:
                            word_length = [len(i.split())-2 for i in re.findall(param,sent)]                                   
                            if word_length and word_length[0] == min_value:
                                oslen.append(sent)
                                os_short = re.findall(param,sent)
                            else:
                                'NA'
                else:
                    return 'NA'
                if oslen:
                    return os_short
                else:
                    'NA'
            else:
                'NA'
    except:
            return 'NA'
        
def lencheck(abs,fulltxt):
    if abs!='NA' and fulltxt!='NA':
        return fulltxt
    elif abs=='NA' and fulltxt!='NA':
        return fulltxt
    elif abs!='NA' and fulltxt=='NA':
        return abs
    else:
        return "NA"


short_df['MRD_abstract'] = short_df.Abstract_full.apply(mrdsentence)
short_df['MRD_fulltext'] = short_df.Full_body_text.apply(mrdsentence)
short_df['MRD_short'] = short_df.apply(lambda x: lencheck(x.MRD_abstract, x.MRD_fulltext), axis=1)
short_df['MRD_short']

0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
        ... 
8139    None
8140    None
8141    None
8142    None
8143    None
8144    None
8145    None
8146    None
8147    None
8148    None
8149    None
8150    None
8151    None
8152    None
8153    None
8154    None
8155    None
8156    None
8157    None
8158    None
8159    None
8160    None
8161    None
8162    None
8163    None
8164    None
8165    None
8166    None
8167    None
8168    None
Name: MRD_short, Length: 21787, dtype: object

In [158]:
def lesionsentence(text): 
    oslen = []
    worddistance_list = []
    text = str(text)
    try:
        short_sentence_list=sent_tokenize(text)
        short_cleaned_relevant_lst = [x.replace('\n','') for x in short_sentence_list]           
        combinations_list = ['T1 lesion.*?%',
                                         'MRI lesion .*?%','T2 lesion.*?%',
                                         'Gadalonium .*?%','T1 lesion.*CI','lesion.*CI',
                             'T1.*confidence','T2.*lesion.*CI','T2.*confidence','Gadalonium.*?%']
        

        for sent in short_cleaned_relevant_lst:        
            for param in combinations_list:
                word_length = [len(i.split())-2 for i in re.findall(param,sent)] 
                if word_length:
                    worddistance_list.append(word_length[0])
            if len(worddistance_list)>0:
                min_value = min([i for i in worddistance_list if i > 0])
                if min_value:
                    min_sent_list = []
                    for sent in short_cleaned_relevant_lst:
                        for param in combinations_list:
                            word_length = [len(i.split())-2 for i in re.findall(param,sent)]                                   
                            if word_length and word_length[0] == min_value:
                                oslen.append(sent)
                                os_short = re.findall(param,sent)
                            else:
                                'NA'
                else:
                    return 'NA'
                if oslen:
                    return os_short
                else:
                    'NA'
            else:
                'NA'
    except:
            return 'NA'

short_df['lesion_short'] = short_df.Full_body_text.apply(lesionsentence)
short_df['lesion_short']

0                                                    None
1       [T2 lesions at baseline) fingolimod 0.5 mg sig...
2                                                    None
3                                                    None
4                                                    None
5                                                    None
6                                                    None
7       [lesion occurrence with 95% confidence interva...
8                                                    None
9                                                    None
10      [T1‐LV change cm 3 −0.026 −0.096 0.044 0.149 ....
11      [T1 lesions ( P < 0.0001) ‐77%, T1 lesions ( P...
12                                                   None
13                                                   None
14                                                   None
15                                                   None
16                                                   None
17            

In [159]:
def igasentence(text): 
    oslen = []
    worddistance_list = []
    text = str(text)
    try:
        short_sentence_list=sent_tokenize(text)
        short_cleaned_relevant_lst = [x.replace('\n','') for x in short_sentence_list]           
#         combinations_list = ['IGA response.*?%',
#                                          'IGA .*?%','iga response.*?%',
#                                          'PASI .*?%','ACR .*?%','EASI.*?%','NRS.*?%']
        
        combinations_list = ['IGA.*\d+\.\d{2,}','IGA.*\d%','IGA response.*\d%','EASI.*\d+\.\d{2,}','EASI.*\d%','IGA score.*\d+\.\d{2,}','IGA score.*\d%']
        

        for sent in short_cleaned_relevant_lst:        
            for param in combinations_list:
                word_length = [len(i.split())-2 for i in re.findall(param,sent)] 
                if word_length:
                    worddistance_list.append(word_length[0])
            if len(worddistance_list)>0:
                min_value = min([i for i in worddistance_list if i > 0])
                if min_value:
                    min_sent_list = []
                    for sent in short_cleaned_relevant_lst:
                        for param in combinations_list:
                            word_length = [len(i.split())-2 for i in re.findall(param,sent)]                                   
                            if word_length and word_length[0] == min_value:
                                oslen.append(sent)
                                os_short = re.findall(param,sent)
                            else:
                                'NA'
                else:
                    return 'NA'
                if oslen:
                    return os_short
                else:
                    'NA'
            else:
                'NA'
    except:
            return 'NA'

short_df['IGA_abstract'] = short_df.Abstract_full.apply(igasentence)
short_df['IGA_fulltext'] = short_df.Full_body_text.apply(igasentence)
short_df['IGA_short'] = short_df.apply(lambda x: lencheck(x.IGA_abstract, x.IGA_fulltext), axis=1)
short_df['IGA_short']

0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
        ... 
8139    None
8140    None
8141    None
8142    None
8143    None
8144    None
8145    None
8146    None
8147    None
8148    None
8149    None
8150    None
8151    None
8152    None
8153    None
8154    None
8155    None
8156    None
8157    None
8158    None
8159    None
8160    None
8161    None
8162    None
8163    None
8164    None
8165    None
8166    None
8167    None
8168    None
Name: IGA_short, Length: 21787, dtype: object

In [68]:
short_df[['IGA_short']][short_df['IGA_short']!='NA'].tail(60)

IGA_short
8109                                               None
8110                                               None
8111                                               None
8112  [IGA ≤1  DLQI ≤1  PASI 75  PASI 90  PASI 100  ...
8113                                               None
8114                                               None
8115                                               None
8116                                               None
8117                                               None
8118                                               None
8119                                               None
8120                                               None
8121                                               None
8122                                               None
8123                                               None
8124                                               None
8125                                               None
8126                                               None
8127                                               None
8128                                               None
8129                                               None
8130                                               None
8131                                               None
8132                                               None
8133                                               None
8134                                               None
8135                                               None
8136                                               None
8137                                               None
8138                                               None
8139                                               None
8140                                               None
8141                                               None
8142                                               None
8143                                               None
8144                                               None
8145                                               None
8146                                               None
8147                                               None
8148                                               None
8149                                               None
8150                                               None
8151                                               None
8152                                               None
8153                                               None
8154                                               None
8155                                               None
8156                                               None
8157                                               None
8158                                               None
8159                                               None
8160                                               None
8161                                               None
8162                                               None
8163                                               None
8164                                               None
8165                                               None
8166                                               None
8167                                               None
8168                                               None

In [160]:
def bmdsentence(text): 
    oslen = []
    worddistance_list = []
    text = str(text)
    try:
        short_sentence_list=sent_tokenize(text)
        short_cleaned_relevant_lst = [x.replace('\n','') for x in short_sentence_list]           
        combinations_list = ['bone mineral density.*\d', 'number of fractures.*\d','bmd.*\d','BMD.*\d','bone mineral.*\d','fractures.*\d']
        

        for sent in short_cleaned_relevant_lst:        
            for param in combinations_list:
                word_length = [len(i.split())-2 for i in re.findall(param,sent)] 
                if word_length:
                    worddistance_list.append(word_length[0])
            if len(worddistance_list)>0:
                min_value = min([i for i in worddistance_list if i > 0])
                if min_value:
                    min_sent_list = []
                    for sent in short_cleaned_relevant_lst:
                        for param in combinations_list:
                            word_length = [len(i.split())-2 for i in re.findall(param,sent)]                                   
                            if word_length and word_length[0] == min_value:
                                oslen.append(sent)
                                os_short = re.findall(param,sent)
                            else:
                                'NA'
                else:
                    return 'NA'
                if oslen:
                    return os_short
                else:
                    'NA'
            else:
                'NA'
    except:
            return 'NA'

short_df['BMD_short'] = short_df.Full_body_text.apply(bmdsentence)
short_df['BMD_short']

0                                                    None
1                                                    None
2                                                    None
3                                                    None
4                                                    None
5                                                    None
6                                                    None
7                                                    None
8                                                    None
9                                                    None
10                                                   None
11                                                   None
12                                                   None
13                                                   None
14                                                   None
15                                                   None
16                                                   None
17            

In [161]:
def transfusionsentence(text): 
    oslen = []
    worddistance_list = []
    text = str(text)
    try:
        short_sentence_list=sent_tokenize(text)
        short_cleaned_relevant_lst = [x.replace('\n','') for x in short_sentence_list]           
        combinations_list = ['reduction in transfusion burden.*\d', 'transfusion.*\d','RBC transfusion.*\d','RBC.*\d']
        

        for sent in short_cleaned_relevant_lst:        
            for param in combinations_list:
                word_length = [len(i.split())-2 for i in re.findall(param,sent)] 
                if word_length:
                    worddistance_list.append(word_length[0])
            if len(worddistance_list)>0:
                min_value = min([i for i in worddistance_list if i > 0])
                if min_value:
                    min_sent_list = []
                    for sent in short_cleaned_relevant_lst:
                        for param in combinations_list:
                            word_length = [len(i.split())-2 for i in re.findall(param,sent)]                                   
                            if word_length and word_length[0] == min_value:
                                oslen.append(sent)
                                os_short = re.findall(param,sent)
                            else:
                                'NA'
                else:
                    return 'NA'
                if oslen:
                    return os_short
                else:
                    'NA'
            else:
                'NA'
    except:
            return 'NA'

short_df['RBC_transfusion_short'] = short_df.Full_body_text.apply(transfusionsentence)
short_df['RBC_transfusion_short']

0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
        ... 
8139    None
8140    None
8141    None
8142    None
8143    None
8144    None
8145    None
8146    None
8147    None
8148    None
8149    None
8150    None
8151    None
8152    None
8153    None
8154    None
8155    None
8156    None
8157    None
8158    None
8159    None
8160    None
8161    None
8162    None
8163    None
8164    None
8165    None
8166    None
8167    None
8168    None
Name: RBC_transfusion_short, Length: 21787, dtype: object

In [162]:
def PGA_sentence(text): 
    oslen = []
    worddistance_list = []
    text = str(text).strip()
    try:
        short_sentence_list=sent_tokenize(text)
        short_cleaned_relevant_lst = [x.replace('\n','').replace('\\xa','') for x in short_sentence_list]           
        combinations_list = ['PGA.*\d+\.\d{2,}','PGA.*\d+\.\d{2,}%']
        

        for sent in short_cleaned_relevant_lst:        
            for param in combinations_list:
                word_length = [len(i.split())-2 for i in re.findall(param,sent)] 
                if word_length:
                    worddistance_list.append(word_length[0])
            if len(worddistance_list)>0:
                min_value = min([i for i in worddistance_list if i > 0])
                if min_value:
                    min_sent_list = []
                    for sent in short_cleaned_relevant_lst:
                        for param in combinations_list:
                            word_length = [len(i.split())-2 for i in re.findall(param,sent)]                                   
                            if word_length and word_length[0] == min_value:
                                oslen.append(sent)
                                os_short = re.findall(param,sent)
                            else:
                                'NA'
                else:
                    return 'NA'
                if oslen:
                    return os_short
                else:
                    'NA'
            else:
                'NA' 
    except:
            return 'NA'
        
short_df['PGA_abstract'] = short_df.Abstract_full.apply(sPGA_sentence)
short_df['PGA_fulltext'] = short_df.Full_body_text.apply(sPGA_sentence)

short_df['PGA_short'] = short_df.apply(lambda x: lencheck(x.sPGA_abstract, x.sPGA_fulltext), axis=1)

In [72]:
short_df[['PGA_short']][short_df['PGA_short']!='NA'].tail(60)

PGA_short
8109      None
8110      None
8111      None
8112      None
8113      None
8114      None
8115      None
8116      None
8117      None
8118      None
8119      None
8120      None
8121      None
8122      None
8123      None
8124      None
8125      None
8126      None
8127      None
8128      None
8129      None
8130      None
8131      None
8132      None
8133      None
8134      None
8135      None
8136      None
8137      None
8138      None
8139      None
8140      None
8141      None
8142      None
8143      None
8144      None
8145      None
8146      None
8147      None
8148      None
8149      None
8150      None
8151      None
8152      None
8153      None
8154      None
8155      None
8156      None
8157      None
8158      None
8159      None
8160      None
8161      None
8162      None
8163      None
8164      None
8165      None
8166      None
8167      None
8168      None

In [163]:
def orr_biostat(text): 
    text = str(text)             
    try:
        orrstat= re.findall(r'[-+]?\d*\.?\d+%|\d+%', text)
        if orrstat:
            return orrstat[0]
        else:
            return 'NA'
    except:
        return 'NA'


short_df['ORR'] = short_df.ORR_short.apply(orr_biostat)
short_df[['ORR_short','ORR']]

ORR_short    ORR
0                                                    NA     NA
1                                                    NA     NA
2                                                    NA     NA
3                                                    NA     NA
4                                                    NA     NA
5                                                    NA     NA
6                                                    NA     NA
7                                                    NA     NA
8                                                    NA     NA
9                                                    NA     NA
10                                                   NA     NA
11                                                   NA     NA
12                                                   NA     NA
13                                                   NA     NA
14                                                   NA     NA
15                                                   NA     NA
16                                                   NA     NA
17                                                   NA     NA
18                                                   NA     NA
19                                                   NA     NA
20                                                   NA     NA
21                                                   NA     NA
22                                                   NA     NA
23                      [response rate was 54.7% 66.7%]  54.7%
24                                                   NA     NA
25                                                   NA     NA
26                                                   NA     NA
27                                                   NA     NA
28                                                   NA     NA
29                                                   NA     NA
...                                                 ...    ...
8139                                                 NA     NA
8140                                                 NA     NA
8141  [response rate assuming a response rate of 40%...    40%
8142                                                 NA     NA
8143                                                 NA     NA
8144                                                 NA     NA
8145                                                 NA     NA
8146                                                 NA     NA
8147                                                 NA     NA
8148        [response rates at week 12 were 6.8% 15.8%]   6.8%
8149                                                 NA     NA
8150                                                 NA     NA
8151                                                 NA     NA
8152                                                 NA     NA
8153                                                 NA     NA
8154                                                 NA     NA
8155                        [response rates of 56% 44%]    56%
8156                                                 NA     NA
8157                                                 NA     NA
8158                                                 NA     NA
8159                                                 NA     NA
8160                                                 NA     NA
8161                                                 NA     NA
8162                                                 NA     NA
8163                                                 NA     NA
8164                                                 NA     NA
8165                                                 NA     NA
8166                                                 NA     NA
8167                                                 NA     NA
8168                                                 NA     NA

[21787 rows x 2 columns]

In [164]:
def os_biostat_percentage(text): 
    text = str(text)             
    try:
#         osstat= re.findall(r'[-+]?\d*\.?\d+ *month|\d+ *month|[-+]?\d*\.?\d+ week|\d+ *week|[-+]?\d*\.?\d+%|\d+%', text)
        osstat= re.findall(r'[-+]?\d*\.?\d+%|\d+%', text)
        if osstat:
            return osstat[0]
        else:
            return 'NA'
    except:
        return 'NA'


short_df['OS_percentage'] = short_df.OS_short.apply(os_biostat_percentage)
short_df[['OS_short','OS_percentage']]

OS_short OS_percentage
0        None            NA
1        None            NA
2        None            NA
3        None            NA
4        None            NA
5        None            NA
6        None            NA
7        None            NA
8        None            NA
9        None            NA
10       None            NA
11       None            NA
12       None            NA
13       None            NA
14       None            NA
15       None            NA
16       None            NA
17       None            NA
18       None            NA
19       None            NA
20       None            NA
21       None            NA
22       None            NA
23       None            NA
24       None            NA
25       None            NA
26       None            NA
27       None            NA
28       None            NA
29       None            NA
...       ...           ...
8139     None            NA
8140     None            NA
8141     None            NA
8142     None            NA
8143     None            NA
8144     None            NA
8145     None            NA
8146     None            NA
8147     None            NA
8148     None            NA
8149     None            NA
8150     None            NA
8151     None            NA
8152     None            NA
8153     None            NA
8154     None            NA
8155     None            NA
8156     None            NA
8157     None            NA
8158     None            NA
8159     None            NA
8160     None            NA
8161     None            NA
8162     None            NA
8163     None            NA
8164     None            NA
8165     None            NA
8166     None            NA
8167     None            NA
8168     None            NA

[21787 rows x 2 columns]

In [165]:
def os_biostat_month(text): 
    text = str(text)             
    try:
        osstat= re.findall(r'[-+]?\d*\.?\d+ *month|\d+ *month|[-+]?\d*\.?\d+ *mo|[-+]?\d*\.?\d+ week|\d+ *week', text)
#         osstat= re.findall(r'[-+]?\d*\.?\d+%|\d+%', text)
        if osstat:
            return osstat[0]
        else:
            return 'NA'
    except:
        return 'NA'


short_df['OS_month'] = short_df.OS_short.apply(os_biostat_month)
short_df[['OS_short','OS_month']]

OS_short OS_month
0        None       NA
1        None       NA
2        None       NA
3        None       NA
4        None       NA
5        None       NA
6        None       NA
7        None       NA
8        None       NA
9        None       NA
10       None       NA
11       None       NA
12       None       NA
13       None       NA
14       None       NA
15       None       NA
16       None       NA
17       None       NA
18       None       NA
19       None       NA
20       None       NA
21       None       NA
22       None       NA
23       None       NA
24       None       NA
25       None       NA
26       None       NA
27       None       NA
28       None       NA
29       None       NA
...       ...      ...
8139     None       NA
8140     None       NA
8141     None       NA
8142     None       NA
8143     None       NA
8144     None       NA
8145     None       NA
8146     None       NA
8147     None       NA
8148     None       NA
8149     None       NA
8150     None       NA
8151     None       NA
8152     None       NA
8153     None       NA
8154     None       NA
8155     None       NA
8156     None       NA
8157     None       NA
8158     None       NA
8159     None       NA
8160     None       NA
8161     None       NA
8162     None       NA
8163     None       NA
8164     None       NA
8165     None       NA
8166     None       NA
8167     None       NA
8168     None       NA

[21787 rows x 2 columns]

In [166]:
def pfs_biostat_percentage(text): 
    text = str(text)             
    try:
#         osstat= re.findall(r'[-+]?\d*\.?\d+ *month|\d+ *month|[-+]?\d*\.?\d+ week|\d+ *week|[-+]?\d*\.?\d+%|\d+%', text)
        pfstat= re.findall(r'[-+]?\d*\.?\d+%|\d+%', text)
        if pfstat:
            return pfstat[0]
        else:
            return 'NA'
    except:
        return 'NA'


short_df['PFS_percentage'] = short_df.PFS_short.apply(pfs_biostat_percentage)
short_df[['PFS_short','PFS_percentage']]

PFS_short PFS_percentage
0         None             NA
1         None             NA
2         None             NA
3         None             NA
4         None             NA
5         None             NA
6         None             NA
7         None             NA
8         None             NA
9         None             NA
10        None             NA
11        None             NA
12        None             NA
13        None             NA
14        None             NA
15        None             NA
16        None             NA
17        None             NA
18        None             NA
19        None             NA
20        None             NA
21        None             NA
22        None             NA
23        None             NA
24        None             NA
25        None             NA
26        None             NA
27        None             NA
28        None             NA
29        None             NA
...        ...            ...
8139      None             NA
8140      None             NA
8141      None             NA
8142      None             NA
8143      None             NA
8144      None             NA
8145      None             NA
8146      None             NA
8147      None             NA
8148      None             NA
8149      None             NA
8150      None             NA
8151      None             NA
8152      None             NA
8153      None             NA
8154      None             NA
8155      None             NA
8156      None             NA
8157      None             NA
8158      None             NA
8159      None             NA
8160      None             NA
8161      None             NA
8162      None             NA
8163      None             NA
8164      None             NA
8165      None             NA
8166      None             NA
8167      None             NA
8168      None             NA

[21787 rows x 2 columns]

In [167]:
def pfs_biostat_month(text): 
    text = str(text)             
    try:
        pfsstat= re.findall(r'[-+]?\d*\.?\d+ *month|\d+ *month|[-+]?\d*\.?\d+ *mo|[-+]?\d*\.?\d+ week|\d+ *week', text)
#         osstat= re.findall(r'[-+]?\d*\.?\d+%|\d+%', text)
        if pfsstat:
            return pfsstat[0]
        else:
            return 'NA'
    except:
        return 'NA'


short_df['PFS_month'] = short_df.PFS_short.apply(pfs_biostat_month)
short_df[['PFS_short','PFS_month']]

PFS_short PFS_month
0         None        NA
1         None        NA
2         None        NA
3         None        NA
4         None        NA
5         None        NA
6         None        NA
7         None        NA
8         None        NA
9         None        NA
10        None        NA
11        None        NA
12        None        NA
13        None        NA
14        None        NA
15        None        NA
16        None        NA
17        None        NA
18        None        NA
19        None        NA
20        None        NA
21        None        NA
22        None        NA
23        None        NA
24        None        NA
25        None        NA
26        None        NA
27        None        NA
28        None        NA
29        None        NA
...        ...       ...
8139      None        NA
8140      None        NA
8141      None        NA
8142      None        NA
8143      None        NA
8144      None        NA
8145      None        NA
8146      None        NA
8147      None        NA
8148      None        NA
8149      None        NA
8150      None        NA
8151      None        NA
8152      None        NA
8153      None        NA
8154      None        NA
8155      None        NA
8156      None        NA
8157      None        NA
8158      None        NA
8159      None        NA
8160      None        NA
8161      None        NA
8162      None        NA
8163      None        NA
8164      None        NA
8165      None        NA
8166      None        NA
8167      None        NA
8168      None        NA

[21787 rows x 2 columns]

In [168]:
def efs_value(text): 
    text = str(text)             
    try:
        pfsstat= re.findall(r'[-+]?\d*\.?\d+ *month|\d+ *month|[-+]?\d*\.?\d+ *mo|[-+]?\d*\.?\d+ week|\d+ *week', text)
#         osstat= re.findall(r'[-+]?\d*\.?\d+%|\d+%', text)
        if pfsstat:
            return pfsstat[0]
        else:
            return 'NA'
    except:
        return 'NA'


short_df['EFS'] = short_df.EFS_short.apply(efs_value)
short_df[['EFS_short','EFS']]

EFS_short EFS
0         None  NA
1         None  NA
2         None  NA
3         None  NA
4         None  NA
5         None  NA
6         None  NA
7         None  NA
8         None  NA
9         None  NA
10        None  NA
11        None  NA
12        None  NA
13        None  NA
14        None  NA
15        None  NA
16        None  NA
17        None  NA
18        None  NA
19        None  NA
20        None  NA
21        None  NA
22        None  NA
23        None  NA
24        None  NA
25        None  NA
26        None  NA
27        None  NA
28        None  NA
29        None  NA
...        ...  ..
8139      None  NA
8140      None  NA
8141      None  NA
8142      None  NA
8143      None  NA
8144      None  NA
8145      None  NA
8146      None  NA
8147      None  NA
8148      None  NA
8149      None  NA
8150      None  NA
8151      None  NA
8152      None  NA
8153      None  NA
8154      None  NA
8155      None  NA
8156      None  NA
8157      None  NA
8158      None  NA
8159      None  NA
8160      None  NA
8161      None  NA
8162      None  NA
8163      None  NA
8164      None  NA
8165      None  NA
8166      None  NA
8167      None  NA
8168      None  NA

[21787 rows x 2 columns]

In [78]:
short_df[['EFS_short','EFS']]

EFS_short EFS
0         None  NA
1         None  NA
2         None  NA
3         None  NA
4         None  NA
5         None  NA
6         None  NA
7         None  NA
8         None  NA
9         None  NA
10        None  NA
11        None  NA
12        None  NA
13        None  NA
14        None  NA
15        None  NA
16        None  NA
17        None  NA
18        None  NA
19        None  NA
20        None  NA
21        None  NA
22        None  NA
23        None  NA
24        None  NA
25        None  NA
26        None  NA
27        None  NA
28        None  NA
29        None  NA
...        ...  ..
8139      None  NA
8140      None  NA
8141      None  NA
8142      None  NA
8143      None  NA
8144      None  NA
8145      None  NA
8146      None  NA
8147      None  NA
8148      None  NA
8149      None  NA
8150      None  NA
8151      None  NA
8152      None  NA
8153      None  NA
8154      None  NA
8155      None  NA
8156      None  NA
8157      None  NA
8158      None  NA
8159      None  NA
8160      None  NA
8161      None  NA
8162      None  NA
8163      None  NA
8164      None  NA
8165      None  NA
8166      None  NA
8167      None  NA
8168      None  NA

[21787 rows x 2 columns]

In [74]:
short_df.columns

Index(['XML', 'Title', 'Full_body_text', 'Abstract_full', 'DOI', 'URL',
       'Molecule', 'Publisher_type', 'Molecule_text', 'Intervention',
       ...
       'lesion_short', 'IGA_abstract', 'IGA_fulltext', 'IGA_short',
       'BMD_short', 'RBC_transfusion_short', 'PGA_abstract', 'PGA_fulltext',
       'PGA_short', 'EFS'],
      dtype='object', length=148)

In [169]:
def ARR_value(text): 
    text = str(text)             
#     try:
    orrstat= re.findall(r'\b\d{1,2}(?:\.\d{1,2})', text)       
    if orrstat:
        return orrstat[0]
#             if len(orrstat)>1:
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#             elif len(orrstat)==1:
# #                 print(orrstat)
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#                     else:
#                         return val

#             else:
#                 return 'NA'
    else:
        return 'NA'
#     except:
#         return 'NA'


short_df['ARR'] = short_df.ARR_short.apply(ARR_value)
short_df[['ARR_short','ARR']]

ARR_short ARR
0         None  NA
1         None  NA
2         None  NA
3         None  NA
4         None  NA
5         None  NA
6         None  NA
7         None  NA
8         None  NA
9         None  NA
10        None  NA
11        None  NA
12        None  NA
13        None  NA
14        None  NA
15        None  NA
16        None  NA
17        None  NA
18        None  NA
19        None  NA
20        None  NA
21        None  NA
22        None  NA
23        None  NA
24        None  NA
25        None  NA
26        None  NA
27        None  NA
28        None  NA
29        None  NA
...        ...  ..
8139      None  NA
8140      None  NA
8141      None  NA
8142      None  NA
8143      None  NA
8144      None  NA
8145      None  NA
8146      None  NA
8147      None  NA
8148      None  NA
8149      None  NA
8150      None  NA
8151      None  NA
8152      None  NA
8153      None  NA
8154      None  NA
8155      None  NA
8156      None  NA
8157      None  NA
8158      None  NA
8159      None  NA
8160      None  NA
8161      None  NA
8162      None  NA
8163      None  NA
8164      None  NA
8165      None  NA
8166      None  NA
8167      None  NA
8168      None  NA

[21787 rows x 2 columns]

In [80]:
Daclizumab_df = short_df[short_df.Molecule =='Natalizumab']
Daclizumab_df.shape

(150, 149)

In [81]:
Daclizumab_df[['ARR_short','ARR']][Daclizumab_df['ARR_short']!='NA']

ARR_short ARR
0          None  NA
1          None  NA
2          None  NA
3          None  NA
4          None  NA
...         ...  ..
12257      None  NA
12258      None  NA
12259      None  NA
12260      None  NA
12261      None  NA

[150 rows x 2 columns]

In [82]:
# Daclizumab_df.to_excel('/Users/savioel2/Desktop/HawkAISandoz/2022_release/Insights Module/interim_qlik/metrics/Natalizumab_arr.xlsx')


In [170]:
def exacerbation_value(text): 
    text = str(text)             
#     try:
    orrstat= re.findall(r'\b\d{1,2}(?:\.\d{1,2})', text)       
    if orrstat:
        return orrstat[0]
#             if len(orrstat)>1:
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#             elif len(orrstat)==1:
# #                 print(orrstat)
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#                     else:
#                         return val

#             else:
#                 return 'NA'
    else:
        return 'NA'
    #     except:
    #         return 'NA'


short_df['exacerbation_rate'] = short_df.exacerbation_short.apply(exacerbation_value)
short_df[['exacerbation_short','exacerbation_rate']]

exacerbation_short exacerbation_rate
0                  None                NA
1                  None                NA
2                  None                NA
3                  None                NA
4                  None                NA
5                  None                NA
6                  None                NA
7                  None                NA
8                  None                NA
9                  None                NA
10                 None                NA
11                 None                NA
12                 None                NA
13                 None                NA
14                 None                NA
15                 None                NA
16                 None                NA
17                 None                NA
18                 None                NA
19                 None                NA
20                 None                NA
21                 None                NA
22                 None                NA
23                 None                NA
24                 None                NA
25                 None                NA
26                 None                NA
27                 None                NA
28                 None                NA
29                 None                NA
...                 ...               ...
8139               None                NA
8140               None                NA
8141               None                NA
8142               None                NA
8143               None                NA
8144               None                NA
8145               None                NA
8146               None                NA
8147               None                NA
8148               None                NA
8149               None                NA
8150               None                NA
8151               None                NA
8152               None                NA
8153               None                NA
8154               None                NA
8155               None                NA
8156               None                NA
8157               None                NA
8158               None                NA
8159               None                NA
8160               None                NA
8161               None                NA
8162               None                NA
8163               None                NA
8164               None                NA
8165               None                NA
8166               None                NA
8167               None                NA
8168               None                NA

[21787 rows x 2 columns]

In [75]:
Emicizumab_df = short_df[short_df.Molecule =='Emicizumab']
Emicizumab_df.shape

(76, 148)

In [76]:
Emicizumab_df.columns.to_list()

['XML',
 'Title',
 'Full_body_text',
 'Abstract_full',
 'DOI',
 'URL',
 'Molecule',
 'Publisher_type',
 'Molecule_text',
 'Intervention',
 'STRENGTH',
 'DRUG',
 'Molecule_in_Title',
 'DISEASE_abstract',
 'DISEASE_title',
 'DISEASE',
 'Title_short',
 'combined_text',
 '1st_line/2nd_line',
 'Patients',
 'ORR_abstract',
 'ORR_fulltext',
 'ORR_short',
 'OS_short',
 'PFS_short',
 'Comparator',
 'Arms',
 'Mono/Combi',
 'phase',
 'PHASES',
 'phase2',
 'PHASES2',
 'Age',
 'Age_Number',
 'age_groups',
 'Age2',
 'Age2_value',
 'Age2_number',
 'Age2_groups',
 'Unnamed: 0.3',
 'Unnamed: 0.2',
 'Unnamed: 0.1',
 'Unnamed: 0',
 'phase21',
 'Intervention2',
 'DOSAGE',
 'Number of evaluable subjects',
 'ORR',
 'OS_percentage',
 'OS_month',
 'PFS_percentage',
 'PFS_month',
 'DOSAGE_VAL',
 'exacerbation_abstract',
 'exacerbation_fulltext',
 'exacerbation_short',
 'NPS_abstract',
 'NPS_fulltext',
 'NPS_short',
 'ARR_abstract',
 'ARR_fulltext',
 'ARR_short',
 'EDSS_abstract',
 'EDSS_fulltext',
 'EDSS_short

In [77]:
Emicizumab_df[['ORR',
 'sPGA',
 'TSS',
 'MRI',
 'PASI',
 'DAS',
 'ES',
 'IGA',
 'CRP',
 'NRS',
 'ASAS',
 'BASDAI']].tail(60)

KeyError: "['CRP', 'MRI', 'sPGA', 'PASI', 'NRS', 'DAS', 'ES', 'IGA', 'BASDAI', 'TSS', 'ASAS'] not in index"

In [141]:
Emicizumab_df[['ABR_short','ABR']][Emicizumab_df['ABR']!='NA'].head(60)

Empty DataFrame
Columns: [ABR_short, ABR]
Index: []

In [86]:
# Emicizumab_df.to_excel('/Users/savioel2/Desktop/HawkAISandoz/2022_release/Insights Module/interim_qlik/metrics/Reslizumab_exa_rate.xlsx')



In [171]:
def EDSS_value(text): 
    text = str(text)             
    try:
        orrstat= re.findall(r'[-+]?\d*\.?\d+\.\d{2,}|\b\d{1,2}(?:\.\d{1,2})*%?', text)
        if orrstat:
            if len(orrstat)>1:
                for val in orrstat:
                    if '%' in val:
                        return val
            elif len(orrstat)==1:
#                 print(orrstat)
                for val in orrstat:
                    if '%' in val:
                        return val
                    else:
                        return val
                        
            else:
                return 'NA'
        else:
            return 'NA'
    except:
        return 'NA'


short_df['EDSS'] = short_df.EDSS_short.apply(EDSS_value)
short_df[['EDSS_short','EDSS']]

EDSS_short EDSS
0                                                  None   NA
1                                                  None   NA
2                                                  None   NA
3                                                  None   NA
4                                                  None   NA
5                                                  None   NA
6                                                  None   NA
7                                                  None   NA
8                                                  None   NA
9                                                  None   NA
10                                                 None   NA
11                                                 None   NA
12                                                 None   NA
13                                                 None   NA
14                                                 None   NA
15                                                 None   NA
16                                                 None   NA
17                                                 None   NA
18                                                 None   NA
19                                                 None   NA
20                                                 None   NA
21                                                 None   NA
22                                                 None   NA
23                                                 None   NA
24                                                 None   NA
25                                                 None   NA
26    [EDSS ≥7.0 (hazard ratio=0.54 95% confidence i...  95%
27                                                 None   NA
28                                                 None   NA
29                                                 None   NA
...                                                 ...  ...
8139                                               None   NA
8140                                               None   NA
8141                                               None   NA
8142                                               None   NA
8143                                               None   NA
8144                                               None   NA
8145                                               None   NA
8146                                               None   NA
8147                                               None   NA
8148                                               None   NA
8149                                               None   NA
8150                                               None   NA
8151                                               None   NA
8152                                               None   NA
8153                                               None   NA
8154                                               None   NA
8155                                               None   NA
8156                                               None   NA
8157                                               None   NA
8158                                               None   NA
8159                                               None   NA
8160                                               None   NA
8161                                               None   NA
8162                                               None   NA
8163                                               None   NA
8164                                               None   NA
8165                                               None   NA
8166                                               None   NA
8167                                               None   NA
8168                                               None   NA

[21787 rows x 2 columns]

In [82]:
short_df

XML  \
0     2da20cd7-0125-4728-a219-7cde1329c3e0.xml   
1     b979da00-7c42-457e-b0d4-900c4844459c.xml   
2     ecd677c9-8722-4314-aa15-9c956b38bbd3.xml   
3     53c91dea-5b3c-48fe-ba40-536c77e50012.xml   
4     d1a3a84e-d66f-439c-9874-3c3fdfaf6403.xml   
5     fe4f166b-c4ee-4356-8225-b67f3c0bb8ca.xml   
6     9afd06e7-31b1-4eb9-9e9c-07782b63400d.xml   
7     468cb4e3-6cec-4ff8-85fa-7e1eeb87f0fd.xml   
8     c11b4865-ceec-4566-837f-c62b7f65c993.xml   
9     edf10fd7-46d0-4c41-a0f3-56451b8e5124.xml   
10    cdab46fb-e282-421f-a3eb-cef7d3aab403.xml   
11    b58aa178-fe45-4fb0-8b27-604d69dd7b21.xml   
12    b0d4787a-7f52-4850-a120-d1d3567b3cd9.xml   
13    c90d9fcf-0021-47b7-af91-a91530f107ef.xml   
14    3cba0e7d-164e-47e1-9acc-c2f58906cf37.xml   
15    0749ac49-4bff-4210-b1cf-d6152db6fa23.xml   
16    ed33c249-b022-4474-a7af-8a5072ca7e96.xml   
17    634ba728-aa7a-44f4-be49-be1a18e4a354.xml   
18    530caae7-74a9-4c3b-9977-ed62f918a16c.xml   
19    39a8d70a-492e-4b6a-b2a9-4683fcd41f8f.xml   
20    ee27a2f0-953c-4376-8374-c483aee80664.xml   
21    9cc217c2-56df-4582-950e-e37ebd4b9326.xml   
22    f28c0497-3a2c-4da6-a9a8-f88628529aba.xml   
23    f35ef6ea-4a6e-461e-b6c1-972039914957.xml   
24    64379b97-4601-4821-bdb5-b310c6c6e67e.xml   
25    58d5546d-f559-43ac-bb94-2d4ad2d40d2a.xml   
26    b129cc5e-285f-4928-a560-a03a2d7c7a22.xml   
27    cb74abed-5ed4-4029-9117-078efbce9780.xml   
28    b06c6c50-069f-4e22-8701-3c657477de1d.xml   
29    f5b54f94-638b-48e8-94c2-33288d13e506.xml   
...                                        ...   
8139  fa1ce4bd-0435-4485-8b25-a8a1e20a825f.xml   
8140  fa5a8c19-d879-450b-a4cc-b1713899fe8b.xml   
8141  fa8059de-5ff1-49ac-aff7-9b52de669a9f.xml   
8142  faa15633-9789-4742-9c6e-e5feff7ae6fb.xml   
8143  fabd969e-d1a1-4e67-a009-e5624bb2534c.xml   
8144  fac56fbd-b582-4e88-a18b-3fe935dd52c2.xml   
8145  facde1b2-65bc-4dac-833e-0d7e578b38aa.xml   
8146  fad08045-7720-4297-9388-07a558201c85.xml   
8147  fb0c5b81-591a-4434-a3a3-0bad12be99f3.xml   
8148  fb2177d1-b7d5-4632-9013-5d83162dd111.xml   
8149  fb62cbb2-8867-44c8-8907-bf458f6e5b94.xml   
8150  fb8611c5-21b8-4ba1-9fe9-c60f75a5ceb9.xml   
8151  fb91d08a-22c4-4963-896d-e2fe64cc964a.xml   
8152  fb9b0a8a-0e1d-4ea0-98d1-fbafe67d36cf.xml   
8153  fba562cd-37da-4b74-b501-d6f22d04bddd.xml   
8154  fba72093-bb09-4de1-a691-971c0d263053.xml   
8155  fba74847-62a6-499e-8d50-7e42045a6132.xml   
8156  fbb324d7-1502-4b9e-9e79-d5df613165eb.xml   
8157  fbca531d-af7b-403f-b0d2-80ca65ca5f8a.xml   
8158  fbdc4654-9ed1-41a9-aeb2-4fa7df5eb9a1.xml   
8159  fbe5db4c-bff0-4e31-9a6f-7e62386765f6.xml   
8160  fbf16690-32e7-40ee-9b56-0bb416c0f0a9.xml   
8161  fbfc8576-6ded-4324-9c05-b5d9e63385a3.xml   
8162  fc03532a-7473-4553-94a6-57a996b9003e.xml   
8163  fc09dd4f-1489-4dc7-b9c3-08047aafcb1e.xml   
8164  fc23a41e-50fa-421c-b9f5-ceef8e86743a.xml   
8165  fc49ba3a-1b7d-4484-a45c-cae5cc900101.xml   
8166  fc82b4ea-6d8f-41ec-941d-c77d1db1fc31.xml   
8167  fc97341b-f488-4117-a093-bf3c1b1a70d2.xml   
8168  fc9aa26f-355c-4720-bbfa-effa75b61916.xml   

                                                  Title  \
0     Review article: anti-adhesion therapies for in...   
1     Treatment options for patients with multiple s...   
2     Intravenous immunoglobulin for maintenance tre...   
3     Natalizumab improves ambulation in relapsing&#...   
4     Application of Quantitative Pharmacology Appro...   
5     Current and emerging disease-modulatory therap...   
6     Immunoadsorption and plasma exchange&#8212;Eff...   
7     Modeling the Efficacy of Natalizumab in Multip...   
8     Clinical pharmacology, safety and effects of a...   
9     Paediatric multiple sclerosis: a new era in di...   
10    Effect of natalizumab on brain atrophy and dis...   
11    Therapeutic Application of Monoclonal Antibodi...   
12    A Randomized Trial Evaluating Various Administ...   
13    Established and novel disease-modifying treatm...   
14    Wielding the Double-Edged Sword of Inflammatio...  

In [172]:
def ABR_value(text): 
    text = str(text)             
#     try:
    orrstat= re.findall(r'\b\d{1,2}(?:\.\d{1,2})', text)       
    if orrstat:
        return orrstat[0]
#             if len(orrstat)>1:
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#             elif len(orrstat)==1:
# #                 print(orrstat)
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#                     else:
#                         return val

#             else:
#                 return 'NA'
    else:
        return 'NA'
#     except:
#         return 'NA'


short_df['ABR'] = short_df.ABR_short.apply(ABR_value)
short_df[['ABR_short','ABR']]

ABR_short ABR
0         None  NA
1         None  NA
2         None  NA
3         None  NA
4         None  NA
5         None  NA
6         None  NA
7         None  NA
8         None  NA
9         None  NA
10        None  NA
11        None  NA
12        None  NA
13        None  NA
14        None  NA
15        None  NA
16        None  NA
17        None  NA
18        None  NA
19        None  NA
20        None  NA
21        None  NA
22        None  NA
23        None  NA
24        None  NA
25        None  NA
26        None  NA
27        None  NA
28        None  NA
29        None  NA
...        ...  ..
8139      None  NA
8140      None  NA
8141      None  NA
8142      None  NA
8143      None  NA
8144      None  NA
8145      None  NA
8146      None  NA
8147      None  NA
8148      None  NA
8149      None  NA
8150      None  NA
8151      None  NA
8152      None  NA
8153      None  NA
8154      None  NA
8155      None  NA
8156      None  NA
8157      None  NA
8158      None  NA
8159      None  NA
8160      None  NA
8161      None  NA
8162      None  NA
8163      None  NA
8164      None  NA
8165      None  NA
8166      None  NA
8167      None  NA
8168      None  NA

[21787 rows x 2 columns]

In [173]:
def PGA_value(text): 
    text = str(text)             
#     try:
    orrstat= re.findall(r'\b\d{1,2}(?:\.\d{1,2})', text)       
    if orrstat:
        return orrstat[0]
#             if len(orrstat)>1:
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#             elif len(orrstat)==1:
# #                 print(orrstat)
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#                     else:
#                         return val

#             else:
#                 return 'NA'
    else:
        return 'NA'
#     except:
#         return 'NA'


short_df['PGA'] = short_df.PGA_short.apply(PGA_value)
short_df[['PGA_short','PGA']]

PGA_short PGA
0         None  NA
1         None  NA
2         None  NA
3         None  NA
4         None  NA
5         None  NA
6         None  NA
7         None  NA
8         None  NA
9         None  NA
10        None  NA
11        None  NA
12        None  NA
13        None  NA
14        None  NA
15        None  NA
16        None  NA
17        None  NA
18        None  NA
19        None  NA
20        None  NA
21        None  NA
22        None  NA
23        None  NA
24        None  NA
25        None  NA
26        None  NA
27        None  NA
28        None  NA
29        None  NA
...        ...  ..
8139      None  NA
8140      None  NA
8141      None  NA
8142      None  NA
8143      None  NA
8144      None  NA
8145      None  NA
8146      None  NA
8147      None  NA
8148      None  NA
8149      None  NA
8150      None  NA
8151      None  NA
8152      None  NA
8153      None  NA
8154      None  NA
8155      None  NA
8156      None  NA
8157      None  NA
8158      None  NA
8159      None  NA
8160      None  NA
8161      None  NA
8162      None  NA
8163      None  NA
8164      None  NA
8165      None  NA
8166      None  NA
8167      None  NA
8168      None  NA

[21787 rows x 2 columns]

In [85]:
Emicizumab_df = short_df[short_df.Molecule =='Benralizumab']
Emicizumab_df.shape

(147, 149)

In [86]:
Emicizumab_df.head()

XML  \
775  e4a50995-91b3-432b-b93f-70923c53c9b0.xml   
776  007e2a29-b707-43cb-be9d-cb747873c09c.xml   
777  5bfc79c6-54df-4d89-90ba-79b1c8cbcf86.xml   
778  5e19be44-1778-4274-beaa-f35062602108.xml   
779  e1bbf861-a11b-4c54-8246-541b2b2297b3.xml   

                                                 Title  \
775  Benralizumab effect on severe chronic rhinosin...   
776  Treatment of lymphocyte-variant hypereosinophi...   
777  Treating severe asthma: Targeting the IL-5 pat...   
778  Adherence to inhaled corticosteroids and clini...   
779  New and emerging drug treatments for severe as...   

                                        Full_body_text  \
775  Abbreviations   ADCC  antibody‐dependent cell‐...   
776  Hypereosinophilic syndromes (HES) are a group ...   
777  INTRODUCTION  Asthma is a common chronic disea...   
778  To the Editor,   Asthma is a common inflammato...   
779  INTRODUCTION  Asthma is a common airways disea...   

                                         Abstract_full                DOI  \
775  {'Abstract': 'Chronic rhinosinusitis with nasa...  10.1111/cea.13852   
776  {'Abstract': 'Lymphocyte‐variant hypereosinoph...  10.1111/bjh.17615   
777  {'Abstract': 'Severe asthma is a heterogeneous...  10.1111/cea.13885   
778                                                 {}  10.1111/all.14737   
779  {'Abstract': 'Asthma is a common chronic infla...  10.1111/cea.13086   

                                   URL      Molecule Publisher_type  \
775  https://doi.org/10.1111/cea.13852  Benralizumab          Wiley   
776  https://doi.org/10.1111/bjh.17615  Benralizumab          Wiley   
777  https://doi.org/10.1111/cea.13885  Benralizumab          Wiley   
778  https://doi.org/10.1111/all.14737  Benralizumab          Wiley   
779  https://doi.org/10.1111/cea.13086  Benralizumab          Wiley   

                                         Molecule_text  \
775  ['benralizumab, an afucosylated monoclonal ant...   
776  ['the anti‐il‐5 monoclonal antibodies mepolizu...   
777  ['monoclonal antibodies that target the eosino...   
778  ['amongst these is benralizumab, a humanized m...   
779  ['biological agents for moderate‐to‐severe ast...   

                                          Intervention  ... IGA_abstract  \
775  patients with severe np (defined by endoscopic...  ...         None   
776  at a dose of 30 mg subcutaneously every 4 week...  ...         None   
777  35  benralizumab is administered subcutaneousl...  ...         None   
778  we retrospectively assessed patients who comme...  ...         None   
779  biological agents for moderate‐to‐severe asthm...  ...         None   

    IGA_fulltext IGA_short BMD_short RBC_transfusion_short PGA_abstract  \
775         None      None      None                  None         None   
776         None      None      None                  None         None   
777         None      None      None                  None         None   
778         None      None      None                  None         None   
779         None      None      None                  None         None   

    PGA_fulltext PGA_short EFS PGA  
775         None      None  NA  NA  
776         None      None  NA  NA  
777         None      None  NA  NA  
778         None      None  NA  NA  
779         None      None  NA  NA  

[5 rows x 149 columns]

In [92]:
pd.set_option('display.max_colwidth', None)

In [93]:
Emicizumab_df = Emicizumab_df[['ABR_short','ABR']][Emicizumab_df['ABR_short']!='NA']

In [94]:
# Emicizumab_df.to_excel('/Users/savioel2/Desktop/HawkAISandoz/2022_release/Insights Module/interim_qlik/metrics/Emicizumab_abr.xlsx')



In [174]:
def FEV1_value(text): 
    text = str(text)             
#     try:
    orrstat= re.findall(r'[-+]?\d*\.?\d+\.\d{2,}|\b\d{1,2}(?:\.\d{1,2})*%?', text)       
    if orrstat:
#         return orrstat[0]
        if len(orrstat)>1:
            for val in orrstat:
                if '%' in val and val!=1:
                    return val
        elif len(orrstat)==1:
#                 print(orrstat)
            for val in orrstat:
                if '%' in val and val!=1:
                    return val
#                 elif :
#                     return val

        else:
            return 'NA'
    else:
        return 'NA'
#     except:
#         return 'NA'


short_df['FEV1'] = short_df.FEV_short.apply(FEV1_value)
short_df[['FEV_short','FEV1']]

FEV_short FEV1
0         None   NA
1         None   NA
2         None   NA
3         None   NA
4         None   NA
5         None   NA
6         None   NA
7         None   NA
8         None   NA
9         None   NA
10        None   NA
11        None   NA
12        None   NA
13        None   NA
14        None   NA
15        None   NA
16        None   NA
17        None   NA
18        None   NA
19        None   NA
20        None   NA
21        None   NA
22        None   NA
23        None   NA
24        None   NA
25        None   NA
26        None   NA
27        None   NA
28        None   NA
29        None   NA
...        ...  ...
8139      None   NA
8140      None   NA
8141      None   NA
8142      None   NA
8143      None   NA
8144      None   NA
8145      None   NA
8146      None   NA
8147      None   NA
8148      None   NA
8149      None   NA
8150      None   NA
8151      None   NA
8152      None   NA
8153      None   NA
8154      None   NA
8155      None   NA
8156      None   NA
8157      None   NA
8158      None   NA
8159      None   NA
8160      None   NA
8161      None   NA
8162      None   NA
8163      None   NA
8164      None   NA
8165      None   NA
8166      None   NA
8167      None   NA
8168      None   NA

[21787 rows x 2 columns]

In [96]:
# Emicizumab_df[['FEV_short','FEV1']][Emicizumab_df['FEV_short']!='NA'].tail(60)

In [97]:
# Emicizumab_df.to_excel('/Users/savioel2/Desktop/HawkAISandoz/2022_release/Insights Module/interim_qlik/metrics/Benralizumab_fev1.xlsx')



In [175]:
def SGRQ_value(text): 
    text = str(text)             
    try:
        orrstat= re.findall(r'[-+]?\d*\.?\d+\.\d{2,}|\b\d{1,2}(?:\.\d{1,2})*%?', text)
        if orrstat:
            for val in orrstat:
                if len(orrstat)>1:
                    return orrstat
#             if len(orrstat)>1:
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#             elif len(orrstat)==1:
# #                 print(orrstat)
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#                     else:
#                         return val

#             else:
#                 return 'NA'
        else:
            return 'NA'
#     except:
#         return 'NA
    except:
        return 'NA'


short_df['SGRQ'] = short_df.SGRQ_short.apply(SGRQ_value)
short_df[['SGRQ_short','SGRQ']]

SGRQ_short SGRQ
0          None   NA
1          None   NA
2          None   NA
3          None   NA
4          None   NA
5          None   NA
6          None   NA
7          None   NA
8          None   NA
9          None   NA
10         None   NA
11         None   NA
12         None   NA
13         None   NA
14         None   NA
15         None   NA
16         None   NA
17         None   NA
18         None   NA
19         None   NA
20         None   NA
21         None   NA
22         None   NA
23         None   NA
24         None   NA
25         None   NA
26         None   NA
27         None   NA
28         None   NA
29         None   NA
...         ...  ...
8139       None   NA
8140       None   NA
8141       None   NA
8142       None   NA
8143       None   NA
8144       None   NA
8145       None   NA
8146       None   NA
8147       None   NA
8148       None   NA
8149       None   NA
8150       None   NA
8151       None   NA
8152       None   NA
8153       None   NA
8154       None   NA
8155       None   NA
8156       None   NA
8157       None   NA
8158       None   NA
8159       None   NA
8160       None   NA
8161       None   NA
8162       None   NA
8163       None   NA
8164       None   NA
8165       None   NA
8166       None   NA
8167       None   NA
8168       None   NA

[21787 rows x 2 columns]

In [176]:
def ACQ_value(text): 
    text = str(text)             
#     try:
    orrstat= re.findall(r'\b\d{1,2}(?:\.\d{1,2})', text)      
    if orrstat:
        return orrstat[0]
#             if len(orrstat)>1:
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#             elif len(orrstat)==1:
# #                 print(orrstat)
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#                     else:
#                         return val

#             else:
#                 return 'NA'
    else:
        return 'NA'
#     except:
#         return 'NA'


short_df['ACQ'] = short_df.ACQ_short.apply(ACQ_value)
short_df[['ACQ_short','ACQ']]

ACQ_short ACQ
0         None  NA
1         None  NA
2         None  NA
3         None  NA
4         None  NA
5         None  NA
6         None  NA
7         None  NA
8         None  NA
9         None  NA
10        None  NA
11        None  NA
12        None  NA
13        None  NA
14        None  NA
15        None  NA
16        None  NA
17        None  NA
18        None  NA
19        None  NA
20        None  NA
21        None  NA
22        None  NA
23        None  NA
24        None  NA
25        None  NA
26        None  NA
27        None  NA
28        None  NA
29        None  NA
...        ...  ..
8139      None  NA
8140      None  NA
8141      None  NA
8142      None  NA
8143      None  NA
8144      None  NA
8145      None  NA
8146      None  NA
8147      None  NA
8148      None  NA
8149      None  NA
8150      None  NA
8151      None  NA
8152      None  NA
8153      None  NA
8154      None  NA
8155      None  NA
8156      None  NA
8157      None  NA
8158      None  NA
8159      None  NA
8160      None  NA
8161      None  NA
8162      None  NA
8163      None  NA
8164      None  NA
8165      None  NA
8166      None  NA
8167      None  NA
8168      None  NA

[21787 rows x 2 columns]

In [100]:
# Emicizumab_df = short_df[short_df.Molecule =='Reslizumab']
# Emicizumab_df.shape

In [101]:
# Emicizumab_df[['ACQ_short','ACQ']][Emicizumab_df['ACQ_short']!='NA']

In [102]:
# Emicizumab_df.to_excel('/Users/savioel2/Desktop/HawkAISandoz/2022_release/Insights Module/interim_qlik/metrics/Reslizumab_acq.xlsx')




In [177]:
def EDSS_value(text): 
    text = str(text)             
#     try:
    orrstat= re.findall(r'[-+]?\d*\.?\d+\.\d{2,}|\b\d{1,2}(?:\.\d{1,2})*%?', text)       
    if orrstat:
        return orrstat[0]
#             if len(orrstat)>1:
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#             elif len(orrstat)==1:
# #                 print(orrstat)
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#                     else:
#                         return val

#             else:
#                 return 'NA'
    else:
        return 'NA'
#     except:
#         return 'NA'


short_df['EDSS'] = short_df.EDSS_short.apply(EDSS_value)
short_df[['EDSS_short','EDSS']]

EDSS_short EDSS
0                                                  None   NA
1                                                  None   NA
2                                                  None   NA
3                                                  None   NA
4                                                  None   NA
5                                                  None   NA
6                                                  None   NA
7                                                  None   NA
8                                                  None   NA
9                                                  None   NA
10                                                 None   NA
11                                                 None   NA
12                                                 None   NA
13                                                 None   NA
14                                                 None   NA
15                                                 None   NA
16                                                 None   NA
17                                                 None   NA
18                                                 None   NA
19                                                 None   NA
20                                                 None   NA
21                                                 None   NA
22                                                 None   NA
23                                                 None   NA
24                                                 None   NA
25                                                 None   NA
26    [EDSS ≥7.0 (hazard ratio=0.54 95% confidence i...  7.0
27                                                 None   NA
28                                                 None   NA
29                                                 None   NA
...                                                 ...  ...
8139                                               None   NA
8140                                               None   NA
8141                                               None   NA
8142                                               None   NA
8143                                               None   NA
8144                                               None   NA
8145                                               None   NA
8146                                               None   NA
8147                                               None   NA
8148                                               None   NA
8149                                               None   NA
8150                                               None   NA
8151                                               None   NA
8152                                               None   NA
8153                                               None   NA
8154                                               None   NA
8155                                               None   NA
8156                                               None   NA
8157                                               None   NA
8158                                               None   NA
8159                                               None   NA
8160                                               None   NA
8161                                               None   NA
8162                                               None   NA
8163                                               None   NA
8164                                               None   NA
8165                                               None   NA
8166                                               None   NA
8167                                               None   NA
8168                                               None   NA

[21787 rows x 2 columns]

In [178]:
def NEDA_value(text): 
    text = str(text)             
#     try:
    orrstat= re.findall(r'[-+]?\d*\.?\d+\.\d{2,}|\b\d{1,2}(?:\.\d{1,2})*%?', text)       
    if orrstat:
        return orrstat[0]
#             if len(orrstat)>1:
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#             elif len(orrstat)==1:
# #                 print(orrstat)
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#                     else:
#                         return val

#             else:
#                 return 'NA'
    else:
        return 'NA'
#     except:
#         return 'NA'


short_df['NEDA'] = short_df.NEDA_short.apply(NEDA_value)
short_df[['NEDA_short','NEDA']]

NEDA_short   NEDA
0                                                  None     NA
1                                                  None     NA
2                                                  None     NA
3                                                  None     NA
4                                                  None     NA
5                                                  None     NA
6                                                  None     NA
7                                                  None     NA
8                                                  None     NA
9                                                  None     NA
10                                                 None     NA
11                                                 None     NA
12                                                 None     NA
13                                                 None     NA
14                                                 None     NA
15                                                 None     NA
16                                                 None     NA
17                                                 None     NA
18                                                 None     NA
19                                                 None     NA
20                                                 None     NA
21                                                 None     NA
22                                                 None     NA
23                                                 None     NA
24    [NEDA was highest among patients enrolled due ...  80.6%
25                                                 None     NA
26                                                 None     NA
27                                                 None     NA
28    [NEDA was highest among patients enrolled due ...  80.6%
29                                                 None     NA
...                                                 ...    ...
8139                                               None     NA
8140                                               None     NA
8141                                               None     NA
8142                                               None     NA
8143                                               None     NA
8144                                               None     NA
8145                                               None     NA
8146                                               None     NA
8147                                               None     NA
8148                                               None     NA
8149                                               None     NA
8150                                               None     NA
8151                                               None     NA
8152                                               None     NA
8153                                               None     NA
8154                                               None     NA
8155                                               None     NA
8156                                               None     NA
8157                                               None     NA
8158                                               None     NA
8159                                               None     NA
8160                                               None     NA
8161                                               None     NA
8162                                               None     NA
8163                                               None     NA
8164                                               None     NA
8165                                               None     NA
8166                                               None     NA
8167                                               None     NA
8168                                               None     NA

[21787 rows x 2 columns]

In [179]:
def AQLQ_value(text): 
    text = str(text)             
#     try:
    orrstat= re.findall(r'[-+]?\d*\.?\d+\.\d{2,}|\b\d{1,2}(?:\.\d{1,2})*%?', text)       
    if orrstat:
        return orrstat[0]
#             if len(orrstat)>1:
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#             elif len(orrstat)==1:
# #                 print(orrstat)
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#                     else:
#                         return val

#             else:
#                 return 'NA'
    else:
        return 'NA'
#     except:
#         return 'NA'


short_df['AQLQ'] = short_df.AQLQ_short.apply(AQLQ_value)
short_df[['AQLQ_short','AQLQ']]

AQLQ_short AQLQ
0          None   NA
1          None   NA
2          None   NA
3          None   NA
4          None   NA
5          None   NA
6          None   NA
7          None   NA
8          None   NA
9          None   NA
10         None   NA
11         None   NA
12         None   NA
13         None   NA
14         None   NA
15         None   NA
16         None   NA
17         None   NA
18         None   NA
19         None   NA
20         None   NA
21         None   NA
22         None   NA
23         None   NA
24         None   NA
25         None   NA
26         None   NA
27         None   NA
28         None   NA
29         None   NA
...         ...  ...
8139       None   NA
8140       None   NA
8141       None   NA
8142       None   NA
8143       None   NA
8144       None   NA
8145       None   NA
8146       None   NA
8147       None   NA
8148       None   NA
8149       None   NA
8150       None   NA
8151       None   NA
8152       None   NA
8153       None   NA
8154       None   NA
8155       None   NA
8156       None   NA
8157       None   NA
8158       None   NA
8159       None   NA
8160       None   NA
8161       None   NA
8162       None   NA
8163       None   NA
8164       None   NA
8165       None   NA
8166       None   NA
8167       None   NA
8168       None   NA

[21787 rows x 2 columns]

In [180]:
def ACQ_value(text): 
    text = str(text)             
#     try:
    orrstat= re.findall(r'[-+]?\d*\.?\d+\.\d{2,}|\b\d{1,2}(?:\.\d{1,2})*%?', text)       
    if orrstat:
        return orrstat[0]
#             if len(orrstat)>1:
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#             elif len(orrstat)==1:
# #                 print(orrstat)
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#                     else:
#                         return val

#             else:
#                 return 'NA'
    else:
        return 'NA'
#     except:
#         return 'NA'


short_df['ACQ'] = short_df.ACQ_short.apply(ACQ_value)
short_df[['ACQ_short','ACQ']]

ACQ_short ACQ
0         None  NA
1         None  NA
2         None  NA
3         None  NA
4         None  NA
5         None  NA
6         None  NA
7         None  NA
8         None  NA
9         None  NA
10        None  NA
11        None  NA
12        None  NA
13        None  NA
14        None  NA
15        None  NA
16        None  NA
17        None  NA
18        None  NA
19        None  NA
20        None  NA
21        None  NA
22        None  NA
23        None  NA
24        None  NA
25        None  NA
26        None  NA
27        None  NA
28        None  NA
29        None  NA
...        ...  ..
8139      None  NA
8140      None  NA
8141      None  NA
8142      None  NA
8143      None  NA
8144      None  NA
8145      None  NA
8146      None  NA
8147      None  NA
8148      None  NA
8149      None  NA
8150      None  NA
8151      None  NA
8152      None  NA
8153      None  NA
8154      None  NA
8155      None  NA
8156      None  NA
8157      None  NA
8158      None  NA
8159      None  NA
8160      None  NA
8161      None  NA
8162      None  NA
8163      None  NA
8164      None  NA
8165      None  NA
8166      None  NA
8167      None  NA
8168      None  NA

[21787 rows x 2 columns]

In [181]:
def AQoL_value(text): 
    text = str(text)             
#     try:
    orrstat= re.findall(r'[-+]?\d*\.?\d+\.\d{2,}|\b\d{1,2}(?:\.\d{1,2})*%?', text)       
    if orrstat:
        return orrstat[0]
#             if len(orrstat)>1:
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#             elif len(orrstat)==1:
# #                 print(orrstat)
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#                     else:
#                         return val

#             else:
#                 return 'NA'
    else:
        return 'NA'
#     except:
#         return 'NA'


short_df['AQoL'] = short_df.AQLQ_short.apply(AQoL_value)
short_df[['AQLQ_short','AQoL']]

AQLQ_short AQoL
0          None   NA
1          None   NA
2          None   NA
3          None   NA
4          None   NA
5          None   NA
6          None   NA
7          None   NA
8          None   NA
9          None   NA
10         None   NA
11         None   NA
12         None   NA
13         None   NA
14         None   NA
15         None   NA
16         None   NA
17         None   NA
18         None   NA
19         None   NA
20         None   NA
21         None   NA
22         None   NA
23         None   NA
24         None   NA
25         None   NA
26         None   NA
27         None   NA
28         None   NA
29         None   NA
...         ...  ...
8139       None   NA
8140       None   NA
8141       None   NA
8142       None   NA
8143       None   NA
8144       None   NA
8145       None   NA
8146       None   NA
8147       None   NA
8148       None   NA
8149       None   NA
8150       None   NA
8151       None   NA
8152       None   NA
8153       None   NA
8154       None   NA
8155       None   NA
8156       None   NA
8157       None   NA
8158       None   NA
8159       None   NA
8160       None   NA
8161       None   NA
8162       None   NA
8163       None   NA
8164       None   NA
8165       None   NA
8166       None   NA
8167       None   NA
8168       None   NA

[21787 rows x 2 columns]

In [182]:
def ACR_value(text): 
    text = str(text)             
#     try:
#     orrstat= re.findall(r'.*\d*[.,]%.*\d*%', text)   
    orrstat= re.findall(r'[-+]?\d*\.?\d+%|\d+%', text)
    if orrstat:
        return orrstat[0]
#             if len(orrstat)>1:
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#             elif len(orrstat)==1:
# #                 print(orrstat)
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#                     else:
#                         return val

#             else:
#                 return 'NA'
    else:
        return ''
#     except:
#         return 'NA'


short_df['ACR'] = short_df.ACR_short.apply(ACR_value)
short_df[['ACR_short','ACR']]

ACR_short    ACR
0                                                  None       
1                                                  None       
2                                                  None       
3                                                  None       
4                                                  None       
5                                                  None       
6                                                  None       
7                                                  None       
8                                                  None       
9                                                  None       
10                                                 None       
11                                                 None       
12                                                 None       
13                                                 None       
14                                                 None       
15                                                 None       
16                                                 None       
17                                                 None       
18                                                 None       
19                                                 None       
20                                                 None       
21    [ACR20 were infliximab+methotrexate vs. methot...    95%
22                                                 None       
23                                                 None       
24                                                 None       
25                                                 None       
26                                                 None       
27                                                 None       
28                                                 None       
29    [ACR20 responses were 22.0% in the placebo gro...  22.0%
...                                                 ...    ...
8139                                               None       
8140                                               None       
8141                                               None       
8142                                               None       
8143                                               None       
8144                                               None       
8145                                               None       
8146                                               None       
8147                                               None       
8148                                               None       
8149                                               None       
8150                                               None       
8151                                               None       
8152                                               None       
8153                                               None       
8154                                               None       
8155                                               None       
8156                                               None       
8157                                               None       
8158                                               None       
8159                                               None       
8160                                               None       
8161                                               None       
8162                                               None       
8163                                               None       
8164                                               None       
8165                                               None       
8166                                               None       
8167                                               None       
8168                                               None       

[21787 rows x 2 columns]

In [183]:
def sPGA_value(text): 
    text = str(text)             
#     try:
#     orrstat= re.findall(r'.*\d*[.,]%.*\d*%', text)   
    orrstat= re.findall(r'[-+]?\d*\.?\d+%|\d+%', text)
    if orrstat:
        return orrstat[0]
#             if len(orrstat)>1:
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#             elif len(orrstat)==1:
# #                 print(orrstat)
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#                     else:
#                         return val

#             else:
#                 return 'NA'
    else:
        return ''
#     except:
#         return 'NA'


short_df['sPGA'] = short_df.sPGA_short.apply(sPGA_value)
short_df[['sPGA_short','sPGA']]

sPGA_short sPGA
0          None     
1          None     
2          None     
3          None     
4          None     
5          None     
6          None     
7          None     
8          None     
9          None     
10         None     
11         None     
12         None     
13         None     
14         None     
15         None     
16         None     
17         None     
18         None     
19         None     
20         None     
21         None     
22         None     
23         None     
24         None     
25         None     
26         None     
27         None     
28         None     
29         None     
...         ...  ...
8139       None     
8140       None     
8141       None     
8142       None     
8143       None     
8144       None     
8145       None     
8146       None     
8147       None     
8148       None     
8149       None     
8150       None     
8151       None     
8152       None     
8153       None     
8154       None     
8155       None     
8156       None     
8157       None     
8158       None     
8159       None     
8160       None     
8161       None     
8162       None     
8163       None     
8164       None     
8165       None     
8166       None     
8167       None     
8168       None     

[21787 rows x 2 columns]

In [184]:
def TSS_value(text): 
    text = str(text)             
#     try:
    orrstat= re.findall(r'\b\d{1,2}(?:\.\d{1,2})', text)      
    if orrstat:
        return orrstat[0]
#             if len(orrstat)>1:
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#             elif len(orrstat)==1:
# #                 print(orrstat)
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#                     else:
#                         return val

#             else:
#                 return 'NA'
    else:
        return 'NA'
#     except:
#         return 'NA'


short_df['TSS'] = short_df.TSS_short.apply(TSS_value)
short_df[['TSS_short','TSS']]

TSS_short TSS
0         None  NA
1         None  NA
2         None  NA
3         None  NA
4         None  NA
5         None  NA
6         None  NA
7         None  NA
8         None  NA
9         None  NA
10        None  NA
11        None  NA
12        None  NA
13        None  NA
14        None  NA
15        None  NA
16        None  NA
17        None  NA
18        None  NA
19        None  NA
20        None  NA
21        None  NA
22        None  NA
23        None  NA
24        None  NA
25        None  NA
26        None  NA
27        None  NA
28        None  NA
29        None  NA
...        ...  ..
8139      None  NA
8140      None  NA
8141      None  NA
8142      None  NA
8143      None  NA
8144      None  NA
8145      None  NA
8146      None  NA
8147      None  NA
8148      None  NA
8149      None  NA
8150      None  NA
8151      None  NA
8152      None  NA
8153      None  NA
8154      None  NA
8155      None  NA
8156      None  NA
8157      None  NA
8158      None  NA
8159      None  NA
8160      None  NA
8161      None  NA
8162      None  NA
8163      None  NA
8164      None  NA
8165      None  NA
8166      None  NA
8167      None  NA
8168      None  NA

[21787 rows x 2 columns]

In [185]:
def MRI_value(text): 
    text = str(text)             
#     try:
#     orrstat= re.findall(r'.*\d*[.,]%.*\d*%', text)   
    orrstat= re.findall(r'[-+]?\d*\.?\d+%|\d+%', text)
    if orrstat:
        return orrstat[0]
#             if len(orrstat)>1:
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#             elif len(orrstat)==1:
# #                 print(orrstat)
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#                     else:
#                         return val

#             else:
#                 return 'NA'
    else:
        return ''
#     except:
#         return 'NA'


short_df['MRI'] = short_df.MRI_short.apply(MRI_value)
short_df[['MRI_short','MRI']]

MRI_short    MRI
0                                                  None       
1                                                  None       
2                                                  None       
3                                                  None       
4                                                  None       
5                                                  None       
6                                                  None       
7                                                  None       
8                                                  None       
9                                                  None       
10    [MRI) status the number of natalizumab infusio...       
11                                                 None       
12                                                 None       
13                                                 None       
14                                                 None       
15                                                 None       
16                                                 None       
17                                                 None       
18                                                 None       
19                                                 None       
20                                                 None       
21                                                 None       
22                                                 None       
23                                                 None       
24    [MRI activity alone (80.6% [95% CI 68.6–89.6] ...  80.6%
25                                                 None       
26                                                 None       
27                                                 None       
28    [MRI activity alone (80.6% [95% CI 68.6–89.6] ...  80.6%
29                                                 None       
...                                                 ...    ...
8139                                               None       
8140                                               None       
8141                                               None       
8142                                               None       
8143                                               None       
8144                                               None       
8145                                               None       
8146                                               None       
8147                                               None       
8148                                               None       
8149                                               None       
8150                                               None       
8151                                               None       
8152                                               None       
8153                                               None       
8154                                               None       
8155                                               None       
8156                                               None       
8157                                               None       
8158                                               None       
8159                                               None       
8160                                               None       
8161                                               None       
8162                                               None       
8163                                               None       
8164                                               None       
8165                                               None       
8166                                               None       
8167                                               None       
8168                                               None       

[21787 rows x 2 columns]

In [186]:
def PASI_value(text): 
    text = str(text)             
#     try:
#     orrstat= re.findall(r'.*\d*[.,]%.*\d*%', text)   
    orrstat= re.findall(r'[-+]?\d*\.?\d+%|\d+%', text) 
    if orrstat:
        return orrstat[0]
#             if len(orrstat)>1:
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#             elif len(orrstat)==1:
# #                 print(orrstat)
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#                     else:
#                         return val

#             else:
#                 return 'NA'
    else:
        return ''
#     except:
#         return 'NA'


short_df['PASI'] = short_df.PASI_short.apply(PASI_value)
short_df[['PASI_short','PASI']]

PASI_short PASI
0                                                  None     
1                                                  None     
2                                                  None     
3                                                  None     
4                                                  None     
5                                                  None     
6                                                  None     
7                                                  None     
8                                                  None     
9                                                  None     
10                                                 None     
11                                                 None     
12                                                 None     
13                                                 None     
14                                                 None     
15                                                 None     
16                                                 None     
17                                                 None     
18                                                 None     
19                                                 None     
20                                                 None     
21                                                 None     
22                                                 None     
23                                                 None     
24                                                 None     
25                                                 None     
26                                                 None     
27                                                 None     
28                                                 None     
29                                                 None     
...                                                 ...  ...
8139                                               None     
8140                                               None     
8141                                               None     
8142                                               None     
8143                                               None     
8144                                               None     
8145                                               None     
8146                                               None     
8147                                               None     
8148                                               None     
8149                                               None     
8150                                               None     
8151                                               None     
8152                                               None     
8153                                               None     
8154                                               None     
8155                                               None     
8156                                               None     
8157                                               None     
8158                                               None     
8159                                               None     
8160                                               None     
8161                                               None     
8162                                               None     
8163                                               None     
8164  [PASI 75 response was achieved by 10 of 22 pat...  45%
8165                                               None     
8166                                               None     
8167                                               None     
8168                                               None     

[21787 rows x 2 columns]

In [187]:
def DAS_value(text): 
    text = str(text)             
#     try:
#     orrstat= re.findall(r'.*\d*[.,]%.*\d*%', text)   
    orrstat= re.findall(r'[-+]?\d*\.?\d+%|\d+%', text) 
    if orrstat:
        return orrstat[0]
#             if len(orrstat)>1:
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#             elif len(orrstat)==1:
# #                 print(orrstat)
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#                     else:
#                         return val

#             else:
#                 return 'NA'
    else:
        return ''
#     except:
#         return 'NA'


short_df['DAS'] = short_df.DAS_short.apply(DAS_value)
short_df[['DAS_short','DAS']]

DAS_short DAS
0         None    
1         None    
2         None    
3         None    
4         None    
5         None    
6         None    
7         None    
8         None    
9         None    
10        None    
11        None    
12        None    
13        None    
14        None    
15        None    
16        None    
17        None    
18        None    
19        None    
20        None    
21        None    
22        None    
23        None    
24        None    
25        None    
26        None    
27        None    
28        None    
29        None    
...        ...  ..
8139      None    
8140      None    
8141      None    
8142      None    
8143      None    
8144      None    
8145      None    
8146      None    
8147      None    
8148      None    
8149      None    
8150      None    
8151      None    
8152      None    
8153      None    
8154      None    
8155      None    
8156      None    
8157      None    
8158      None    
8159      None    
8160      None    
8161      None    
8162      None    
8163      None    
8164      None    
8165      None    
8166      None    
8167      None    
8168      None    

[21787 rows x 2 columns]

In [102]:
short_df.columns.to_list()

['XML',
 'Title',
 'Full_body_text',
 'Abstract_full',
 'DOI',
 'URL',
 'Molecule',
 'Publisher_type',
 'Molecule_text',
 'Intervention',
 'STRENGTH',
 'DRUG',
 'Molecule_in_Title',
 'DISEASE_abstract',
 'DISEASE_title',
 'DISEASE',
 'Title_short',
 'combined_text',
 '1st_line/2nd_line',
 'Patients',
 'ORR_abstract',
 'ORR_fulltext',
 'ORR_short',
 'OS_short',
 'PFS_short',
 'Comparator',
 'Arms',
 'Mono/Combi',
 'phase',
 'PHASES',
 'phase2',
 'PHASES2',
 'Age',
 'Age_Number',
 'age_groups',
 'Age2',
 'Age2_value',
 'Age2_number',
 'Age2_groups',
 'Unnamed: 0.3',
 'Unnamed: 0.2',
 'Unnamed: 0.1',
 'Unnamed: 0',
 'phase21',
 'Intervention2',
 'DOSAGE',
 'Number of evaluable subjects',
 'ORR',
 'OS_percentage',
 'OS_month',
 'PFS_percentage',
 'PFS_month',
 'DOSAGE_VAL',
 'exacerbation_abstract',
 'exacerbation_fulltext',
 'exacerbation_short',
 'NPS_abstract',
 'NPS_fulltext',
 'NPS_short',
 'ARR_abstract',
 'ARR_fulltext',
 'ARR_short',
 'EDSS_abstract',
 'EDSS_fulltext',
 'EDSS_short

In [188]:
def ES_value(text): 
    text = str(text)             
#     try:
#     orrstat= re.findall(r'.*\d*[.,]%.*\d*%', text)   
    orrstat= re.findall(r'[-+]?\d*\.?\d+%|\d+%', text)
    if orrstat:
        return orrstat[0]
#             if len(orrstat)>1:
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#             elif len(orrstat)==1:
# #                 print(orrstat)
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#                     else:
#                         return val

#             else:
#                 return 'NA'
    else:
        return ''
#     except:
#         return 'NA'


short_df['ES'] = short_df.ES_short.apply(ES_value)
short_df[['ES_short','ES']]

ES_short ES
0        None   
1        None   
2        None   
3        None   
4        None   
5        None   
6        None   
7        None   
8        None   
9        None   
10       None   
11       None   
12       None   
13       None   
14       None   
15       None   
16       None   
17       None   
18       None   
19       None   
20       None   
21       None   
22       None   
23       None   
24       None   
25       None   
26       None   
27       None   
28       None   
29       None   
...       ... ..
8139     None   
8140     None   
8141     None   
8142     None   
8143     None   
8144     None   
8145     None   
8146     None   
8147     None   
8148     None   
8149     None   
8150     None   
8151     None   
8152     None   
8153     None   
8154     None   
8155     None   
8156     None   
8157     None   
8158     None   
8159     None   
8160     None   
8161     None   
8162     None   
8163     None   
8164     None   
8165     None   
8166     None   
8167     None   
8168     None   

[21787 rows x 2 columns]

In [189]:
def IGA_value(text): 
    text = str(text)             
#     try:
#     orrstat= re.findall(r'.*\d*[.,]%.*\d*%', text)   
    orrstat= re.findall(r'[-+]?\d*\.?\d+%|\d+%', text)
    if orrstat:
        return orrstat[0]
#             if len(orrstat)>1:
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#             elif len(orrstat)==1:
# #                 print(orrstat)
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#                     else:
#                         return val

#             else:
#                 return 'NA'
    else:
        return ''
#     except:
#         return 'NA'


short_df['IGA'] = short_df.IGA_short.apply(IGA_value)
short_df[['IGA_short','IGA']]

IGA_short IGA
0         None    
1         None    
2         None    
3         None    
4         None    
5         None    
6         None    
7         None    
8         None    
9         None    
10        None    
11        None    
12        None    
13        None    
14        None    
15        None    
16        None    
17        None    
18        None    
19        None    
20        None    
21        None    
22        None    
23        None    
24        None    
25        None    
26        None    
27        None    
28        None    
29        None    
...        ...  ..
8139      None    
8140      None    
8141      None    
8142      None    
8143      None    
8144      None    
8145      None    
8146      None    
8147      None    
8148      None    
8149      None    
8150      None    
8151      None    
8152      None    
8153      None    
8154      None    
8155      None    
8156      None    
8157      None    
8158      None    
8159      None    
8160      None    
8161      None    
8162      None    
8163      None    
8164      None    
8165      None    
8166      None    
8167      None    
8168      None    

[21787 rows x 2 columns]

In [190]:
def CRP_value(text): 
    text = str(text)             
#     try:
#     orrstat= re.findall(r'.*\d*[.,]%.*\d*%', text)   
    orrstat= re.findall(r'[-+]?\d*\.?\d+ * mg/l|\d*\.?\d+ * mg/L|\d*\.?\d+ * mg/dl', text)
    
#     (r'[-+]?\d*\.?\d+ *month|\d+ *month|[-+]?\d*\.?\d+ *mo|[-+]?\d*\.?\d+ week|\d+ *week', text)
    if orrstat:
        return orrstat[0]
#             if len(orrstat)>1:
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#             elif len(orrstat)==1:
# #                 print(orrstat)
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#                     else:
#                         return val

#             else:
#                 return 'NA'
    else:
        return ''
#     except:
#         return 'NA'


short_df['CRP'] = short_df.CRP_short.apply(CRP_value)
short_df[['CRP_short','CRP']]

CRP_short CRP
0         None    
1         None    
2         None    
3         None    
4         None    
5         None    
6         None    
7         None    
8         None    
9         None    
10        None    
11        None    
12        None    
13        None    
14        None    
15        None    
16        None    
17        None    
18        None    
19        None    
20        None    
21        None    
22        None    
23        None    
24        None    
25        None    
26        None    
27        None    
28        None    
29        None    
...        ...  ..
8139      None    
8140      None    
8141      None    
8142      None    
8143      None    
8144      None    
8145      None    
8146      None    
8147      None    
8148      None    
8149      None    
8150      None    
8151      None    
8152      None    
8153      None    
8154      None    
8155      None    
8156      None    
8157      None    
8158      None    
8159      None    
8160      None    
8161      None    
8162      None    
8163      None    
8164      None    
8165      None    
8166      None    
8167      None    
8168      None    

[21787 rows x 2 columns]

In [191]:
def IGA_value(text): 
    text = str(text)             
#     try:
#     orrstat= re.findall(r'.*\d*[.,]%.*\d*%', text)   
    orrstat= re.findall(r'[-+]?\d*\.?\d+%|\d+%', text)
    if orrstat:
        return orrstat[0]
#             if len(orrstat)>1:
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#             elif len(orrstat)==1:
# #                 print(orrstat)
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#                     else:
#                         return val

#             else:
#                 return 'NA'
    else:
        return ''
#     except:
#         return 'NA'


short_df['IGA'] = short_df.IGA_short.apply(IGA_value)
short_df[['IGA_short','IGA']]

IGA_short IGA
0         None    
1         None    
2         None    
3         None    
4         None    
5         None    
6         None    
7         None    
8         None    
9         None    
10        None    
11        None    
12        None    
13        None    
14        None    
15        None    
16        None    
17        None    
18        None    
19        None    
20        None    
21        None    
22        None    
23        None    
24        None    
25        None    
26        None    
27        None    
28        None    
29        None    
...        ...  ..
8139      None    
8140      None    
8141      None    
8142      None    
8143      None    
8144      None    
8145      None    
8146      None    
8147      None    
8148      None    
8149      None    
8150      None    
8151      None    
8152      None    
8153      None    
8154      None    
8155      None    
8156      None    
8157      None    
8158      None    
8159      None    
8160      None    
8161      None    
8162      None    
8163      None    
8164      None    
8165      None    
8166      None    
8167      None    
8168      None    

[21787 rows x 2 columns]

In [192]:
def IGA_value(text): 
    text = str(text)             
#     try:
#     orrstat= re.findall(r'.*\d*[.,]%.*\d*%', text)   
    orrstat= re.findall(r'[-+]?\d*\.?\d+%|\d+%', text)
    if orrstat:
        return orrstat[0]
#             if len(orrstat)>1:
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#             elif len(orrstat)==1:
# #                 print(orrstat)
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#                     else:
#                         return val

#             else:
#                 return 'NA'
    else:
        return ''
#     except:
#         return 'NA'


short_df['IGA'] = short_df.IGA_short.apply(IGA_value)
short_df[['IGA_short','IGA']]

IGA_short IGA
0         None    
1         None    
2         None    
3         None    
4         None    
5         None    
6         None    
7         None    
8         None    
9         None    
10        None    
11        None    
12        None    
13        None    
14        None    
15        None    
16        None    
17        None    
18        None    
19        None    
20        None    
21        None    
22        None    
23        None    
24        None    
25        None    
26        None    
27        None    
28        None    
29        None    
...        ...  ..
8139      None    
8140      None    
8141      None    
8142      None    
8143      None    
8144      None    
8145      None    
8146      None    
8147      None    
8148      None    
8149      None    
8150      None    
8151      None    
8152      None    
8153      None    
8154      None    
8155      None    
8156      None    
8157      None    
8158      None    
8159      None    
8160      None    
8161      None    
8162      None    
8163      None    
8164      None    
8165      None    
8166      None    
8167      None    
8168      None    

[21787 rows x 2 columns]

In [108]:
def IGA_value(text): 
    text = str(text)             
#     try:
#     orrstat= re.findall(r'.*\d*[.,]%.*\d*%', text)   
    orrstat= re.findall(r'[-+]?\d*\.?\d+%|\d+%', text)
    if orrstat:
        return orrstat[0]
#             if len(orrstat)>1:
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#             elif len(orrstat)==1:
# #                 print(orrstat)
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#                     else:
#                         return val

#             else:
#                 return 'NA'
    else:
        return ''
#     except:
#         return 'NA'


short_df['IGA'] = short_df.IGA_short.apply(IGA_value)
short_df[['IGA_short','IGA']]

IGA_short IGA
0         None    
1         None    
2         None    
3         None    
4         None    
5         None    
6         None    
7         None    
8         None    
9         None    
10        None    
11        None    
12        None    
13        None    
14        None    
15        None    
16        None    
17        None    
18        None    
19        None    
20        None    
21        None    
22        None    
23        None    
24        None    
25        None    
26        None    
27        None    
28        None    
29        None    
...        ...  ..
8139      None    
8140      None    
8141      None    
8142      None    
8143      None    
8144      None    
8145      None    
8146      None    
8147      None    
8148      None    
8149      None    
8150      None    
8151      None    
8152      None    
8153      None    
8154      None    
8155      None    
8156      None    
8157      None    
8158      None    
8159      None    
8160      None    
8161      None    
8162      None    
8163      None    
8164      None    
8165      None    
8166      None    
8167      None    
8168      None    

[21787 rows x 2 columns]

In [109]:
def IGA_value(text): 
    text = str(text)             
#     try:
#     orrstat= re.findall(r'.*\d*[.,]%.*\d*%', text)   
    orrstat= re.findall(r'[-+]?\d*\.?\d+%|\d+%', text)
    if orrstat:
        return orrstat[0]
#             if len(orrstat)>1:
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#             elif len(orrstat)==1:
# #                 print(orrstat)
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#                     else:
#                         return val

#             else:
#                 return 'NA'
    else:
        return ''
#     except:
#         return 'NA'


short_df['IGA'] = short_df.IGA_short.apply(IGA_value)
short_df[['IGA_short','IGA']]

IGA_short IGA
0         None    
1         None    
2         None    
3         None    
4         None    
5         None    
6         None    
7         None    
8         None    
9         None    
10        None    
11        None    
12        None    
13        None    
14        None    
15        None    
16        None    
17        None    
18        None    
19        None    
20        None    
21        None    
22        None    
23        None    
24        None    
25        None    
26        None    
27        None    
28        None    
29        None    
...        ...  ..
8139      None    
8140      None    
8141      None    
8142      None    
8143      None    
8144      None    
8145      None    
8146      None    
8147      None    
8148      None    
8149      None    
8150      None    
8151      None    
8152      None    
8153      None    
8154      None    
8155      None    
8156      None    
8157      None    
8158      None    
8159      None    
8160      None    
8161      None    
8162      None    
8163      None    
8164      None    
8165      None    
8166      None    
8167      None    
8168      None    

[21787 rows x 2 columns]

In [193]:
def NRS_value(text): 
    text = str(text)             
#     try:
    orrstat= re.findall(r'\b\d{1,2}(?:\.\d{1,2})', text)      
    if orrstat:
        return orrstat[0]
#             if len(orrstat)>1:
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#             elif len(orrstat)==1:
# #                 print(orrstat)
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#                     else:
#                         return val

#             else:
#                 return 'NA'
    else:
        return 'NA'
#     except:
#         return 'NA'


short_df['NRS'] = short_df.NRS_short.apply(NRS_value)
short_df[['NRS_short','NRS']]

NRS_short NRS
0         None  NA
1         None  NA
2         None  NA
3         None  NA
4         None  NA
5         None  NA
6         None  NA
7         None  NA
8         None  NA
9         None  NA
10        None  NA
11        None  NA
12        None  NA
13        None  NA
14        None  NA
15        None  NA
16        None  NA
17        None  NA
18        None  NA
19        None  NA
20        None  NA
21        None  NA
22        None  NA
23        None  NA
24        None  NA
25        None  NA
26        None  NA
27        None  NA
28        None  NA
29        None  NA
...        ...  ..
8139      None  NA
8140      None  NA
8141      None  NA
8142      None  NA
8143      None  NA
8144      None  NA
8145      None  NA
8146      None  NA
8147      None  NA
8148      None  NA
8149      None  NA
8150      None  NA
8151      None  NA
8152      None  NA
8153      None  NA
8154      None  NA
8155      None  NA
8156      None  NA
8157      None  NA
8158      None  NA
8159      None  NA
8160      None  NA
8161      None  NA
8162      None  NA
8163      None  NA
8164      None  NA
8165      None  NA
8166      None  NA
8167      None  NA
8168      None  NA

[21787 rows x 2 columns]

In [194]:
def ASAS_value(text): 
    text = str(text)             
#     try:
#     orrstat= re.findall(r'\b\d{1,2}(?:\.\d{1,2})', text)   
    orrstat= re.findall(r'[-+]?\d*\.?\d+%|\d+%', text)
    if orrstat:
        return orrstat[0]
#             if len(orrstat)>1:
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#             elif len(orrstat)==1:
# #                 print(orrstat)
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#                     else:
#                         return val

#             else:
#                 return 'NA'
    else:
        return 'NA'
#     except:
#         return 'NA'


short_df['ASAS'] = short_df.ASAS_short.apply(ASAS_value)
short_df[['ASAS_short','ASAS']]

ASAS_short ASAS
0          None   NA
1          None   NA
2          None   NA
3          None   NA
4          None   NA
5          None   NA
6          None   NA
7          None   NA
8          None   NA
9          None   NA
10         None   NA
11         None   NA
12         None   NA
13         None   NA
14         None   NA
15         None   NA
16         None   NA
17         None   NA
18         None   NA
19         None   NA
20         None   NA
21         None   NA
22         None   NA
23         None   NA
24         None   NA
25         None   NA
26         None   NA
27         None   NA
28         None   NA
29         None   NA
...         ...  ...
8139       None   NA
8140       None   NA
8141       None   NA
8142       None   NA
8143       None   NA
8144       None   NA
8145       None   NA
8146       None   NA
8147       None   NA
8148       None   NA
8149       None   NA
8150       None   NA
8151       None   NA
8152       None   NA
8153       None   NA
8154       None   NA
8155       None   NA
8156       None   NA
8157       None   NA
8158       None   NA
8159       None   NA
8160       None   NA
8161       None   NA
8162       None   NA
8163       None   NA
8164       None   NA
8165       None   NA
8166       None   NA
8167       None   NA
8168       None   NA

[21787 rows x 2 columns]

In [195]:
def BASDAI_value(text): 
    text = str(text)             
#     try:
#     orrstat= re.findall(r'\b\d{1,2}(?:\.\d{1,2})', text)   
    orrstat= re.findall(r'[-+]?\d*\.?\d+%|\d+%', text)
    if orrstat:
        return orrstat[0]
#             if len(orrstat)>1:
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#             elif len(orrstat)==1:
# #                 print(orrstat)
#                 for val in orrstat:
#                     if '%' in val:
#                         return val
#                     else:
#                         return val

#             else:
#                 return 'NA'
    else:
        return 'NA'
#     except:
#         return 'NA'


short_df['BASDAI'] = short_df.BASDAI_short.apply(BASDAI_value)
short_df[['BASDAI_short','BASDAI']]

BASDAI_short BASDAI
0            None     NA
1            None     NA
2            None     NA
3            None     NA
4            None     NA
5            None     NA
6            None     NA
7            None     NA
8            None     NA
9            None     NA
10           None     NA
11           None     NA
12           None     NA
13           None     NA
14           None     NA
15           None     NA
16           None     NA
17           None     NA
18           None     NA
19           None     NA
20           None     NA
21           None     NA
22           None     NA
23           None     NA
24           None     NA
25           None     NA
26           None     NA
27           None     NA
28           None     NA
29           None     NA
...           ...    ...
8139         None     NA
8140         None     NA
8141         None     NA
8142         None     NA
8143         None     NA
8144         None     NA
8145         None     NA
8146         None     NA
8147         None     NA
8148         None     NA
8149         None     NA
8150         None     NA
8151         None     NA
8152         None     NA
8153         None     NA
8154         None     NA
8155         None     NA
8156         None     NA
8157         None     NA
8158         None     NA
8159         None     NA
8160         None     NA
8161         None     NA
8162         None     NA
8163         None     NA
8164         None     NA
8165         None     NA
8166         None     NA
8167         None     NA
8168         None     NA

[21787 rows x 2 columns]

In [113]:
test_df = short_df[short_df.Molecule =='Emicizumab']
test_df.shape

(76, 161)

In [126]:
test_df[['BASDAI_short','BASDAI']][test_df['BASDAI']!='NA'].tail(60)

Empty DataFrame
Columns: [BASDAI_short, BASDAI]
Index: []

In [114]:
test_df.columns.to_list()

['XML',
 'Title',
 'Full_body_text',
 'Abstract_full',
 'DOI',
 'URL',
 'Molecule',
 'Publisher_type',
 'Molecule_text',
 'Intervention',
 'STRENGTH',
 'DRUG',
 'Molecule_in_Title',
 'DISEASE_abstract',
 'DISEASE_title',
 'DISEASE',
 'Title_short',
 'combined_text',
 '1st_line/2nd_line',
 'Patients',
 'ORR_abstract',
 'ORR_fulltext',
 'ORR_short',
 'OS_short',
 'PFS_short',
 'Comparator',
 'Arms',
 'Mono/Combi',
 'phase',
 'PHASES',
 'phase2',
 'PHASES2',
 'Age',
 'Age_Number',
 'age_groups',
 'Age2',
 'Age2_value',
 'Age2_number',
 'Age2_groups',
 'Unnamed: 0.3',
 'Unnamed: 0.2',
 'Unnamed: 0.1',
 'Unnamed: 0',
 'phase21',
 'Intervention2',
 'DOSAGE',
 'Number of evaluable subjects',
 'ORR',
 'OS_percentage',
 'OS_month',
 'PFS_percentage',
 'PFS_month',
 'DOSAGE_VAL',
 'exacerbation_abstract',
 'exacerbation_fulltext',
 'exacerbation_short',
 'NPS_abstract',
 'NPS_fulltext',
 'NPS_short',
 'ARR_abstract',
 'ARR_fulltext',
 'ARR_short',
 'EDSS_abstract',
 'EDSS_fulltext',
 'EDSS_short

In [115]:
new_mol_df = short_df[['Molecule','Title','Title_short','PHASES', 'Patients',
                                   '1st_line/2nd_line','Intervention','Comparator',
                      'DOSAGE_VAL','URL','DISEASE_title','Mono/Combi',
                      'Arms','Age','age_groups','BASDAI']]
new_mol_df.shape

(21787, 16)

In [116]:
new_mol_df.head(20) 

Molecule                                              Title  \
0   Natalizumab  Review article: anti-adhesion therapies for in...   
1   Natalizumab  Treatment options for patients with multiple s...   
2   Natalizumab  Intravenous immunoglobulin for maintenance tre...   
3   Natalizumab  Natalizumab improves ambulation in relapsing&#...   
4   Natalizumab  Application of Quantitative Pharmacology Appro...   
5   Natalizumab  Current and emerging disease-modulatory therap...   
6   Natalizumab  Immunoadsorption and plasma exchange&#8212;Eff...   
7   Natalizumab  Modeling the Efficacy of Natalizumab in Multip...   
8   Natalizumab  Clinical pharmacology, safety and effects of a...   
9   Natalizumab  Paediatric multiple sclerosis: a new era in di...   
10  Natalizumab  Effect of natalizumab on brain atrophy and dis...   
11  Natalizumab  Therapeutic Application of Monoclonal Antibodi...   
12  Natalizumab  A Randomized Trial Evaluating Various Administ...   
13  Natalizumab  Established and novel disease-modifying treatm...   
14  Natalizumab  Wielding the Double-Edged Sword of Inflammatio...   
15  Natalizumab  Wielding the Double-Edged Sword of Inflammatio...   
16  Natalizumab  Monoclonal antibodies in the treatment of mult...   
17  Natalizumab  Current and emerging disease-modulatory therap...   
18  Natalizumab  Modeling the Efficacy of Natalizumab in Multip...   
19  Natalizumab  Immunoadsorption and plasma exchange&#8212;Eff...   

                                          Title_short   PHASES  \
0   Review article: anti-adhesion therapies for in...  Phase 3   
1   Treatment options for patients with multiple s...      NaN   
2   Intravenous immunoglobulin for maintenance tre...  Phase 3   
3   Natalizumab improves ambulation in relapsing&#...      NaN   
4   Application of Quantitative Pharmacology Appro...      NaN   
5   Current and emerging disease-modulatory therap...  Phase 3   
6   Immunoadsorption and plasma exchange&#8212;Eff...      NaN   
7   Modeling the Efficacy of Natalizumab in Multip...      NaN   
8   Clinical pharmacology, safety and effects of a...      NaN   
9   Paediatric multiple sclerosis: a new era in di...      NaN   
10  Effect of natalizumab on brain atrophy and dis...      NaN   
11  Therapeutic Application of Monoclonal Antibodi...      NaN   
12  A Randomized Trial Evaluating Various Administ...      NaN   
13  Established and novel disease-modifying treatm...      NaN   
14  Wielding the Double-Edged Sword of Inflammatio...      NaN   
15  Wielding the Double-Edged Sword of Inflammatio...      NaN   
16  Monoclonal antibodies in the treatment of mult...      NaN   
17  Current and emerging disease-modulatory therap...  Phase 3   
18  Modeling the Efficacy of Natalizumab in Multip...      NaN   
19  Immunoadsorption and plasma exchange&#8212;Eff...      NaN   

                                             Patients 1st_line/2nd_line  \
0                                          Patients .          2nd Line   
1   Patients  with clinically isolated syndrome (C...          2nd Line   
2   Patients  do not respond to corticosteroids or...               NaN   
3   Patients  in the first year following diagnosi...          1st Line   
4   Patients  lose the ability to walk and also ha...               NaN   
5              Patients  and their relatives [ 1‐3 ].               NaN   
6   Patients , 12  whereas antibodies against GQ1b...          2nd Line   
7   Patients  taking natalizumab receive treatment...               NaN   
8   Patients  were either non‐responders to anti‐T...               NaN   
9   Patients  with multiple sclerosis have their f...               NaN   
10  Patients  with relapsing−remitting (RR) multip...               NaN   
11                                         Patients .          1st Line   
12                                         Patients .          1st Line   
13  Patients  living with MS, but has also enhance...          2nd Line   
14  Patients  with large

In [130]:
# new_mol_df.to_excel('/Users/savioel2/Desktop/HawkAISandoz/2022_release/Insights Module/interim_qlik/metrics/all_BASDAI.xlsx')



In [131]:
test_df[['ORR_short','ORR']][test_df['ORR_short']!='NA'].head(60)

Empty DataFrame
Columns: [ORR_short, ORR]
Index: []

In [117]:
test_df[['OS_short','OS_month']][test_df['OS_short']!='NA'].head(60)

OS_short OS_month
636                                                None       NA
637                                                None       NA
638                                                None       NA
639                                                None       NA
640   [OS) criteria, were as follows: Pu2009=u2009pa...       NA
641                                                None       NA
642                                                None       NA
643                                                None       NA
644                                                None       NA
645                                                None       NA
646                                                None       NA
647                                                None       NA
5549                                               None       NA
5550                                               None       NA
5551                                               None       NA
5552                                               None       NA
5553                                               None       NA
5554                                               None       NA
5555                                               None       NA
5556                                               None       NA
5557                                               None       NA
5558                                               None       NA
5559                                               None       NA
5560                                               None       NA
5561                                               None       NA
5562                                               None       NA
5563                                               None       NA
5564                                               None       NA
5565                                               None       NA
5566                                               None       NA
5567                                               None       NA
5568                                               None       NA
5569                                               None       NA
5570                                               None       NA
5571                                               None       NA
5572                                               None       NA
5573                                               None       NA
5574                                               None       NA
5575                                               None       NA
5576                                               None       NA
5577                                               None       NA
5578                                               None       NA
5579                                               None       NA
5580                                               None       NA
5581                                               None       NA
5582                                               None       NA
5583                                               None       NA
5584                                               None       NA
5585                                               None       NA
5586                                               None       NA
5587                                               None       NA
5588                                               None       NA
5589                                               None       NA
7867                                               None       NA
7868                                               None       NA
7869                                               None       NA
7870                                               None       NA
7871                                               None       NA
7872                                               None       NA
7873                                               None       NA

In [133]:
test_df[['OS_short','OS_percentage']][test_df['OS_short']!='NA'].head(60)

OS_short  \
636                                                                                                                                                                                                                                                                                                                         None   
637                                                                                                                                                                                                                                                                                                                         None   
638                                                                                                                                                                                                                                                                                                                         None   
639                                                                                                                                                                                                                                                                                                                         None   
640   [OS) criteria, were as follows: Pu2009=u2009patients of all ages with mild, moderate, or severe hemophilia A with or without inhibitors, where mild hemophilia was defined as having 6–40% of normal factor levels in the blood, moderate was defined as 1–5% of normal factor levels, and severe was defined as below 1%]   
641                                                                                                                                                                                                                                                                                                                         None   
642                                                                                                                                                                                                                                                                                                                         None   
643                                                                                                                                                                                                                                                                                                                         None   
644                                                                                                                                                                                                                                                                                                                         None   
645                                                                                                                                                                                                                                                                                                                         None   
646                                                                                                                                                                                                                                                                                                                         None   
647                                                                                                                                                                                                                                                                                                                         None   
5549                                                                                                

In [134]:
test_df[['PFS_short','PFS_percentage']][test_df['PFS_short']!='NA'].head(60)

PFS_short PFS_percentage
636       None             NA
637       None             NA
638       None             NA
639       None             NA
640       None             NA
641       None             NA
642       None             NA
643       None             NA
644       None             NA
645       None             NA
646       None             NA
647       None             NA
5549      None             NA
5550      None             NA
5551      None             NA
5552      None             NA
5553      None             NA
5554      None             NA
5555      None             NA
5556      None             NA
5557      None             NA
5558      None             NA
5559      None             NA
5560      None             NA
5561      None             NA
5562      None             NA
5563      None             NA
5564      None             NA
5565      None             NA
5566      None             NA
5567      None             NA
5568      None             NA
5569      None             NA
5570      None             NA
5571      None             NA
5572      None             NA
5573      None             NA
5574      None             NA
5575      None             NA
5576      None             NA
5577      None             NA
5578      None             NA
5579      None             NA
5580      None             NA
5581      None             NA
5582      None             NA
5583      None             NA
5584      None             NA
5585      None             NA
5586      None             NA
5587      None             NA
5588      None             NA
5589      None             NA
7867      None             NA
7868      None             NA
7869      None             NA
7870      None             NA
7871      None             NA
7872      None             NA
7873      None             NA

In [135]:
test_df[['PFS_short','PFS_month']][test_df['PFS_short']!='NA'].head(60)

PFS_short PFS_month
636       None        NA
637       None        NA
638       None        NA
639       None        NA
640       None        NA
641       None        NA
642       None        NA
643       None        NA
644       None        NA
645       None        NA
646       None        NA
647       None        NA
5549      None        NA
5550      None        NA
5551      None        NA
5552      None        NA
5553      None        NA
5554      None        NA
5555      None        NA
5556      None        NA
5557      None        NA
5558      None        NA
5559      None        NA
5560      None        NA
5561      None        NA
5562      None        NA
5563      None        NA
5564      None        NA
5565      None        NA
5566      None        NA
5567      None        NA
5568      None        NA
5569      None        NA
5570      None        NA
5571      None        NA
5572      None        NA
5573      None        NA
5574      None        NA
5575      None        NA
5576      None        NA
5577      None        NA
5578      None        NA
5579      None        NA
5580      None        NA
5581      None        NA
5582      None        NA
5583      None        NA
5584      None        NA
5585      None        NA
5586      None        NA
5587      None        NA
5588      None        NA
5589      None        NA
7867      None        NA
7868      None        NA
7869      None        NA
7870      None        NA
7871      None        NA
7872      None        NA
7873      None        NA

In [136]:
print("len is - " + str(len(set(test_df['ORR'].tolist()))))

len is - 1


In [137]:
print(set(test_df['sPGA'].tolist()))

{''}


In [149]:
short_df.head()

Unnamed: 0.4                                       XML  \
0          34.0  2da20cd7-0125-4728-a219-7cde1329c3e0.xml   
1          35.0  b979da00-7c42-457e-b0d4-900c4844459c.xml   
2          36.0  ecd677c9-8722-4314-aa15-9c956b38bbd3.xml   
3          37.0  53c91dea-5b3c-48fe-ba40-536c77e50012.xml   
4          38.0  d1a3a84e-d66f-439c-9874-3c3fdfaf6403.xml   

                                                                                                                                                                                                    Title  \
0                                                                                                                                  Review article: anti-adhesion therapies for inflammatory bowel disease   
1                                                                                      Treatment options for patients with multiple sclerosis who have a suboptimal response to interferon-&#946; therapy   
2                                                                  Intravenous immunoglobulin for maintenance treatment of multifocal motor neuropathy: A multi-center, open-label, 52-week phase 3 trial   
3                                        Natalizumab improves ambulation in relapsing&#8722;remitting multiple sclerosis: results from the prospective TIMER study and a retrospective analysis of AFFIRM   
4  Application of Quantitative Pharmacology Approaches in Bridging Pharmacokinetics and Pharmacodynamics of Domagrozumab From Adult Healthy Subjects to Pediatric Patients With Duchenne Muscular Disease   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [150]:
short_df.columns

Index(['Unnamed: 0.4', 'XML', 'Title', 'Full_body_text', 'Abstract_full',
       'DOI', 'URL', 'Molecule', 'Publisher_type', 'Molecule_text',
       ...
       'TSS', 'MRI', 'PASI', 'DAS', 'ES', 'IGA', 'CRP', 'NRS', 'ASAS',
       'BASDAI'],
      dtype='object', length=162)

In [151]:
short_df.shape

(21787, 162)

In [118]:
short_df.shape

(21787, 161)

In [198]:
short_df.shape

(21787, 161)

In [200]:
short_df.columns.tolist()

['XML',
 'Title',
 'Full_body_text',
 'Abstract_full',
 'DOI',
 'URL',
 'Molecule',
 'Publisher_type',
 'Molecule_text',
 'Intervention',
 'STRENGTH',
 'DRUG',
 'Molecule_in_Title',
 'DISEASE_abstract',
 'DISEASE_title',
 'DISEASE',
 'Title_short',
 'combined_text',
 '1st_line/2nd_line',
 'Patients',
 'ORR_abstract',
 'ORR_fulltext',
 'ORR_short',
 'OS_short',
 'PFS_short',
 'Comparator',
 'Arms',
 'Mono/Combi',
 'phase',
 'PHASES',
 'phase2',
 'PHASES2',
 'Age',
 'Age_Number',
 'age_groups',
 'Age2',
 'Age2_value',
 'Age2_number',
 'Age2_groups',
 'Unnamed: 0.3',
 'Unnamed: 0.2',
 'Unnamed: 0.1',
 'Unnamed: 0',
 'phase21',
 'Intervention2',
 'DOSAGE',
 'Number of evaluable subjects',
 'ORR',
 'OS_percentage',
 'OS_month',
 'PFS_percentage',
 'PFS_month',
 'DOSAGE_VAL',
 'exacerbation_abstract',
 'exacerbation_fulltext',
 'exacerbation_short',
 'NPS_abstract',
 'NPS_fulltext',
 'NPS_short',
 'ARR_abstract',
 'ARR_fulltext',
 'ARR_short',
 'EDSS_abstract',
 'EDSS_fulltext',
 'EDSS_short

In [197]:
short_df.Molecule.value_counts()

Adalimumab       4235
Etanercept       3934
Rituximab        1926
Nivolumab        1829
Pembrolizumab    1755
Atezolizumab     1072
Ipilimumab        873
Tocilizumab       721
ipilimumab        543
Ustekinumab       435
Durvalumab        427
Mepolizumab       290
Canakinumab       289
Tremelimumab      275
Daratumumab       272
Avelumab          271
Dupilumab         227
Ofatumumab        222
Ixekizumab        206
Pertuzumab        198
Secukinumab       186
Natalizumab       150
Daclizumab        149
Benralizumab      147
Brodalumab        145
Guselkumab        141
Ocrelizumab       108
Sarilumab          96
Reslizumab         96
Risankizumab       92
Tildrakizumab      86
Emicizumab         76
Sintilimab         68
Cemiplimab         68
Isatuximab         65
Tislelizumab       52
Quavonlimab        27
Tralokinumab       17
Spartalizumab      14
Trastuzumab         4
Name: Molecule, dtype: int64

In [205]:
short_df

XML  \
0     2da20cd7-0125-4728-a219-7cde1329c3e0.xml   
1     b979da00-7c42-457e-b0d4-900c4844459c.xml   
2     ecd677c9-8722-4314-aa15-9c956b38bbd3.xml   
3     53c91dea-5b3c-48fe-ba40-536c77e50012.xml   
4     d1a3a84e-d66f-439c-9874-3c3fdfaf6403.xml   
5     fe4f166b-c4ee-4356-8225-b67f3c0bb8ca.xml   
6     9afd06e7-31b1-4eb9-9e9c-07782b63400d.xml   
7     468cb4e3-6cec-4ff8-85fa-7e1eeb87f0fd.xml   
8     c11b4865-ceec-4566-837f-c62b7f65c993.xml   
9     edf10fd7-46d0-4c41-a0f3-56451b8e5124.xml   
10    cdab46fb-e282-421f-a3eb-cef7d3aab403.xml   
11    b58aa178-fe45-4fb0-8b27-604d69dd7b21.xml   
12    b0d4787a-7f52-4850-a120-d1d3567b3cd9.xml   
13    c90d9fcf-0021-47b7-af91-a91530f107ef.xml   
14    3cba0e7d-164e-47e1-9acc-c2f58906cf37.xml   
15    0749ac49-4bff-4210-b1cf-d6152db6fa23.xml   
16    ed33c249-b022-4474-a7af-8a5072ca7e96.xml   
17    634ba728-aa7a-44f4-be49-be1a18e4a354.xml   
18    530caae7-74a9-4c3b-9977-ed62f918a16c.xml   
19    39a8d70a-492e-4b6a-b2a9-4683fcd41f8f.xml   
20    ee27a2f0-953c-4376-8374-c483aee80664.xml   
21    9cc217c2-56df-4582-950e-e37ebd4b9326.xml   
22    f28c0497-3a2c-4da6-a9a8-f88628529aba.xml   
23    f35ef6ea-4a6e-461e-b6c1-972039914957.xml   
24    64379b97-4601-4821-bdb5-b310c6c6e67e.xml   
25    58d5546d-f559-43ac-bb94-2d4ad2d40d2a.xml   
26    b129cc5e-285f-4928-a560-a03a2d7c7a22.xml   
27    cb74abed-5ed4-4029-9117-078efbce9780.xml   
28    b06c6c50-069f-4e22-8701-3c657477de1d.xml   
29    f5b54f94-638b-48e8-94c2-33288d13e506.xml   
...                                        ...   
8139  fa1ce4bd-0435-4485-8b25-a8a1e20a825f.xml   
8140  fa5a8c19-d879-450b-a4cc-b1713899fe8b.xml   
8141  fa8059de-5ff1-49ac-aff7-9b52de669a9f.xml   
8142  faa15633-9789-4742-9c6e-e5feff7ae6fb.xml   
8143  fabd969e-d1a1-4e67-a009-e5624bb2534c.xml   
8144  fac56fbd-b582-4e88-a18b-3fe935dd52c2.xml   
8145  facde1b2-65bc-4dac-833e-0d7e578b38aa.xml   
8146  fad08045-7720-4297-9388-07a558201c85.xml   
8147  fb0c5b81-591a-4434-a3a3-0bad12be99f3.xml   
8148  fb2177d1-b7d5-4632-9013-5d83162dd111.xml   
8149  fb62cbb2-8867-44c8-8907-bf458f6e5b94.xml   
8150  fb8611c5-21b8-4ba1-9fe9-c60f75a5ceb9.xml   
8151  fb91d08a-22c4-4963-896d-e2fe64cc964a.xml   
8152  fb9b0a8a-0e1d-4ea0-98d1-fbafe67d36cf.xml   
8153  fba562cd-37da-4b74-b501-d6f22d04bddd.xml   
8154  fba72093-bb09-4de1-a691-971c0d263053.xml   
8155  fba74847-62a6-499e-8d50-7e42045a6132.xml   
8156  fbb324d7-1502-4b9e-9e79-d5df613165eb.xml   
8157  fbca531d-af7b-403f-b0d2-80ca65ca5f8a.xml   
8158  fbdc4654-9ed1-41a9-aeb2-4fa7df5eb9a1.xml   
8159  fbe5db4c-bff0-4e31-9a6f-7e62386765f6.xml   
8160  fbf16690-32e7-40ee-9b56-0bb416c0f0a9.xml   
8161  fbfc8576-6ded-4324-9c05-b5d9e63385a3.xml   
8162  fc03532a-7473-4553-94a6-57a996b9003e.xml   
8163  fc09dd4f-1489-4dc7-b9c3-08047aafcb1e.xml   
8164  fc23a41e-50fa-421c-b9f5-ceef8e86743a.xml   
8165  fc49ba3a-1b7d-4484-a45c-cae5cc900101.xml   
8166  fc82b4ea-6d8f-41ec-941d-c77d1db1fc31.xml   
8167  fc97341b-f488-4117-a093-bf3c1b1a70d2.xml   
8168  fc9aa26f-355c-4720-bbfa-effa75b61916.xml   

                                                  Title  \
0     Review article: anti-adhesion therapies for in...   
1     Treatment options for patients with multiple s...   
2     Intravenous immunoglobulin for maintenance tre...   
3     Natalizumab improves ambulation in relapsing&#...   
4     Application of Quantitative Pharmacology Appro...   
5     Current and emerging disease-modulatory therap...   
6     Immunoadsorption and plasma exchange&#8212;Eff...   
7     Modeling the Efficacy of Natalizumab in Multip...   
8     Clinical pharmacology, safety and effects of a...   
9     Paediatric multiple sclerosis: a new era in di...   
10    Effect of natalizumab on brain atrophy and dis...   
11    Therapeutic Application of Monoclonal Antibodi...   
12    A Randomized Trial Evaluating Various Administ...   
13    Established and novel disease-modifying treatm...   
14    Wielding the Double-Edged Sword of Inflammatio...  

# write shortdf to new table in rds - insights_full_data

In [206]:
writer = pd.ExcelWriter('input_data/input_data/insights_full_data.xlsx', engine='xlsxwriter',options={'strings_to_urls': False})
short_df.to_excel(writer, sheet_name='All_Messages')
writer.close()

In [154]:
short_df.to_excel('/Users/savioel2/Desktop/HawkAISandoz/2022_release/Insights Module/full_data/all_molecules.xlsx')



/Users/savioel2/opt/anaconda3/lib/python3.8/site-packages/xlsxwriter/worksheet.py:941: UserWarning: Ignoring URL 'http://rehabilitation.cochrane.org%20%20The%20aim%20of%20this%20commentary%20is%20to%20discuss%20the%20published%20Cochrane%20Review%20“%20TNF‐α%20blockers%20for%20the%20treatment%20of%20Kawasaki%20disease%20in%20children%201%20”%20by%20Yamaji%20et%20al.,%20under%20the%20direct%20supervision%20of%20Cochrane%20Review%20Group.%20This%20Cochrane%20Corner%20is%20produced%20in%20agreement%20with%20the%20%20International%20Journal%20of%20Rheumatic%20Diseases%20%20by%20Cochrane%20Rehabilitation.%20%20%20%20BACKGROUND%20%20Kawasaki%20disease%20(KD)%20is%20an%20acute%20vasculitis%20with%20unknown%20etiology%20affecting%20small‐%20and%20medium‐sized%20blood%20vessels,%20particularly%20coronary%20arteries,%20which%20usually%20occurs%20in%20early%20childhood%20up%20to%205%20years%20of%20age.%202%20%20This%20condition%20is%20characterized%20by%20main%20clinical%20features%20including%20

In [138]:
# short_df['Insights_ID']

In [139]:
# new_mol_df.to_excel('/Users/savioel2/Desktop/HawkAISandoz/2022_release/Insights Module/interim_qlik/metrics/molecules_orrospfs.xlsx')



In [ ]:
def FMF_sentence(text): 
    oslen = []
    worddistance_list = []
    text = str(text).strip()
    try:
        short_sentence_list=sent_tokenize(text)
        short_cleaned_relevant_lst = [x.replace('\n','').replace('\\xa','') for x in short_sentence_list]           
        combinations_list = ['FMF.*\d+\.\d{2,}','FMF.*\d+\.\d{2,}%','familial Mediterranean fever.*\d+\.\d{2,}']
        

        for sent in short_cleaned_relevant_lst:        
            for param in combinations_list:
                word_length = [len(i.split())-2 for i in re.findall(param,sent)] 
                if word_length:
                    worddistance_list.append(word_length[0])
            if len(worddistance_list)>0:
                min_value = min([i for i in worddistance_list if i > 0])
                if min_value:
                    min_sent_list = []
                    for sent in short_cleaned_relevant_lst:
                        for param in combinations_list:
                            word_length = [len(i.split())-2 for i in re.findall(param,sent)]                                   
                            if word_length and word_length[0] == min_value:
                                oslen.append(sent)
                                os_short = re.findall(param,sent)
                            else:
                                'NA'
                else:
                    return 'NA'
                if oslen:
                    return os_short
                else:
                    'NA'
            else:
                'NA' 
    except:
            return 'NA'
        
# short_df['ACR_abstract'] = short_df.Abstract_full.apply(ACR_sentence)
# short_df['ACR_fulltext'] = short_df.Full_body_text.apply(ACR_sentence)

# short_df['ACR_short'] = short_df.apply(lambda x: lencheck(x.ACR_abstract, x.ACR_fulltext), axis=1)

In [ ]:
F